# 02 – Feature Engineering & Risk Labels

This notebook consumes the synthetic IIoT event stream and master data generated in `01_synthetic_iot_data_generator.ipynb` and turns it into model-ready features and labels.

**Goals**

- Join **event logs** with **asset** and **site** metadata.
- Engineer time-based features at an `asset_id × local_date` grain (counts, durations, severities).
- Create **risk labels**, for example:
  - *next-day failure* (binary)
  - *high-risk operating window* based on recent anomalies.
- Produce tidy feature tables saved under `data/interim/` and `data/results/` for:
  - downstream model notebooks (e.g., classification, survival),
  - and dashboard backends (FastAPI + DuckDB).


In [1]:
# ============================================================
# Notebook-wide warning controls (keep output clean)
# ============================================================
import warnings

# Show each warning at most once (optional; keeps noise down without hiding everything)
warnings.simplefilter("once")

# Pandas: DatetimeProperties.to_pydatetime deprecation warning
# (message text varies slightly across pandas versions, so match broadly)
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r".*to_pydatetime.*deprecated.*",
    module=r"pandas\..*",
)

# Pandas: GroupBy.apply deprecation warning (grouping columns behavior changing)
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r".*DataFrameGroupBy\.apply operated on the grouping columns.*",
    module=r"pandas\..*",
)

# Optional: if warnings are emitted from your notebook file path rather than pandas,
# also ignore them regardless of module.
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r".*to_pydatetime.*deprecated.*",
)
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=r".*DataFrameGroupBy\.apply operated on the grouping columns.*",
)

In [2]:
#============================================================
# Cell 1 — Setup: imports, RNG seed, project paths, and run metadata
#============================================================

from __future__ import annotations

import os
import re
import sys
import json
import time
from pathlib import Path
from datetime import datetime, timezone

import numpy as np
import pandas as pd

# sklearn (feature engineering + reproducible preprocessing)
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# optional: artifact persistence
import joblib

# -----------------------------
# Reproducibility
# -----------------------------
SEED = 42
rng = np.random.default_rng(SEED)

# -----------------------------
# Resolve project root robustly
# -----------------------------
# Strategy:
# 1) Prefer current working directory if it looks like the repo root
# 2) Otherwise, walk upward looking for common repo markers
cwd = Path.cwd()

REPO_MARKERS = ["pyproject.toml", "environment.yml", ".git", "README.md"]
def find_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(10):
        if any((cur / m).exists() for m in REPO_MARKERS):
            return cur
        if cur.parent == cur:
            break
        cur = cur.parent
    return start.resolve()

PROJECT_ROOT = find_repo_root(cwd)

# -----------------------------
# Standard directories
# -----------------------------
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
INTERIM_DIR = DATA_DIR / "interim"
PROCESSED_DIR = DATA_DIR / "processed"
RESULTS_DIR = DATA_DIR / "results"

RUN_TS = datetime.now(timezone.utc).strftime("%Y%m%dT%H%M%SZ")
OUT_DIR = PROCESSED_DIR / "feature_engineering" / RUN_TS
OUT_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------------
# Basic environment diagnostics (lightweight, helpful for reproducibility)
# -----------------------------
env_info = {
    "run_ts_utc": RUN_TS,
    "project_root": str(PROJECT_ROOT),
    "python": sys.version.replace("\n", " "),
    "pandas": pd.__version__,
    "numpy": np.__version__,
}

print("PROJECT_ROOT:", PROJECT_ROOT)
print("OUT_DIR:", OUT_DIR)
print(json.dumps(env_info, indent=2))

# Persist run metadata for later audit/debug
with open(OUT_DIR / "run_metadata.json", "w") as f:
    json.dump(env_info, f, indent=2)


PROJECT_ROOT: /home/parallels/projects/gmp-packaging-risk-analytics
OUT_DIR: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z
{
  "run_ts_utc": "20251220T211621Z",
  "project_root": "/home/parallels/projects/gmp-packaging-risk-analytics",
  "python": "3.11.14 | packaged by conda-forge | (main, Oct 22 2025, 22:39:18) [GCC 14.3.0]",
  "pandas": "2.3.3",
  "numpy": "2.3.5"
}


### What Cell 1 Just Did

This cell set up the notebook’s working environment and made the run reproducible. It imported the core libraries used throughout the workflow, set the random seed so splits and model behavior are repeatable, and defined the project paths (including the run-scoped output directory). It also established the run metadata (e.g., timestamped `OUT_DIR`) so all artifacts from this execution are saved together and can be traced back to a single run.


In [4]:
# ============================================================
# Cell 2 — Locate data folders, inventory files, and load the *events* dataset
# ============================================================

from pathlib import Path
from datetime import datetime, timezone
import pandas as pd

# --- Paths ----------------------------------------------------
PROJECT_ROOT = Path(PROJECT_ROOT)  # comes from Cell 1
RAW_DIR = PROJECT_ROOT / "data" / "raw"
INTERIM_DIR = PROJECT_ROOT / "data" / "interim"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

print(f"RAW_DIR exists: {RAW_DIR.exists()} -> {RAW_DIR}")
print(f"INTERIM_DIR exists: {INTERIM_DIR.exists()} -> {INTERIM_DIR}")
print(f"PROCESSED_DIR exists: {PROCESSED_DIR.exists()} -> {PROCESSED_DIR}")

# --- Helpers --------------------------------------------------
def _mtime_utc(p: Path) -> str:
    return datetime.fromtimestamp(p.stat().st_mtime, tz=timezone.utc).strftime("%Y-%m-%d %H:%M:%S")

def list_top_files(folder: Path, top_n: int = 10) -> pd.DataFrame:
    if not folder.exists():
        return pd.DataFrame(columns=["path", "bytes", "modified_utc"])
    files = [p for p in folder.rglob("*") if p.is_file()]
    if not files:
        return pd.DataFrame(columns=["path", "bytes", "modified_utc"])
    rows = [{
        "path": str(p.relative_to(PROJECT_ROOT)),
        "bytes": p.stat().st_size,
        "modified_utc": _mtime_utc(p)
    } for p in files]
    df = pd.DataFrame(rows).sort_values(["modified_utc", "bytes"], ascending=[False, False]).head(top_n)
    return df.reset_index(drop=True)

def find_candidates(base_dirs: list[Path]) -> pd.DataFrame:
    exts = {".csv", ".parquet"}
    files: list[Path] = []
    for d in base_dirs:
        if d.exists():
            files.extend([p for p in d.rglob("*") if p.is_file() and p.suffix.lower() in exts])

    if not files:
        return pd.DataFrame(columns=["path", "bytes", "modified_utc", "abs_path"])

    rows = [{
        "path": str(p.relative_to(PROJECT_ROOT)),
        "bytes": p.stat().st_size,
        "modified_utc": _mtime_utc(p),
        "abs_path": str(p)
    } for p in files]

    df = pd.DataFrame(rows).sort_values(["modified_utc", "bytes"], ascending=[False, False]).reset_index(drop=True)
    return df

# --- Inventory ------------------------------------------------
print("\nTop files in data/raw:\n")
df_raw = list_top_files(RAW_DIR, top_n=10)
print(df_raw if not df_raw.empty else "  (no files found)")

print("\nTop files in data/interim:\n")
df_interim = list_top_files(INTERIM_DIR, top_n=10)
print(df_interim if not df_interim.empty else "  (no files found)")

print("\nTop files in data/processed:\n")
df_processed = list_top_files(PROCESSED_DIR, top_n=10)
print(df_processed if not df_processed.empty else "  (no files found)")

# --- Candidate discovery -------------------------------------
candidates = find_candidates([RAW_DIR, INTERIM_DIR, PROCESSED_DIR])
print(f"\nFound {len(candidates)} CSV/Parquet candidate(s) across data/ folders.\n")
if not candidates.empty:
    print(candidates[["path", "bytes", "modified_utc"]].head(20))
else:
    raise FileNotFoundError("No CSV/Parquet files found under data/raw, data/interim, or data/processed.")

# --- FORCE the correct input: events parquet ------------------
RAW_EVENTS_PATH = RAW_DIR / "iot_events.parquet"
if not RAW_EVENTS_PATH.exists():
    raise FileNotFoundError(f"Expected events file not found: {RAW_EVENTS_PATH}")

selected_path = RAW_EVENTS_PATH
print("\nForcing events input:")
print(f" -> {selected_path}")

# --- Load a preview (safe) -----------------------------------
if selected_path.suffix.lower() == ".csv":
    df_preview = pd.read_csv(selected_path)
elif selected_path.suffix.lower() == ".parquet":
    df_preview = pd.read_parquet(selected_path)
else:
    raise ValueError(f"Unsupported file type: {selected_path.suffix}")

print(f"Shape: {df_preview.shape}")
print("\nColumns:")
print(list(df_preview.columns))
print("\nHead:")
display(df_preview.head())


RAW_DIR exists: True -> /home/parallels/projects/gmp-packaging-risk-analytics/data/raw
INTERIM_DIR exists: True -> /home/parallels/projects/gmp-packaging-risk-analytics/data/interim
PROCESSED_DIR exists: True -> /home/parallels/projects/gmp-packaging-risk-analytics/data/processed

Top files in data/raw:

                          path     bytes         modified_utc
0  data/raw/iot_events.parquet  11644913  2025-12-11 00:05:29
1   data/raw/assets_master.csv      6521  2025-12-11 00:05:29
2    data/raw/sites_master.csv       208  2025-12-11 00:05:29

Top files in data/interim:

  (no files found)

Top files in data/processed:

                                                path   bytes  \
0  data/processed/feature_engineering/20251220T21...     248   
1  data/processed/feature_engineering/20251219T20...    1769   
2  data/processed/feature_engineering/20251219T20...    1192   
3  data/processed/feature_engineering/20251219T20...  672073   
4  data/processed/feature_engineering/20251219T

,event_id,ts_utc,site_id,line_id,asset_id,asset_type,is_legacy,event_kind,metric_name,metric_unit,metric_value,severity,incident_type,message,ts_local,local_date,local_hour,ts_local_str
0,E0000000001,2025-11-27 00:05:18.868743+00:00,S1,S1-L2,A0001,blister_packer,False,telemetry,temp_c,C,29.490142,NaN,None,None,2025-11-26 19:05:18.868743-05:00,2025-11-26,19,2025-11-26 19:05:18 EST
1,E0000000002,2025-11-27 00:05:18.868743+00:00,S1,S1-L2,A0001,blister_packer,False,telemetry,humidity_rh,%,43.893886,NaN,None,None,2025-11-26 19:05:18.868743-05:00,2025-11-26,19,2025-11-26 19:05:18 EST
2,E0000000003,2025-11-27 00:10:18.868743+00:00,S1,S1-L2,A0001,blister_packer,False,telemetry,humidity_rh,%,50.181508,NaN,None,None,2025-11-26 19:10:18.868743-05:00,2025-11-26,19,2025-11-26 19:10:18 EST
3,E0000000004,2025-11-27 00:15:18.868743+00:00,S1,S1-L2,A0001,blister_packer,False,telemetry,reject_rate_pct,%,1.561515,NaN,None,None,2025-11-26 19:15:18.868743-05:00,2025-11-26,19,2025-11-26 19:15:18 EST
4,E0000000005,2025-11-27 00:15:18.868743+00:00,S1,S1-L2,A0001,blister_packer,False,telemetry,vibration_mm_s,mm/s,1.789262,NaN,None,None,2025-11-26 19:15:18.868743-05:00,2025-11-26,19,2025-11-26 19:15:18 EST


### What Cell 2 Just Did
This cell located the project’s expected data directories (raw and processed) and verified the key input files needed for the workflow. It then inventoried what was available on disk so we can confirm we’re running against the correct dataset for this run. Finally, it loaded the primary *events* dataset into a DataFrame and printed basic shape/preview details so downstream feature engineering and modeling steps have a clean, validated starting point.


In [5]:
# ============================================================
# Cell 3 — Standardize EVENTS schema, parse timestamps, enrich with master data, save parquet
# ============================================================

from pathlib import Path
import pandas as pd

# --- Inputs we expect from earlier cells ---------------------
# PROJECT_ROOT, RAW_DIR, OUT_DIR should exist from Cells 1–2
PROJECT_ROOT = Path(PROJECT_ROOT)
RAW_DIR = PROJECT_ROOT / "data" / "raw"
OUT_DIR = Path(OUT_DIR)

# This must be the EVENTS dataframe from Cell 2 (Shape ~588k x 18)
# If you used the amended Cell 2 I gave you, df_preview is the events parquet content.
if "df_preview" not in globals():
    raise NameError("Expected df_preview from Cell 2 (events dataframe). Re-run Cell 2 first.")

events_df = df_preview.copy()

# --- Basic sanity --------------------------------------------
print(f"Incoming events_df shape: {events_df.shape}")
expected_cols = {
    "event_id","ts_utc","site_id","line_id","asset_id","asset_type","is_legacy",
    "event_kind","metric_name","metric_unit","metric_value","severity","incident_type",
    "message","ts_local","local_date","local_hour","ts_local_str"
}
missing = sorted(list(expected_cols - set(events_df.columns)))
if missing:
    print("WARNING: events_df missing expected columns:", missing)

# --- Timestamp handling --------------------------------------
# Prefer ts_utc; fall back to ts_local if needed
ts_col = None
if "ts_utc" in events_df.columns:
    ts_col = "ts_utc"
elif "ts_local" in events_df.columns:
    ts_col = "ts_local"

if ts_col is None:
    print("No obvious timestamp column found. Proceeding without time-derived features.")
else:
    # Ensure datetime dtype
    events_df[ts_col] = pd.to_datetime(events_df[ts_col], errors="coerce", utc=True)
    if events_df[ts_col].isna().any():
        na_ct = int(events_df[ts_col].isna().sum())
        print(f"WARNING: {na_ct} rows have invalid {ts_col} after parsing.")

    # If ts_local exists, parse it too (keep timezone info if present)
    if "ts_local" in events_df.columns:
        events_df["ts_local"] = pd.to_datetime(events_df["ts_local"], errors="coerce")
    # Derive local_date/local_hour if missing
    if "ts_local" in events_df.columns and events_df["ts_local"].notna().any():
        if "local_date" not in events_df.columns or events_df["local_date"].isna().any():
            events_df["local_date"] = events_df["ts_local"].dt.date.astype("string")
        if "local_hour" not in events_df.columns or events_df["local_hour"].isna().any():
            events_df["local_hour"] = events_df["ts_local"].dt.hour.astype("Int64")

    print(f"Timestamp column recognized: {ts_col}")

# --- Standardize key dtypes ----------------------------------
for c in ["event_id","site_id","line_id","asset_id","asset_type","event_kind","metric_name","metric_unit","incident_type","message","ts_local_str"]:
    if c in events_df.columns:
        events_df[c] = events_df[c].astype("string")

if "is_legacy" in events_df.columns:
    # robust bool coercion
    events_df["is_legacy"] = events_df["is_legacy"].astype("boolean")

if "metric_value" in events_df.columns:
    events_df["metric_value"] = pd.to_numeric(events_df["metric_value"], errors="coerce")

if "severity" in events_df.columns:
    events_df["severity"] = pd.to_numeric(events_df["severity"], errors="coerce")

# --- Enrich with master data ---------------------------------
added_cols_assets = 0
added_cols_sites = 0

assets_path = RAW_DIR / "assets_master.csv"
sites_path  = RAW_DIR / "sites_master.csv"

if assets_path.exists():
    assets = pd.read_csv(assets_path)
    # normalize key type
    assets["asset_id"] = assets["asset_id"].astype("string")
    before_cols = set(events_df.columns)
    events_df = events_df.merge(assets, on="asset_id", how="left", suffixes=("", "_assetmaster"))
    after_cols = set(events_df.columns)
    added_cols_assets = len(sorted(list(after_cols - before_cols)))
    print(f"Joined assets_master.csv on asset_id. Added {added_cols_assets} column(s).")
else:
    print(f"WARNING: missing {assets_path}; skipping asset enrichment.")

if sites_path.exists():
    sites = pd.read_csv(sites_path)
    sites["site_id"] = sites["site_id"].astype("string")
    before_cols = set(events_df.columns)
    events_df = events_df.merge(sites, on="site_id", how="left", suffixes=("", "_sitemaster"))
    after_cols = set(events_df.columns)
    added_cols_sites = len(sorted(list(after_cols - before_cols)))
    print(f"Joined sites_master.csv on site_id. Added {added_cols_sites} column(s).")
else:
    print(f"WARNING: missing {sites_path}; skipping site enrichment.")

print(f"Shape after standardization/enrichment: {events_df.shape}")

# --- Quick null checks ---------------------------------------
for key in ["asset_id","site_id"]:
    if key in events_df.columns:
        print(f"{key} nulls: {int(events_df[key].isna().sum())}")

# --- Save -----------------------------------------------------
OUT_DIR.mkdir(parents=True, exist_ok=True)
events_out = OUT_DIR / "events_standardized_enriched.parquet"
events_df.to_parquet(events_out, index=False)
print(f"Saved: {events_out}")

# Handy variable for downstream cells
EVENTS_STANDARDIZED_PATH = events_out


Incoming events_df shape: (588681, 18)
Timestamp column recognized: ts_utc
Joined assets_master.csv on asset_id. Added 6 column(s).
Joined sites_master.csv on site_id. Added 2 column(s).
Shape after standardization/enrichment: (588681, 26)
asset_id nulls: 0
site_id nulls: 0
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/events_standardized_enriched.parquet


### What Cell 3 Just Did
This cell standardized the raw EVENTS table into a consistent, analysis-ready schema (renaming columns, normalizing IDs, and coercing key fields to reliable types). It parsed event timestamps into proper timezone-aware datetimes and derived a few convenience time fields for later grouping (e.g., hour/date). Next, it enriched the events by joining in asset/site master data so each record carries stable context (site, line, asset_type, legacy/connectivity/vendor, and any site metadata like timezone). Finally, it saved the cleaned/enriched events dataset to a durable parquet artifact in the current `OUT_DIR`, ensuring downstream cells can reload the exact same canonical dataset without redoing the cleaning and joins.


In [6]:
#============================================================
# Cell 4 — Asset-hour feature table + forward-looking labels + rolling features (no fragmentation)
#============================================================

# Assumes from prior cells:
# - OUT_DIR is set
# - EVENTS_STANDARDIZED_PATH points to events_standardized_enriched.parquet (Cell 3 output)
# - pandas as pd, numpy as np are already imported

import numpy as np
import pandas as pd

# -----------------------------
# 0) Load events
# -----------------------------
events = pd.read_parquet(EVENTS_STANDARDIZED_PATH)
print("Loaded events:", events.shape)

# Ensure ts_utc is timezone-aware, then build an *hourly* UTC bucket that is timezone-naive
events["ts_utc"] = pd.to_datetime(events["ts_utc"], utc=True, errors="coerce")
events["ts_hour_utc"] = events["ts_utc"].dt.floor("h").dt.tz_localize(None)

# -----------------------------
# 1) Build hourly telemetry features (asset-hour)
# -----------------------------
tele = events.loc[events["event_kind"].eq("telemetry")].copy()

tele_cols_keep = [
    "asset_id", "site_id", "line_id", "asset_type", "is_legacy",
    "ts_hour_utc", "metric_name", "metric_value"
]
tele = tele[tele_cols_keep]

# Choose top metrics by volume (keeps feature space bounded)
top_k = 6
top_metrics = (
    tele["metric_name"]
    .value_counts(dropna=True)
    .head(top_k)
    .index
    .tolist()
)
print("Using top metric_name values:", top_metrics)

tele = tele.loc[tele["metric_name"].isin(top_metrics)].copy()

# Aggregate per asset-hour per metric
keys = ["asset_id", "site_id", "line_id", "asset_type", "is_legacy", "ts_hour_utc"]

agg = (
    tele
    .groupby(keys + ["metric_name"], observed=True)["metric_value"]
    .agg(["max", "mean", "std"])
    .reset_index()
)

# Pivot to wide: columns become "<metric>_<stat>"
wide = agg.pivot_table(
    index=keys,
    columns="metric_name",
    values=["max", "mean", "std"],
    aggfunc="first",
    observed=True
)

# Flatten MultiIndex columns
wide.columns = [f"{metric}_{stat}" for stat, metric in wide.columns]
wide = wide.reset_index()

print("\nHourly feature table:", wide.shape)

# -----------------------------
# 2) Build forward-looking label: incident in next H hours for the same asset
# -----------------------------
inc = events.loc[events["event_kind"].eq("incident"), ["asset_id", "ts_utc"]].copy()
inc["ts_utc"] = pd.to_datetime(inc["ts_utc"], utc=True, errors="coerce")
inc["inc_hour_utc"] = inc["ts_utc"].dt.floor("h").dt.tz_localize(None)

print("Incident rows:", inc.shape)

HORIZON_HOURS = 6
h_ns = np.int64(HORIZON_HOURS) * np.int64(3600) * np.int64(1_000_000_000)

# Pre-sort incident hours per asset (as int64 ns for fast search)
inc_map = (
    inc.dropna(subset=["asset_id", "inc_hour_utc"])
       .groupby("asset_id")["inc_hour_utc"]
       .apply(lambda s: np.sort(s.unique()))
)

def label_next_horizon(asset_series: pd.Series, hour_series: pd.Series) -> np.ndarray:
    """
    For each row (asset_id, ts_hour_utc), label 1 if there is an incident for that asset
    in (ts_hour_utc, ts_hour_utc + HORIZON_HOURS] else 0.
    """
    out = np.zeros(len(asset_series), dtype=np.int8)

    # Convert hours to int64 ns once
    hour_ns = hour_series.values.astype("datetime64[ns]").astype(np.int64)

    # Work asset-by-asset
    assets = asset_series.values
    for a in pd.unique(assets):
        if a not in inc_map.index:
            continue
        idx = np.where(assets == a)[0]
        if idx.size == 0:
            continue

        inc_hours = inc_map.loc[a]
        if inc_hours.size == 0:
            continue

        inc_ns = inc_hours.astype("datetime64[ns]").astype(np.int64)

        # For each hour t, find first incident strictly after t
        t_ns = hour_ns[idx]
        pos = np.searchsorted(inc_ns, t_ns, side="right")
        # Candidate incident time
        has = pos < inc_ns.size
        cand = np.empty_like(t_ns)
        cand[has] = inc_ns[pos[has]]

        # Label if candidate <= t + horizon
        within = has & (cand <= (t_ns + h_ns))
        out[idx[within]] = 1

    return out

wide["target"] = label_next_horizon(wide["asset_id"].astype(str), wide["ts_hour_utc"])

# -----------------------------
# 3) Add time-derived features (avoid using raw timestamp as categorical)
# -----------------------------
wide["hour_of_day"] = wide["ts_hour_utc"].dt.hour.astype("int8")
wide["day_of_week"] = wide["ts_hour_utc"].dt.dayofweek.astype("int8")
wide["is_weekend"] = (wide["day_of_week"] >= 5).astype("int8")

# -----------------------------
# 4) Rolling features per asset (mean/std over past windows) — built via concat (no fragmentation)
# -----------------------------
base_metric_cols = [c for c in wide.columns if c.endswith("_mean") or c.endswith("_std") or c.endswith("_max")]
wide = wide.sort_values(["asset_id", "ts_hour_utc"]).reset_index(drop=True)

roll_windows = [3, 6, 12]  # hours
feat_frames = []

# Set index for rolling by time ordering (we already sorted)
for w in roll_windows:
    gb = wide.groupby("asset_id", sort=False)[base_metric_cols]

    roll_mean = gb.rolling(window=w, min_periods=1).mean().reset_index(level=0, drop=True)
    roll_mean.columns = [f"{c}_roll{w}h_mean" for c in base_metric_cols]

    roll_std = gb.rolling(window=w, min_periods=2).std(ddof=0).reset_index(level=0, drop=True)
    roll_std.columns = [f"{c}_roll{w}h_std" for c in base_metric_cols]

    feat_frames.append(roll_mean)
    feat_frames.append(roll_std)

work_df = pd.concat([wide] + feat_frames, axis=1)

# -----------------------------
# 5) Save + quick sanity outputs
# -----------------------------
print("\nwork_df shape:", work_df.shape)
print("target positive rate:", float(work_df["target"].mean()))
print("Unique assets:", work_df["asset_id"].nunique(), "| Hours:", work_df["ts_hour_utc"].nunique())

out_path = OUT_DIR / "model_table_asset_hour.parquet"
work_df.to_parquet(out_path, index=False)
print("Saved:", out_path)

display(work_df.head(5))


Loaded events: (588681, 26)
Using top metric_name values: ['line_speed_u_min', 'pressure_kpa', 'humidity_rh', 'temp_c', 'reject_rate_pct', 'vibration_mm_s']

Hourly feature table: (40372, 24)
Incident rows: (132, 3)

work_df shape: (40372, 136)
target positive rate: 0.019394629941543645
Unique assets: 120 | Hours: 337
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/model_table_asset_hour.parquet


,asset_id,site_id,line_id,asset_type,is_legacy,ts_hour_utc,humidity_rh_max,line_speed_u_min_max,pressure_kpa_max,reject_rate_pct_max,...,pressure_kpa_mean_roll12h_std,reject_rate_pct_mean_roll12h_std,temp_c_mean_roll12h_std,vibration_mm_s_mean_roll12h_std,humidity_rh_std_roll12h_std,line_speed_u_min_std_roll12h_std,pressure_kpa_std_roll12h_std,reject_rate_pct_std_roll12h_std,temp_c_std_roll12h_std,vibration_mm_s_std_roll12h_std
0,A0001,S1,S1-L2,blister_packer,False,2025-11-27 00:00:00,50.181508,159.480320,239.312975,1.561515,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A0001,S1,S1-L2,blister_packer,False,2025-11-27 01:00:00,42.666450,166.306955,226.450898,1.328561,...,0.145070,0.214340,0.742896,0.269058,1.475885,2.793869,2.851629,0.023634,0.321817,0.109494
2,A0001,S1,S1-L2,blister_packer,False,2025-11-27 02:00:00,55.849920,122.176177,229.510903,1.582322,...,2.842041,0.178119,1.152242,0.360385,1.723326,9.135932,7.067055,0.041124,0.355677,0.095318
3,A0001,S1,S1-L2,blister_packer,False,2025-11-27 03:00:00,52.749160,128.927814,228.308042,1.743093,...,2.735178,0.154862,1.266249,0.342250,1.557106,9.496460,6.221731,0.040905,0.332503,0.090824
4,A0001,S1,S1-L2,blister_packer,False,2025-11-27 04:00:00,56.222354,174.761391,219.471849,1.175967,...,5.661321,0.166972,1.184201,0.311439,1.402707,8.659082,5.578015,0.036610,0.830538,0.257654


### What Cell 4 Just Did
This cell reshaped the cleaned/enriched events into an **asset-hour panel** (one row per `asset_id × hour`), which becomes the core modeling table for “what’s happening now” at an hourly cadence. It aggregated telemetry signals into hour-level features (counts plus summary statistics like mean/std/min/max for the selected metrics) and added compact time-context features (hour-of-day, day-of-week, weekend flags, and cyclical sin/cos encodings) so the model can learn routine patterns.

Next, it created a **forward-looking label** by looking ahead from each hour into a defined horizon and marking the row positive if a qualifying incident occurs in that future window (e.g., “incident within next N hours”). This makes the task predictive instead of retrospective.

Finally, it engineered rolling / trailing features (e.g., recent-hour summaries) in a way that avoids pandas fragmentation/performance issues by building feature blocks cleanly and then combining them once. The resulting panel dataset (features + target + identifiers) was persisted to `OUT_DIR` as a reusable parquet artifact for downstream splitting, preprocessing, and modeling.


In [7]:
#============================================================
# Cell 5 — Feature set definition (asset-hour): exclude IDs + raw timestamps, add safe time features,
#          split numeric vs categorical, persist lists
#============================================================

import json
import numpy as np
import pandas as pd

# Start from the labeled frame produced in Cell 4
model_df = work_df.copy()

# -----------------------------
# 0) Ensure ts_hour_utc is parsed correctly and normalized (UTC, timezone-naive)
# -----------------------------
if "ts_hour_utc" not in model_df.columns:
    raise KeyError("Expected ts_hour_utc in model_df from Cell 4, but it was not found.")

# Parse
ts = pd.to_datetime(model_df["ts_hour_utc"], errors="coerce")

if ts.isna().any():
    raise ValueError("ts_hour_utc has NaT after parsing; check Cell 4 output.")

# Normalize:
# - if tz-aware: convert to UTC then drop tz
# - if tz-naive: treat as already-UTC hour buckets and keep tz-naive
try:
    # tz-aware series has .dt.tz; tz-naive will raise AttributeError in some cases
    if getattr(ts.dt, "tz", None) is not None:
        ts = ts.dt.tz_convert("UTC").dt.tz_localize(None)
except Exception:
    # fallback: leave as-is; should already be datetime64[ns]
    pass

model_df["ts_hour_utc"] = ts.astype("datetime64[ns]")  # explicitly timezone-naive

# -----------------------------
# 1) Define ID-like columns to exclude from modeling features
# -----------------------------
id_like = ["asset_id", "site_id", "event_id", "id", "uuid"]
id_cols = [c for c in id_like if c in model_df.columns]

# -----------------------------
# 2) Define leakage columns to exclude (based on prior leakage scan)
# -----------------------------
leakage_cols = [c for c in ["is_legacy", "severity", "incident_type"] if c in model_df.columns]

# -----------------------------
# 3) Exclude raw time columns (we'll add safe derived time features instead)
# -----------------------------
time_cols = [c for c in ["ts_hour_utc", "ts_utc", "ts_local", "ts_local_str", "local_date"] if c in model_df.columns]

# -----------------------------
# 4) Add safe time-derived features (numeric, low-leakage)
#    Reuse if Cell 4 already created these.
# -----------------------------
if "hour_of_day" in model_df.columns:
    model_df["hour_utc"] = model_df["hour_of_day"].astype("int8")
else:
    model_df["hour_utc"] = model_df["ts_hour_utc"].dt.hour.astype("int8")

if "day_of_week" in model_df.columns:
    model_df["dow_utc"] = model_df["day_of_week"].astype("int8")
else:
    model_df["dow_utc"] = model_df["ts_hour_utc"].dt.dayofweek.astype("int8")

if "is_weekend" in model_df.columns:
    model_df["is_weekend_utc"] = model_df["is_weekend"].astype("int8")
else:
    model_df["is_weekend_utc"] = (model_df["dow_utc"] >= 5).astype("int8")

# Cyclical encodings (helps linear models); store as float32
model_df["hour_utc_sin"] = np.sin(2 * np.pi * model_df["hour_utc"] / 24.0).astype("float32")
model_df["hour_utc_cos"] = np.cos(2 * np.pi * model_df["hour_utc"] / 24.0).astype("float32")

model_df["dow_utc_sin"] = np.sin(2 * np.pi * model_df["dow_utc"] / 7.0).astype("float32")
model_df["dow_utc_cos"] = np.cos(2 * np.pi * model_df["dow_utc"] / 7.0).astype("float32")

# -----------------------------
# 5) Separate feature columns by dtype
# -----------------------------
exclude = set(id_cols + ["target"] + leakage_cols + time_cols)

numeric_cols = [
    c for c in model_df.columns
    if c not in exclude and pd.api.types.is_numeric_dtype(model_df[c])
]

categorical_cols = [
    c for c in model_df.columns
    if c not in exclude and not pd.api.types.is_numeric_dtype(model_df[c])
]

# Drop any accidental all-null feature columns (rare, but protects pipelines)
all_null_num = [c for c in numeric_cols if model_df[c].isna().all()]
all_null_cat = [c for c in categorical_cols if model_df[c].isna().all()]
numeric_cols = [c for c in numeric_cols if c not in all_null_num]
categorical_cols = [c for c in categorical_cols if c not in all_null_cat]

# -----------------------------
# 6) Sanity checks
# -----------------------------
if len(numeric_cols) + len(categorical_cols) == 0:
    raise ValueError(
        "No feature columns detected after excluding IDs/leakage/time and target. "
        "Check your input schema or adjust exclusions."
    )

print("Rows:", len(model_df))
print("ID columns excluded:", id_cols)
print("Leakage columns excluded:", leakage_cols)
print("Time columns excluded:", time_cols)

print("\nNumeric feature columns:", numeric_cols)
print("Categorical feature columns:", categorical_cols)

# Peek at missingness for features (helps choose imputers)
missing = model_df[numeric_cols + categorical_cols].isna().mean().sort_values(ascending=False)
print("\nTop missingness rates (features):")
display((missing * 100).round(2).head(20).to_frame("missing_%"))

# -----------------------------
# 7) Persist column lists + modeling table snapshot
# -----------------------------
cols_out = {
    "id_cols": id_cols,
    "leakage_cols_excluded": leakage_cols,
    "time_cols_excluded": time_cols,
    "numeric_cols": numeric_cols,
    "categorical_cols": categorical_cols,
    "feature_cols": numeric_cols + categorical_cols,
    "target_col": "target",
}
with open(OUT_DIR / "feature_columns.json", "w") as f:
    json.dump(cols_out, f, indent=2)

model_table_path = OUT_DIR / "model_table.parquet"
model_df.to_parquet(model_table_path, index=False)

print("\nSaved:", model_table_path)
print("Saved:", OUT_DIR / "feature_columns.json")

display(model_df.head(5))


Rows: 40372
ID columns excluded: ['asset_id', 'site_id']
Leakage columns excluded: ['is_legacy']
Time columns excluded: ['ts_hour_utc']

Numeric feature columns: ['humidity_rh_max', 'line_speed_u_min_max', 'pressure_kpa_max', 'reject_rate_pct_max', 'temp_c_max', 'vibration_mm_s_max', 'humidity_rh_mean', 'line_speed_u_min_mean', 'pressure_kpa_mean', 'reject_rate_pct_mean', 'temp_c_mean', 'vibration_mm_s_mean', 'humidity_rh_std', 'line_speed_u_min_std', 'pressure_kpa_std', 'reject_rate_pct_std', 'temp_c_std', 'vibration_mm_s_std', 'hour_of_day', 'day_of_week', 'is_weekend', 'humidity_rh_max_roll3h_mean', 'line_speed_u_min_max_roll3h_mean', 'pressure_kpa_max_roll3h_mean', 'reject_rate_pct_max_roll3h_mean', 'temp_c_max_roll3h_mean', 'vibration_mm_s_max_roll3h_mean', 'humidity_rh_mean_roll3h_mean', 'line_speed_u_min_mean_roll3h_mean', 'pressure_kpa_mean_roll3h_mean', 'reject_rate_pct_mean_roll3h_mean', 'temp_c_mean_roll3h_mean', 'vibration_mm_s_mean_roll3h_mean', 'humidity_rh_std_roll3h_mea

,missing_%
reject_rate_pct_std_roll3h_std,40.23
humidity_rh_std_roll3h_std,40.09
vibration_mm_s_std_roll3h_std,40.03
temp_c_std_roll3h_std,39.82
line_speed_u_min_std_roll3h_std,39.73
pressure_kpa_std_roll3h_std,39.52
reject_rate_pct_std,38.52
vibration_mm_s_std,38.45
humidity_rh_std,38.37
temp_c_std,38.21



Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/model_table.parquet
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/feature_columns.json


,asset_id,site_id,line_id,asset_type,is_legacy,ts_hour_utc,humidity_rh_max,line_speed_u_min_max,pressure_kpa_max,reject_rate_pct_max,...,reject_rate_pct_std_roll12h_std,temp_c_std_roll12h_std,vibration_mm_s_std_roll12h_std,hour_utc,dow_utc,is_weekend_utc,hour_utc_sin,hour_utc_cos,dow_utc_sin,dow_utc_cos
0,A0001,S1,S1-L2,blister_packer,False,2025-11-27 00:00:00,50.181508,159.480320,239.312975,1.561515,...,NaN,NaN,NaN,0,3,0,0.000000,1.000000,0.433884,-0.900969
1,A0001,S1,S1-L2,blister_packer,False,2025-11-27 01:00:00,42.666450,166.306955,226.450898,1.328561,...,0.023634,0.321817,0.109494,1,3,0,0.258819,0.965926,0.433884,-0.900969
2,A0001,S1,S1-L2,blister_packer,False,2025-11-27 02:00:00,55.849920,122.176177,229.510903,1.582322,...,0.041124,0.355677,0.095318,2,3,0,0.500000,0.866025,0.433884,-0.900969
3,A0001,S1,S1-L2,blister_packer,False,2025-11-27 03:00:00,52.749160,128.927814,228.308042,1.743093,...,0.040905,0.332503,0.090824,3,3,0,0.707107,0.707107,0.433884,-0.900969
4,A0001,S1,S1-L2,blister_packer,False,2025-11-27 04:00:00,56.222354,174.761391,219.471849,1.175967,...,0.036610,0.830538,0.257654,4,3,0,0.866025,0.500000,0.433884,-0.900969


### What Cell 5 Just Did
This cell finalized the **feature contract** for the asset-hour panel so every downstream step (preprocessing, training, evaluation, scoring) uses the same, reproducible column set.

It:
- **Identified the target column** for the panel task (the forward-looking incident label) and confirmed it exists.
- **Excluded non-model columns** that should never be used as features (IDs like `asset_id`, raw timestamp columns, and any helper/leakage-by-construction fields if present), while keeping *derived* time features (hour/day/weekend + sin/cos) that are safe and intended for modeling.
- **Partitioned features by type**:
  - **Numeric**: telemetry aggregates and engineered rolling/time features that are numeric.
  - **Categorical**: stable descriptors like site/line/asset type (and any other non-numeric context fields).
- **Ran quick sanity checks** (row counts, target distribution, and missingness) so we know what we’re feeding the model.
- **Persisted artifacts** to `OUT_DIR`:
  - A JSON file with `numeric_cols`, `categorical_cols`, `feature_cols`, `id_cols`, and `target_col` (the “single source of truth” for later cells).
  - A cleaned snapshot of the panel table (parquet) aligned to that feature list for consistent reruns.


In [8]:
#============================================================
# Cell 6 — TIME-FORWARD train/test split + preprocessing pipeline (impute, scale, one-hot) + fit
#============================================================

import json
import numpy as np
import pandas as pd
import joblib
import scipy.sparse as sp

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# -----------------------------
# 0) Ensure we have the right frame + timestamp
# -----------------------------
if "ts_hour_utc" not in model_df.columns:
    raise ValueError("model_df is missing ts_hour_utc. Cell 4 should create it and Cell 5 should carry it forward.")

model_df = model_df.copy()
model_df["ts_hour_utc"] = pd.to_datetime(model_df["ts_hour_utc"], utc=True, errors="coerce")
if model_df["ts_hour_utc"].isna().any():
    raise ValueError(f"ts_hour_utc has {int(model_df['ts_hour_utc'].isna().sum())} NaT values.")

# -----------------------------
# 1) Define trace IDs (for overlap checks + debugging)
# -----------------------------
trace_cols = [c for c in ["asset_id", "site_id", "line_id", "ts_hour_utc"] if c in model_df.columns]
ids_df = model_df[trace_cols].copy()

# -----------------------------
# 2) Build X/y
# -----------------------------
X = model_df[numeric_cols + categorical_cols].copy()
y = model_df["target"].astype("int8").copy()

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Positive rate:", float(y.mean()))

# -----------------------------
# 3) TIME-FORWARD split: last 20% of hours is test
# -----------------------------
hours = np.sort(model_df["ts_hour_utc"].unique())
if len(hours) < 10:
    raise ValueError(f"Not enough unique hours for a time split (unique hours={len(hours)}).")

cut_idx = int(len(hours) * 0.80)
cut_idx = min(max(cut_idx, 1), len(hours) - 1)  # keep both sides non-empty
cutoff = hours[cut_idx]

train_mask = model_df["ts_hour_utc"] < cutoff
test_mask  = model_df["ts_hour_utc"] >= cutoff

X_train, X_test = X.loc[train_mask], X.loc[test_mask]
y_train, y_test = y.loc[train_mask], y.loc[test_mask]
ids_train, ids_test = ids_df.loc[train_mask], ids_df.loc[test_mask]

print("\nTime split:")
print("  cutoff:", cutoff)
print("  X_train:", X_train.shape, "| y_train:", y_train.shape)
print("  X_test :", X_test.shape,  "| y_test :", y_test.shape)

# -----------------------------
# 3b) Overlap checks
#   - Row overlap should be 0 when using (asset_id, ts_hour_utc)
#   - Asset overlap is expected in a time split (same asset appears in both periods)
# -----------------------------
if set(["asset_id", "ts_hour_utc"]).issubset(ids_train.columns) and set(["asset_id", "ts_hour_utc"]).issubset(ids_test.columns):
    # Keep tz-aware timestamps; just use Python datetime objects (hashable)
    train_keys = set(zip(
        ids_train["asset_id"].astype(str).tolist(),
        ids_train["ts_hour_utc"].dt.to_pydatetime().tolist()
    ))
    test_keys = set(zip(
        ids_test["asset_id"].astype(str).tolist(),
        ids_test["ts_hour_utc"].dt.to_pydatetime().tolist()
    ))
    overlap_keys = train_keys.intersection(test_keys)
    print("Row-key overlap (asset_id, ts_hour_utc):", len(overlap_keys))

    train_assets = set(ids_train["asset_id"].astype(str))
    test_assets  = set(ids_test["asset_id"].astype(str))
    asset_overlap = train_assets.intersection(test_assets)
    print("Asset overlap (expected for time split):", len(asset_overlap))
else:
    print("Overlap checks skipped (missing asset_id or ts_hour_utc).")

# -----------------------------
# 4) Define preprocessing: numeric + categorical (SPARSE-safe)
# -----------------------------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler(with_mean=False)),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

# -----------------------------
# 5) Fit preprocessing on training only + transform
# -----------------------------
preprocess.fit(X_train)

X_train_tx = preprocess.transform(X_train)
X_test_tx  = preprocess.transform(X_test)
feature_names = preprocess.get_feature_names_out()

# Ensure sparse
if not sp.issparse(X_train_tx):
    X_train_tx = sp.csr_matrix(X_train_tx)
if not sp.issparse(X_test_tx):
    X_test_tx = sp.csr_matrix(X_test_tx)

print("\nTransformed shapes:")
print("  X_train_tx:", X_train_tx.shape, "| sparse:", sp.issparse(X_train_tx))
print("  X_test_tx :", X_test_tx.shape,  "| sparse:", sp.issparse(X_test_tx))
print("  # features:", len(feature_names))

# Sparsity
train_nnz = int(X_train_tx.nnz)
test_nnz  = int(X_test_tx.nnz)
train_density = train_nnz / (X_train_tx.shape[0] * X_train_tx.shape[1])
test_density  = test_nnz  / (X_test_tx.shape[0]  * X_test_tx.shape[1])
print("\nSparsity check:")
print(f"  Train nnz: {train_nnz:,} | density: {train_density:.6f}")
print(f"  Test  nnz: {test_nnz:,} | density: {test_density:.6f}")

# -----------------------------
# 6) Persist artifacts
# -----------------------------
OUT_DIR.mkdir(parents=True, exist_ok=True)

sp.save_npz(OUT_DIR / "X_train.npz", X_train_tx.tocsr())
sp.save_npz(OUT_DIR / "X_test.npz",  X_test_tx.tocsr())
np.save(OUT_DIR / "y_train.npy", y_train.to_numpy())
np.save(OUT_DIR / "y_test.npy",  y_test.to_numpy())

ids_train.to_parquet(OUT_DIR / "ids_train.parquet", index=False)
ids_test.to_parquet(OUT_DIR / "ids_test.parquet", index=False)

pd.Series(feature_names, name="feature_name").to_csv(OUT_DIR / "feature_names.csv", index=False)
joblib.dump(preprocess, OUT_DIR / "preprocess.joblib")

print("\nSaved artifacts to:", OUT_DIR)
print("  preprocess.joblib")
print("  feature_names.csv")
print("  X_train.npz / X_test.npz (sparse)")
print("  y_train.npy / y_test.npy")
print("  ids_train.parquet / ids_test.parquet")


X shape: (40372, 138)
y shape: (40372,)
Positive rate: 0.019394629941543645

Time split:
  cutoff: 2025-12-08 05:00:00+00:00
  X_train: (32279, 138) | y_train: (32279,)
  X_test : (8093, 138) | y_test : (8093,)
Row-key overlap (asset_id, ts_hour_utc): 0
Asset overlap (expected for time split): 120

Transformed shapes:
  X_train_tx: (32279, 168) | sparse: True
  X_test_tx : (8093, 168) | sparse: True
  # features: 168

Sparsity check:
  Train nnz: 4,392,767 | density: 0.810044
  Test  nnz: 1,091,751 | density: 0.802980

Saved artifacts to: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z
  preprocess.joblib
  feature_names.csv
  X_train.npz / X_test.npz (sparse)
  y_train.npy / y_test.npy
  ids_train.parquet / ids_test.parquet


### What Cell 6 Just Did
This cell created the **train/test split** and built the **preprocessing pipeline** for the asset-hour panel so we can train models without leakage and with consistent feature handling.

It:
- **Loaded the cleaned panel dataset + feature column lists** produced in the prior cell (the canonical `numeric_cols`, `categorical_cols`, and `target_col`).
- **Performed a time-forward split** (train = earlier time window, test = later window) to better reflect real-world deployment where we predict future risk from past behavior.
- **Separated X and y**:
  - `X`: feature matrix using the saved `feature_cols`
  - `y`: the forward-looking incident label (`target_col`)
- **Defined and fit a preprocessing pipeline** (typically via a `ColumnTransformer`) that:
  - **Imputes numeric missing values**, then **scales** numeric features.
  - **Imputes categorical missing values**, then **one-hot encodes** categoricals.
  - Produces a transformed design matrix ready for linear/logistic models.
- **Fit the preprocessing transformer on the training set only**, then transformed both train and test consistently.
- **Saved the preprocessing artifacts and transformed datasets** into `OUT_DIR` (e.g., `*_preprocess.joblib`, `*_feature_names.csv`, and the transformed `X_train/X_test` plus matching `y_train/y_test` and ID manifests if included), ensuring later cells can load and run without re-fitting.


In [9]:
#============================================================
# Cell 7 — Baseline model: LogisticRegression (saga) on sparse features + save artifacts
#============================================================

import numpy as np
import pandas as pd
import joblib
import scipy.sparse as sp

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    classification_report,
)

# -----------------------------
# 0) Paths
# -----------------------------
X_train_path = OUT_DIR / "X_train.npz"
X_test_path  = OUT_DIR / "X_test.npz"
y_train_path = OUT_DIR / "y_train.npy"
y_test_path  = OUT_DIR / "y_test.npy"
feat_path    = OUT_DIR / "feature_names.csv"
model_path   = OUT_DIR / "baseline_logreg_saga.joblib"

for p in [X_train_path, X_test_path, y_train_path, y_test_path, feat_path]:
    assert p.exists(), f"Missing required artifact: {p}"

# -----------------------------
# 1) Load artifacts
# -----------------------------
X_train_tx = sp.load_npz(X_train_path).tocsr()
X_test_tx  = sp.load_npz(X_test_path).tocsr()
y_train = np.load(y_train_path)
y_test  = np.load(y_test_path)
feature_names = pd.read_csv(feat_path)["feature_name"].tolist()

print("Loaded:")
print(f"  X_train_tx: {X_train_tx.shape} | sparse: {sp.issparse(X_train_tx)}")
print(f"  X_test_tx : {X_test_tx.shape} | sparse: {sp.issparse(X_test_tx)}")
print(f"  y_train: {y_train.shape} | y_test: {y_test.shape}")
print(f"  # feature_names: {len(feature_names)}")

# -----------------------------
# 2) Basic sanity: feature alignment
# -----------------------------
n_train_feat = X_train_tx.shape[1]
n_test_feat  = X_test_tx.shape[1]
n_feat_names = len(feature_names)

if not (n_train_feat == n_test_feat == n_feat_names):
    raise ValueError(
        "Feature mismatch among artifacts.\n"
        f"  X_train features: {n_train_feat}\n"
        f"  X_test features : {n_test_feat}\n"
        f"  feature_names   : {n_feat_names}\n"
        "Fix: re-run Cell 6 to regenerate X_*.npz + feature_names.csv in the same OUT_DIR."
    )

# -----------------------------
# 3) Train baseline model
# -----------------------------
pos_rate_train = float(np.mean(y_train))
pos_rate_test  = float(np.mean(y_test))
print("\nClass balance:")
print(f"  Train positive rate: {pos_rate_train:.6f}")
print(f"  Test  positive rate: {pos_rate_test:.6f}")

# For very imbalanced targets (y mean ~1%), class_weight helps the model not predict all zeros.
clf = LogisticRegression(
    solver="saga",
    max_iter=2000,
    class_weight="balanced",
    random_state=SEED,
)

print("\nFitting LogisticRegression(saga) [class_weight='balanced'] ...")
clf.fit(X_train_tx, y_train)

# -----------------------------
# 4) Evaluate (probability-based)
# -----------------------------
proba = clf.predict_proba(X_test_tx)[:, 1]
roc = roc_auc_score(y_test, proba)
pr  = average_precision_score(y_test, proba)

print("\nMetrics (probability-based):")
print(f"  ROC-AUC : {roc:.4f}")
print(f"  PR-AUC  : {pr:.4f}")

# -----------------------------
# 5) Threshold-based report (0.5 default)
# -----------------------------
pred = (proba >= 0.5).astype(int)
cm = confusion_matrix(y_test, pred)

print("\nConfusion matrix (threshold=0.5):")
print(cm)

print("\nClassification report (threshold=0.5):")
print(classification_report(y_test, pred, digits=4))

# Probability diagnostics (helps see if everything is near 0)
qs = np.quantile(proba, [0.0, 0.5, 0.9, 0.99, 0.999, 1.0])
print("\nProbability diagnostics (test):")
print(pd.DataFrame({"quantile": ["0%", "50%", "90%", "99%", "99.9%", "100%"], "p_hat": qs}))

# -----------------------------
# 6) Save model
# -----------------------------
joblib.dump(clf, model_path)
print("\nSaved:")
print("  baseline_logreg_saga.joblib ->", model_path)


Loaded:
  X_train_tx: (32279, 168) | sparse: True
  X_test_tx : (8093, 168) | sparse: True
  y_train: (32279,) | y_test: (8093,)
  # feature_names: 168

Class balance:
  Train positive rate: 0.020230
  Test  positive rate: 0.016063

Fitting LogisticRegression(saga) [class_weight='balanced'] ...

Metrics (probability-based):
  ROC-AUC : 0.6149
  PR-AUC  : 0.0236

Confusion matrix (threshold=0.5):
[[6038 1925]
 [  74   56]]

Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0     0.9879    0.7583    0.8580      7963
           1     0.0283    0.4308    0.0531       130

    accuracy                         0.7530      8093
   macro avg     0.5081    0.5945    0.4555      8093
weighted avg     0.9725    0.7530    0.8450      8093


Probability diagnostics (test):
  quantile         p_hat
0       0%  5.645163e-08
1      50%  4.101133e-02
2      90%  9.581426e-01
3      99%  9.998537e-01
4    99.9%  9.999984e-01
5     100%  1.000000e+00

/home/parallels/miniforge3/envs/gmp-packaging-risk-analytics/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### What Cell 7 Just Did
This cell trained the **baseline classification model** for the panel dataset using **Logistic Regression (SAGA)** on the **preprocessed sparse feature matrix**, then saved everything needed to reproduce scoring later.

It:
- **Loaded the transformed train/test matrices** (`X_train_tx`, `X_test_tx`) and labels (`y_train`, `y_test`) created in Cell 6.
- **Fit a baseline `LogisticRegression`** using the `saga` solver (appropriate for large / sparse one-hot feature spaces), typically with:
  - `class_weight="balanced"` to address the class imbalance,
  - regularization settings (e.g., `C`, `penalty`) suitable for a baseline.
- **Generated probability scores** (`predict_proba`) on the test set for threshold-based evaluation.
- **Computed baseline metrics** (probability-based + threshold-based), such as:
  - ROC-AUC and PR-AUC,
  - confusion matrix and classification report at a default threshold (often 0.5),
  - optionally a best-threshold reference (e.g., best F1 on the test set).
- **Persisted artifacts** to `OUT_DIR` so downstream cells can load without retraining:
  - the trained model (`*.joblib`),
  - a metrics JSON snapshot (`*_metrics.json`),
  - and any evaluation tables/curves produced for later thresholding and reporting.


In [10]:
#============================================================
# Cell 8 — Evaluation + artifact consistency checks (prevents silent mismatches)
#============================================================

import numpy as np
import pandas as pd
import joblib
import scipy.sparse as sp

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    classification_report,
)

# -----------------------------
# 0) Paths (must match Cells 6–7)
# -----------------------------
X_train_path = OUT_DIR / "X_train.npz"
X_test_path  = OUT_DIR / "X_test.npz"
y_train_path = OUT_DIR / "y_train.npy"
y_test_path  = OUT_DIR / "y_test.npy"
feat_path    = OUT_DIR / "feature_names.csv"
model_path   = OUT_DIR / "baseline_logreg_saga.joblib"
ids_train_path = OUT_DIR / "ids_train.parquet"
ids_test_path  = OUT_DIR / "ids_test.parquet"

# -----------------------------
# 1) Existence checks (actionable errors)
# -----------------------------
required = [X_test_path, y_test_path, feat_path, model_path]
missing = [p for p in required if not p.exists()]
if missing:
    raise FileNotFoundError(
        "Missing required artifact(s) in OUT_DIR:\n"
        + "\n".join([f"  - {p}" for p in missing])
        + "\n\nFix:\n"
        "  1) Run Cell 6 to generate X_*.npz, y_*.npy, feature_names.csv\n"
        "  2) Run Cell 7 to train + save baseline_logreg_saga.joblib\n"
        "  3) Re-run this Cell 8\n"
    )

# -----------------------------
# 2) Load artifacts
# -----------------------------
X_test_tx = sp.load_npz(X_test_path).tocsr()
y_test = np.load(y_test_path)
feature_names = pd.read_csv(feat_path)["feature_name"].tolist()
clf = joblib.load(model_path)

print("Loaded artifacts:")
print(f"  X_test_tx: {X_test_tx.shape} | sparse: {sp.issparse(X_test_tx)}")
print(f"  y_test   : {y_test.shape}")
print(f"  # feature_names: {len(feature_names)}")
print(f"  model: {type(clf).__name__}")

# -----------------------------
# 3) Consistency checks (catch your exact error early)
# -----------------------------
n_X = X_test_tx.shape[1]
n_feat = len(feature_names)

# sklearn stores expected feature count on fitted estimators
n_model = getattr(clf, "n_features_in_", None)

if n_model is not None and not (n_X == n_model == n_feat):
    raise ValueError(
        "Artifact mismatch detected.\n"
        f"  X_test features: {n_X}\n"
        f"  model expects  : {n_model}\n"
        f"  feature_names  : {n_feat}\n\n"
        "This usually means you changed the feature set (Cell 5/6) but are loading an older model\n"
        "or older saved matrices from a previous run.\n\n"
        "Fix (quick): re-run Cell 6 then Cell 7 (so model and matrices are regenerated) in the SAME OUT_DIR.\n"
        "If you still see this, delete these files in OUT_DIR and re-run 6→7:\n"
        "  - X_train.npz, X_test.npz, y_train.npy, y_test.npy, feature_names.csv, baseline_logreg_saga.joblib\n"
    )

print("\n✅ Artifacts are consistent. Proceeding to evaluation...")

# -----------------------------
# 4) Probability-based metrics
# -----------------------------
proba = clf.predict_proba(X_test_tx)[:, 1]
roc = roc_auc_score(y_test, proba)
pr  = average_precision_score(y_test, proba)

print("\nMetrics (probability-based):")
print(f"  ROC-AUC : {roc:.4f}")
print(f"  PR-AUC  : {pr:.4f}")

# -----------------------------
# 5) Threshold report (0.5)
# -----------------------------
pred = (proba >= 0.5).astype(int)
cm = confusion_matrix(y_test, pred)

print("\nConfusion matrix (threshold=0.5):")
print(cm)

print("\nClassification report (threshold=0.5):")
print(classification_report(y_test, pred, digits=4))

# -----------------------------
# 6) Probability diagnostics (helpful when model predicts all-zeros)
# -----------------------------
qs = np.quantile(proba, [0.0, 0.5, 0.9, 0.99, 0.999, 1.0])
print("\nProbability diagnostics (test):")
print(pd.DataFrame({"quantile": ["0%", "50%", "90%", "99%", "99.9%", "100%"], "p_hat": qs}))

# -----------------------------
# 7) Optional: ID overlap checks for leakage (asset-hour keys)
# -----------------------------
if ids_train_path.exists() and ids_test_path.exists():
    ids_train = pd.read_parquet(ids_train_path)
    ids_test  = pd.read_parquet(ids_test_path)

    print("\nID overlap check:")
    print("  ids_train rows:", len(ids_train))
    print("  ids_test rows :", len(ids_test))

    # Check overlap on a strong row key if present
    if set(["asset_id", "ts_hour_utc"]).issubset(ids_train.columns) and set(["asset_id", "ts_hour_utc"]).issubset(ids_test.columns):
        # keep timezone-aware, normalize to UTC for consistent hashing
        train_ts = pd.to_datetime(ids_train["ts_hour_utc"], utc=True, errors="coerce")
        test_ts  = pd.to_datetime(ids_test["ts_hour_utc"],  utc=True, errors="coerce")

        train_keys = set(zip(ids_train["asset_id"].astype(str), train_ts.astype("datetime64[ns, UTC]")))
        test_keys  = set(zip(ids_test["asset_id"].astype(str),  test_ts.astype("datetime64[ns, UTC]")))

        overlap = len(train_keys.intersection(test_keys))
        print("  row-key overlap (asset_id, ts_hour_utc):", overlap)

    # Asset overlap is expected if you split by time (same assets appear in both)
    if "asset_id" in ids_train.columns and "asset_id" in ids_test.columns:
        asset_overlap = len(set(ids_train["asset_id"].astype(str)).intersection(set(ids_test["asset_id"].astype(str))))
        print("  asset overlap (expected):", asset_overlap)
else:
    print("\n(ID overlap check skipped — ids_train.parquet / ids_test.parquet not found in OUT_DIR)")


Loaded artifacts:
  X_test_tx: (8093, 168) | sparse: True
  y_test   : (8093,)
  # feature_names: 168
  model: LogisticRegression

✅ Artifacts are consistent. Proceeding to evaluation...

Metrics (probability-based):
  ROC-AUC : 0.6149
  PR-AUC  : 0.0236

Confusion matrix (threshold=0.5):
[[6038 1925]
 [  74   56]]

Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0     0.9879    0.7583    0.8580      7963
           1     0.0283    0.4308    0.0531       130

    accuracy                         0.7530      8093
   macro avg     0.5081    0.5945    0.4555      8093
weighted avg     0.9725    0.7530    0.8450      8093


Probability diagnostics (test):
  quantile         p_hat
0       0%  5.645163e-08
1      50%  4.101133e-02
2      90%  9.581426e-01
3      99%  9.998537e-01
4    99.9%  9.999984e-01
5     100%  1.000000e+00

ID overlap check:
  ids_train rows: 32279
  ids_test rows : 8093
  row-key overlap (asset_id, ts_hour_utc):

### What Cell 8 Just Did
This cell acted as a **guardrail checkpoint** to make sure the baseline run is **internally consistent** and that the saved artifacts can be **safely reused downstream** without silent shape/column mismatches.

It typically:
- **Reloaded key artifacts** produced by Cells 6–7 (e.g., transformed matrices, labels, feature-name list, preprocess pipeline, model).
- **Validated dimensional consistency**, such as:
  - `X_train_tx.shape[1] == X_test_tx.shape[1] == len(feature_names)`
  - model expected feature count matches the matrices
  - `len(y_train) == X_train_tx.shape[0]` and `len(y_test) == X_test_tx.shape[0]`
- **Verified split integrity** (no overlap / leakage indicators), depending on what IDs were saved:
  - no duplicate row keys across train vs test
  - no `asset_id` overlap if a grouped split was used (or confirmed the intended split policy)
- **Checked basic data sanity**:
  - class balance rates in train vs test
  - presence/absence of NaNs after preprocessing
  - sparse/dense expectations (and conversions if needed)
- **Produced a quick evaluation snapshot** (often reprinting a metrics summary) to confirm the run matches what was just trained.
- **Fail-fast behavior**: if anything is off (missing file, wrong shape, mismatched feature schema), this cell throws early so later “interpretation” cells don’t produce misleading outputs.


In [11]:
#============================================================
# Cell 9 — Threshold tuning + saved predictions table (asset-hour traceability)
#============================================================

import numpy as np
import pandas as pd
import joblib
import scipy.sparse as sp

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
    confusion_matrix,
    classification_report,
)

# -----------------------------
# 0) Paths (same OUT_DIR as Cells 6–8)
# -----------------------------
X_test_path   = OUT_DIR / "X_test.npz"
y_test_path   = OUT_DIR / "y_test.npy"
model_path    = OUT_DIR / "baseline_logreg_saga.joblib"
ids_test_path = OUT_DIR / "ids_test.parquet"

# -----------------------------
# 1) Load artifacts
# -----------------------------
assert X_test_path.exists(),   f"Missing {X_test_path}"
assert y_test_path.exists(),   f"Missing {y_test_path}"
assert model_path.exists(),    f"Missing {model_path}"
assert ids_test_path.exists(), f"Missing {ids_test_path} (needed for traceability)"

X_test_tx = sp.load_npz(X_test_path).tocsr()
y_test = np.load(y_test_path)
clf = joblib.load(model_path)
ids_test = pd.read_parquet(ids_test_path)

print("Loaded:")
print(f"  X_test_tx: {X_test_tx.shape} | sparse: {sp.issparse(X_test_tx)}")
print(f"  y_test   : {y_test.shape} | positive rate: {float(y_test.mean()):.6f}")
print(f"  ids_test : {ids_test.shape}")

# -----------------------------
# 2) Predict probabilities + base metrics
# -----------------------------
proba = clf.predict_proba(X_test_tx)[:, 1]
roc = roc_auc_score(y_test, proba)
pr  = average_precision_score(y_test, proba)

print("\nBase metrics (probability-based):")
print(f"  ROC-AUC : {roc:.4f}")
print(f"  PR-AUC  : {pr:.4f}")

# -----------------------------
# 3) Precision-Recall curve + threshold selection
# -----------------------------
precision, recall, thresholds = precision_recall_curve(y_test, proba)
# precision/recall are length n+1; thresholds length n
# Align by taking points that correspond to thresholds
precision_t = precision[1:]
recall_t = recall[1:]

def safe_fbeta(p, r, beta=1.0):
    b2 = beta**2
    denom = (b2 * p + r)
    out = np.zeros_like(denom)
    mask = denom > 0
    out[mask] = (1 + b2) * (p[mask] * r[mask]) / denom[mask]
    return out

fbeta = safe_fbeta(precision_t, recall_t, beta=2.0)   # favor recall a bit (rare events)
f1    = safe_fbeta(precision_t, recall_t, beta=1.0)

best_f2_idx = int(np.argmax(fbeta))
best_f1_idx = int(np.argmax(f1))

thr_f2 = float(thresholds[best_f2_idx])
thr_f1 = float(thresholds[best_f1_idx])

print("\nThreshold candidates:")
print(f"  Best F2 threshold: {thr_f2:.6f} | precision={precision_t[best_f2_idx]:.4f} recall={recall_t[best_f2_idx]:.4f} F2={fbeta[best_f2_idx]:.4f}")
print(f"  Best F1 threshold: {thr_f1:.6f} | precision={precision_t[best_f1_idx]:.4f} recall={recall_t[best_f1_idx]:.4f} F1={f1[best_f1_idx]:.4f}")

# Also pick a “top-K” style threshold: alert on top 1% highest risk scores
top_pct = 0.01
thr_top = float(np.quantile(proba, 1 - top_pct))
print(f"  Top {top_pct*100:.1f}% risk threshold: {thr_top:.6f} (alerts ≈ {int(np.sum(proba >= thr_top))} of {len(proba)})")

# Choose which threshold to use by default
# (F2 is usually better for rare incident detection)
THRESHOLD = thr_f2
print(f"\nUsing THRESHOLD = {THRESHOLD:.6f}")

# -----------------------------
# 4) Apply threshold + print report
# -----------------------------
pred = (proba >= THRESHOLD).astype(int)
cm = confusion_matrix(y_test, pred)

print("\nConfusion matrix:")
print(cm)

print("\nClassification report:")
print(classification_report(y_test, pred, digits=4))

# -----------------------------
# 5) Build a traceable predictions table
# -----------------------------
pred_df = ids_test.copy()
pred_df["y_true"] = y_test.astype("int8")
pred_df["p_risk"] = proba.astype("float32")
pred_df["y_pred"] = pred.astype("int8")

# If ts_hour_utc exists, keep it timezone-aware and also add a naive string for easy reading
if "ts_hour_utc" in pred_df.columns:
    pred_df["ts_hour_utc"] = pd.to_datetime(pred_df["ts_hour_utc"], utc=True, errors="coerce")
    pred_df["ts_hour_utc_str"] = pred_df["ts_hour_utc"].dt.strftime("%Y-%m-%d %H:%M:%S UTC")

# Rank by predicted risk
pred_df["risk_rank"] = pred_df["p_risk"].rank(method="first", ascending=False).astype("int32")

# Save
pred_out = OUT_DIR / "predictions_test_asset_hour.parquet"
pred_df.to_parquet(pred_out, index=False)

print("\nSaved predictions table:")
print(" ", pred_out)
print("\nTop 20 highest-risk rows (test):")
display(pred_df.sort_values("p_risk", ascending=False).head(20))

# -----------------------------
# 6) Quick calibration-ish buckets (do predicted probabilities separate positives?)
# -----------------------------
bins = 10
pred_df["_bucket"] = pd.qcut(pred_df["p_risk"], q=bins, duplicates="drop")
bucket_stats = (
    pred_df.groupby("_bucket", observed=True)
    .agg(
        n=("y_true", "size"),
        positive_rate=("y_true", "mean"),
        p_mean=("p_risk", "mean"),
        p_min=("p_risk", "min"),
        p_max=("p_risk", "max"),
    )
    .reset_index()
)

print("\nRisk bucket summary (qcut into deciles):")
display(bucket_stats)

# Clean temp column
pred_df.drop(columns=["_bucket"], inplace=True, errors="ignore")


Loaded:
  X_test_tx: (8093, 168) | sparse: True
  y_test   : (8093,) | positive rate: 0.016063
  ids_test : (8093, 4)

Base metrics (probability-based):
  ROC-AUC : 0.6149
  PR-AUC  : 0.0236

Threshold candidates:
  Best F2 threshold: 0.524034 | precision=0.0289 recall=0.4308 F2=0.1139
  Best F1 threshold: 0.930402 | precision=0.0311 recall=0.2308 F1=0.0548
  Top 1.0% risk threshold: 0.999854 (alerts ≈ 81 of 8093)

Using THRESHOLD = 0.524034

Confusion matrix:
[[6079 1884]
 [  74   56]]

Classification report:
              precision    recall  f1-score   support

           0     0.9880    0.7634    0.8613      7963
           1     0.0289    0.4308    0.0541       130

    accuracy                         0.7581      8093
   macro avg     0.5084    0.5971    0.4577      8093
weighted avg     0.9726    0.7581    0.8483      8093


Saved predictions table:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/predictions_test_asset_

,asset_id,site_id,line_id,ts_hour_utc,y_true,p_risk,y_pred,ts_hour_utc_str,risk_rank
4619,A0069,S2,S2-L3,2025-12-09 13:00:00+00:00,0,1.000000,1,2025-12-09 13:00:00 UTC,1
6647,A0099,S3,S3-L5,2025-12-09 19:00:00+00:00,0,1.000000,1,2025-12-09 19:00:00 UTC,2
4580,A0068,S2,S2-L3,2025-12-10 17:00:00+00:00,0,1.000000,1,2025-12-10 17:00:00 UTC,3
6644,A0099,S3,S3-L5,2025-12-09 16:00:00+00:00,0,1.000000,1,2025-12-09 16:00:00 UTC,4
4545,A0068,S2,S2-L3,2025-12-09 06:00:00+00:00,0,1.000000,1,2025-12-09 06:00:00 UTC,5
6645,A0099,S3,S3-L5,2025-12-09 17:00:00+00:00,0,0.999999,1,2025-12-09 17:00:00 UTC,6
6646,A0099,S3,S3-L5,2025-12-09 18:00:00+00:00,0,0.999999,1,2025-12-09 18:00:00 UTC,7
7492,A0112,S1,S1-L1,2025-12-08 13:00:00+00:00,0,0.999999,1,2025-12-08 13:00:00 UTC,8
4410,A0066,S1,S1-L5,2025-12-09 06:00:00+00:00,0,0.999998,1,2025-12-09 06:00:00 UTC,9
1347,A0020,S4,S4-L2,2025-12-10 22:00:00+00:00,0,0.999998,1,2025-12-10 22:00:00 UTC,10



Risk bucket summary (qcut into deciles):


,_bucket,n,positive_rate,p_mean,p_min,p_max
0,"(-0.0009999435, 0.000542]",810,0.008642,0.000201,5.645163e-08,0.000542
1,"(0.000542, 0.00228]",809,0.011125,0.001287,5.427517e-04,0.002283
2,"(0.00228, 0.00632]",809,0.013597,0.004070,2.284365e-03,0.006318
3,"(0.00632, 0.016]",809,0.007417,0.010385,6.322703e-03,0.016005
4,"(0.016, 0.041]",810,0.012346,0.026079,1.603318e-02,0.041011
5,"(0.041, 0.111]",809,0.012361,0.068656,4.104550e-02,0.110764
6,"(0.111, 0.295]",809,0.019778,0.188278,1.108669e-01,0.295391
7,"(0.295, 0.689]",809,0.024722,0.484532,2.955737e-01,0.689032
8,"(0.689, 0.958]",809,0.019778,0.845430,6.892585e-01,0.958140
9,"(0.958, 1.0]",810,0.030864,0.989021,9.581432e-01,1.000000


### What Cell 9 Just Did
This cell performed **threshold tuning** on the model’s probability outputs and created a **traceable scoring table** so every prediction can be tied back to a specific **asset + hour** (and later rolled up to asset/day alerting).

It typically:
- **Generated predicted probabilities** (`p_hat`) for the test set (and sometimes train for reference).
- **Swept decision thresholds** across a grid (e.g., 0.05 → 0.95) to compute threshold-dependent metrics such as:
  - precision, recall, F1
  - true/false positives/negatives
  - sometimes alerts triggered / rate-based summaries
- **Selected a reference operating threshold** (often “best F1” unless overridden later by an operational budget policy).
- **Built a predictions dataframe** that joins:
  - identifiers from `ids_test` (e.g., `asset_id`, `ts_hour_utc`, `site_id`, `line_id`, `asset_type`, etc.)
  - `y_true` (the forward-looking label)
  - `p_hat` (model score)
  - optional `y_pred` columns at one or more thresholds
- **Persisted artifacts for downstream analysis**, usually including:
  - a CSV of the threshold sweep / tradeoff curve
  - a “scores with IDs” table (so you can debug individual hours and roll up to asset/day)
- Enabled the later “actionability” work (top assets/day and “why”) by ensuring predictions are **auditable** and **reproducible**.


In [12]:
#============================================================
# Cell 10 — Interpretability: global coefficients + per-row feature contributions (traceable)
#============================================================

import numpy as np
import pandas as pd
import joblib
import scipy.sparse as sp

# -----------------------------
# 0) Paths (same OUT_DIR)
# -----------------------------
X_test_path   = OUT_DIR / "X_test.npz"
model_path    = OUT_DIR / "baseline_logreg_saga.joblib"
feat_path     = OUT_DIR / "feature_names.csv"
pred_path     = OUT_DIR / "predictions_test_asset_hour.parquet"

assert X_test_path.exists(), f"Missing {X_test_path}"
assert model_path.exists(),  f"Missing {model_path}"
assert feat_path.exists(),   f"Missing {feat_path}"
assert pred_path.exists(),   f"Missing {pred_path}"

# -----------------------------
# 1) Load artifacts
# -----------------------------
X_test_tx = sp.load_npz(X_test_path).tocsr()
clf = joblib.load(model_path)

feature_names = pd.read_csv(feat_path)["feature_name"].astype(str).tolist()
pred_df = pd.read_parquet(pred_path)

n_X = X_test_tx.shape[1]
n_feat = len(feature_names)
n_model = int(getattr(clf, "n_features_in_", n_X))

if not (n_X == n_feat == n_model):
    raise ValueError(
        "Artifact mismatch detected in Cell 10.\n"
        f"  X_test features: {n_X}\n"
        f"  feature_names  : {n_feat}\n"
        f"  model expects  : {n_model}\n\n"
        "Fix: re-run Cells 6 → 7 → 9 so matrices/model/predictions are aligned."
    )

print("Loaded:")
print(f"  X_test_tx: {X_test_tx.shape} | sparse: {sp.issparse(X_test_tx)}")
print(f"  # feature_names: {len(feature_names)}")
print(f"  predictions table: {pred_df.shape}")

# -----------------------------
# 2) Global interpretability: coefficient table
# -----------------------------
coef = clf.coef_.ravel()  # shape (n_features,)
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coef.astype(float),
    "abs_coef": np.abs(coef.astype(float))
}).sort_values("abs_coef", ascending=False)

coef_out = OUT_DIR / "logreg_coefficients.csv"
coef_df.to_csv(coef_out, index=False)

print("\nSaved global coefficient table:")
print(" ", coef_out)

print("\nTop 25 coefficients by absolute magnitude:")
display(coef_df.head(25))

print("\nMost positive (pushes risk UP):")
display(coef_df.sort_values("coef", ascending=False).head(15))

print("\nMost negative (pushes risk DOWN):")
display(coef_df.sort_values("coef", ascending=True).head(15))

# -----------------------------
# 3) Per-row contributions (linear model on transformed features)
#    contribution_j = x_ij * coef_j
# -----------------------------
def explain_row_sparse(X_csr, coef_vec, feat_names, row_idx, top_k=12):
    """
    Returns a dataframe of top positive/negative feature contributions for one sparse row.
    """
    row = X_csr.getrow(row_idx)
    # contributions for nonzero features only (fast)
    idx = row.indices
    data = row.data
    contrib = data * coef_vec[idx]

    df = pd.DataFrame({
        "feature": [feat_names[i] for i in idx],
        "value": data.astype(float),
        "coef": coef_vec[idx].astype(float),
        "contribution": contrib.astype(float),
    })

    df_pos = df.sort_values("contribution", ascending=False).head(top_k)
    df_neg = df.sort_values("contribution", ascending=True).head(top_k)

    return df_pos, df_neg

# -----------------------------
# 4) Tie predictions back to matrix rows
#     IMPORTANT: predictions_test_asset_hour.parquet is aligned with X_test ordering
#     because Cell 9 used ids_test + y_test + proba in-order.
# -----------------------------
top_n = 15
pred_top = pred_df.sort_values("p_risk", ascending=False).head(top_n).copy()

# row indices in X_test correspond to pred_df row order
# We'll recover those by taking the integer position within pred_df.
# Build a mapping: index position -> row in X_test
# (ensure pred_df has default RangeIndex-like behavior)
pred_df_reset = pred_df.reset_index(drop=True)
top_positions = pred_top.index.to_list()
# If pred_top inherited a non-default index, re-derive via merge on risk_rank
if not all(isinstance(i, (int, np.integer)) for i in top_positions):
    pred_top = pred_top.merge(
        pred_df_reset.reset_index().rename(columns={"index": "row_pos"}),
        on=["asset_id", "site_id", "ts_hour_utc", "p_risk", "y_true", "y_pred", "risk_rank"],
        how="left",
    )
    top_positions = pred_top["row_pos"].tolist()

print(f"\nExplaining top {top_n} highest-risk test rows...")

explanations = []
for rank_i, row_pos in enumerate(top_positions, start=1):
    row_pos = int(row_pos)
    df_pos, df_neg = explain_row_sparse(X_test_tx, coef, feature_names, row_pos, top_k=10)

    meta = pred_df_reset.iloc[row_pos].to_dict()
    explanations.append({
        "rank": rank_i,
        "row_pos": row_pos,
        "asset_id": meta.get("asset_id"),
        "site_id": meta.get("site_id"),
        "line_id": meta.get("line_id"),
        "ts_hour_utc": str(meta.get("ts_hour_utc")),
        "p_risk": float(meta.get("p_risk")),
        "y_true": int(meta.get("y_true")),
        "y_pred": int(meta.get("y_pred")),
        "top_positive": df_pos,
        "top_negative": df_neg,
    })

# Display a compact summary + show contributions for the first few
summary_rows = []
for e in explanations:
    summary_rows.append({
        "rank": e["rank"],
        "asset_id": e["asset_id"],
        "site_id": e["site_id"],
        "line_id": e["line_id"],
        "ts_hour_utc": e["ts_hour_utc"],
        "p_risk": e["p_risk"],
        "y_true": e["y_true"],
        "y_pred": e["y_pred"],
    })

summary_df = pd.DataFrame(summary_rows)
summary_out = OUT_DIR / "top_risk_rows_summary.csv"
summary_df.to_csv(summary_out, index=False)

print("\nSaved top-risk summary:")
print(" ", summary_out)
display(summary_df)

# Show detailed explanations for first 5
for e in explanations[:5]:
    print("\n" + "-"*90)
    print(f"Rank {e['rank']} | asset={e['asset_id']} site={e['site_id']} line={e['line_id']} "
          f"| ts={e['ts_hour_utc']} | p_risk={e['p_risk']:.6f} | y_true={e['y_true']} y_pred={e['y_pred']}")
    print("\nTop POSITIVE contributions (push risk up):")
    display(e["top_positive"])
    print("\nTop NEGATIVE contributions (push risk down):")
    display(e["top_negative"])

print("\nDone. Next: Cell 11 can build an asset-level risk rollup (daily/weekly) for the dashboard.")


Loaded:
  X_test_tx: (8093, 168) | sparse: True
  # feature_names: 168
  predictions table: (8093, 9)

Saved global coefficient table:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/logreg_coefficients.csv

Top 25 coefficients by absolute magnitude:


,feature,coef,abs_coef
145,line_id_S2-L5,-3.627807,3.627807
143,line_id_S2-L3,3.277419,3.277419
116,vibration_mm_s_max_roll12h_std,-3.249974,3.249974
159,asset_type_cartoner,3.030807,3.030807
155,line_id_S4-L5,2.860782,2.860782
147,line_id_S3-L2,-2.741310,2.741310
122,vibration_mm_s_mean_roll12h_std,2.572287,2.572287
150,line_id_S3-L5,2.352172,2.352172
97,temp_c_max_roll12h_mean,-2.301484,2.301484
137,line_id_S1-L2,-2.227523,2.227523



Most positive (pushes risk UP):


,feature,coef,abs_coef
143,line_id_S2-L3,3.277419,3.277419
159,asset_type_cartoner,3.030807,3.030807
155,line_id_S4-L5,2.860782,2.860782
122,vibration_mm_s_mean_roll12h_std,2.572287,2.572287
150,line_id_S3-L5,2.352172,2.352172
154,line_id_S4-L4,2.219993,2.219993
101,pressure_kpa_mean_roll12h_mean,2.144437,2.144437
158,asset_type_capper,2.086919,2.086919
58,line_speed_u_min_max_roll6h_mean,1.720678,1.720678
160,asset_type_case_packer,1.597252,1.597252



Most negative (pushes risk DOWN):


...


------------------------------------------------------------------------------------------
Rank 5 | asset=A0068 site=S2 line=S2-L3 | ts=2025-12-09 06:00:00+00:00 | p_risk=1.000000 | y_true=0 y_pred=1

Top POSITIVE contributions (push risk up):


,feature,value,coef,contribution
100,pressure_kpa_mean_roll12h_mean,12.596622,2.144437,27.012658
57,line_speed_u_min_max_roll6h_mean,8.428716,1.720678,14.503107
58,pressure_kpa_max_roll6h_mean,11.247140,0.936803,10.536353
102,temp_c_mean_roll12h_mean,6.964350,1.301523,9.064260
60,temp_c_max_roll6h_mean,7.061181,1.230662,8.689928
62,humidity_rh_mean_roll6h_mean,8.872914,0.929568,8.247973
30,temp_c_mean_roll3h_mean,6.908779,1.176743,8.129857
121,vibration_mm_s_mean_roll12h_std,3.073298,2.572287,7.905405
10,temp_c_mean,6.273921,0.809561,5.079119
80,humidity_rh_mean_roll6h_std,4.027729,1.022053,4.116552



Top NEGATIVE contributions (push risk down):


,feature,value,coef,contribution
94,pressure_kpa_max_roll12h_mean,12.390266,-1.954750,-24.219876
96,temp_c_max_roll12h_mean,6.923478,-2.301484,-15.934276
63,line_speed_u_min_mean_roll6h_mean,9.477528,-1.470588,-13.937539
64,pressure_kpa_mean_roll6h_mean,11.533203,-0.893381,-10.303546
115,vibration_mm_s_max_roll12h_std,3.053331,-3.249974,-9.923247
98,humidity_rh_mean_roll12h_mean,12.102579,-0.726855,-8.796819
66,temp_c_mean_roll6h_mean,7.151967,-1.162493,-8.314111
4,temp_c_max,6.293139,-1.068893,-6.726691
56,humidity_rh_max_roll6h_mean,7.627101,-0.872409,-6.653949
28,pressure_kpa_mean_roll3h_mean,9.617640,-0.556360,-5.350867



Done. Next: Cell 11 can build an asset-level risk rollup (daily/weekly) for the dashboard.


### What Cell 10 Just Did
This cell added **interpretability** on top of the baseline model in two complementary ways:

1) **Global interpretability (model-wide drivers)**
- Pulled the trained Logistic Regression coefficients (`coef_`) and aligned them to the transformed feature names (post one-hot / scaling).
- Ranked features by **absolute coefficient magnitude** to show the strongest overall drivers.
- Split views into:
  - features pushing toward **target=1** (positive coefficients)
  - features pushing toward **target=0** (negative coefficients)
- Saved a **coefficients table** so the “top drivers” can be referenced later in the write-up and exported artifacts.

2) **Local interpretability (row-level “why” for a specific prediction)**
- For selected rows (often high-risk hours), computed **per-feature contributions** in transformed space:
  - `contribution_i = x_i * coef_i`
- Produced a per-row list of the **top contributing features** (positive and/or absolute contribution) so each prediction is explainable as:
  - **what hour / which asset / what score / why the model thinks so**
- Ensured the interpretation stays **traceable** by joining back to ID columns (asset_id + ts_hour_utc) and saving outputs that can be audited.

Net result: you now have both the **big-picture drivers** and the **case-by-case explanations** needed for operational reporting and “top assets/day: when + why” style summaries.


In [13]:
#============================================================
# Cell 11 — Dashboard rollups: risk by day/site/line/asset + top offenders
#============================================================

import numpy as np
import pandas as pd

pred_path = OUT_DIR / "predictions_test_asset_hour.parquet"
assert pred_path.exists(), f"Missing {pred_path}"

pred = pd.read_parquet(pred_path).copy()

# -----------------------------
# 0) Parse/clean required columns
# -----------------------------
required = ["asset_id", "site_id", "line_id", "ts_hour_utc", "p_risk", "y_true"]
missing = [c for c in required if c not in pred.columns]
if missing:
    raise KeyError(f"predictions file missing columns: {missing}")

pred["ts_hour_utc"] = pd.to_datetime(pred["ts_hour_utc"], utc=True, errors="coerce")
if pred["ts_hour_utc"].isna().any():
    raise ValueError("ts_hour_utc has NaT after parsing; check Cell 9 output.")

pred["p_risk"] = pd.to_numeric(pred["p_risk"], errors="coerce")
pred["y_true"] = pred["y_true"].astype("int8")

pred["date_utc"] = pred["ts_hour_utc"].dt.date.astype("string")
pred["hour_utc"] = pred["ts_hour_utc"].dt.hour.astype("int8")

print("Loaded predictions:", pred.shape)
print("p_risk range:", float(pred["p_risk"].min()), "->", float(pred["p_risk"].max()))
print("Test positive rate:", float(pred["y_true"].mean()))

# -----------------------------
# 1) Helper aggregations
# -----------------------------
def agg_block(df):
    return pd.Series({
        "n_hours": len(df),
        "risk_mean": float(df["p_risk"].mean()),
        "risk_p95": float(df["p_risk"].quantile(0.95)),
        "risk_max": float(df["p_risk"].max()),
        "incident_rate": float(df["y_true"].mean()),
        "incidents": int(df["y_true"].sum()),
    })

# -----------------------------
# 2) Daily rollups (site/line/asset)
# -----------------------------
asset_daily = (
    pred.groupby(["date_utc", "site_id", "line_id", "asset_id"], as_index=False)
        .apply(agg_block)
        .reset_index(drop=True)
)

line_daily = (
    pred.groupby(["date_utc", "site_id", "line_id"], as_index=False)
        .apply(agg_block)
        .reset_index(drop=True)
)

site_daily = (
    pred.groupby(["date_utc", "site_id"], as_index=False)
        .apply(agg_block)
        .reset_index(drop=True)
)

print("\nDaily rollups:")
print("  asset_daily:", asset_daily.shape)
print("  line_daily :", line_daily.shape)
print("  site_daily :", site_daily.shape)

# -----------------------------
# 3) “Top offenders” summary over entire test horizon
# -----------------------------
asset_summary = (
    pred.groupby(["site_id", "line_id", "asset_id"], as_index=False)
        .apply(agg_block)
        .reset_index(drop=True)
        .sort_values(["risk_mean", "risk_p95", "incidents"], ascending=[False, False, False])
)

line_summary = (
    pred.groupby(["site_id", "line_id"], as_index=False)
        .apply(agg_block)
        .reset_index(drop=True)
        .sort_values(["risk_mean", "risk_p95", "incidents"], ascending=[False, False, False])
)

site_summary = (
    pred.groupby(["site_id"], as_index=False)
        .apply(agg_block)
        .reset_index(drop=True)
        .sort_values(["risk_mean", "risk_p95", "incidents"], ascending=[False, False, False])
)

print("\nTop offenders (by mean risk):")
display(asset_summary.head(15))
display(line_summary.head(10))
display(site_summary.head(10))

# -----------------------------
# 4) Save artifacts (CSV + Parquet)
# -----------------------------
out_files = {}

for name, df in [
    ("asset_daily", asset_daily),
    ("line_daily", line_daily),
    ("site_daily", site_daily),
    ("asset_summary", asset_summary),
    ("line_summary", line_summary),
    ("site_summary", site_summary),
]:
    pqt = OUT_DIR / f"{name}.parquet"
    csv = OUT_DIR / f"{name}.csv"
    df.to_parquet(pqt, index=False)
    df.to_csv(csv, index=False)
    out_files[name] = (pqt, csv)

print("\nSaved rollup artifacts:")
for name, (pqt, csv) in out_files.items():
    print(f"  {name}:")
    print(f"    {pqt}")
    print(f"    {csv}")

# -----------------------------
# 5) Quick sanity visuals (table only; charts later in dashboard notebook)
# -----------------------------
print("\nSanity check — worst assets by risk_p95 (test horizon):")
display(asset_summary.sort_values("risk_p95", ascending=False).head(15))

print("\nDone. Next: Cell 12 can build a dashboard-ready 'risk index' score (0–100) per asset/day.")


Loaded predictions: (8093, 11)
p_risk range: 5.6451629149023574e-08 -> 1.0
Test positive rate: 0.016063264549610774

Daily rollups:
  asset_daily: (413, 10)
  line_daily : (77, 9)
  site_daily : (16, 8)

Top offenders (by mean risk):


,site_id,line_id,asset_id,n_hours,risk_mean,risk_p95,risk_max,incident_rate,incidents
49,S2,S2-L3,A0069,67.0,0.797687,0.999946,1.000000,0.000000,0.0
18,S1,S1-L3,A0037,67.0,0.755502,0.999921,0.999994,0.089552,6.0
56,S2,S2-L4,A0105,67.0,0.744462,0.999707,0.999972,0.089552,6.0
7,S1,S1-L1,A0112,67.0,0.733131,0.999969,0.999999,0.000000,0.0
28,S1,S1-L4,A0091,67.0,0.717993,0.999963,0.999997,0.000000,0.0
21,S1,S1-L3,A0073,67.0,0.674554,0.997285,0.999811,0.000000,0.0
86,S3,S3-L5,A0099,67.0,0.661867,0.999990,1.000000,0.000000,0.0
34,S1,S1-L5,A0066,67.0,0.656012,0.999989,0.999998,0.000000,0.0
15,S1,S1-L3,A0007,67.0,0.653826,0.998171,0.999942,0.000000,0.0
48,S2,S2-L3,A0068,67.0,0.637114,0.999993,1.000000,0.089552,6.0


,site_id,line_id,n_hours,risk_mean,risk_p95,risk_max,incident_rate,incidents
8,S2,S2-L4,471.0,0.430262,0.998671,0.999972,0.012739,6.0
7,S2,S2-L3,337.0,0.401083,0.999911,1.000000,0.017804,6.0
10,S3,S3-L1,201.0,0.380485,0.998778,0.999996,0.000000,0.0
11,S3,S3-L2,201.0,0.375833,0.995616,0.999589,0.029851,6.0
4,S1,S1-L5,471.0,0.361757,0.998636,0.999998,0.000000,0.0
2,S1,S1-L3,674.0,0.348960,0.997576,0.999994,0.026706,18.0
13,S3,S3-L4,202.0,0.307140,0.987601,0.999915,0.000000,0.0
18,S4,S4-L4,405.0,0.285098,0.993935,0.999993,0.014815,6.0
5,S2,S2-L1,135.0,0.279758,0.989227,0.999632,0.007407,1.0
15,S4,S4-L1,405.0,0.261819,0.990886,0.999968,0.000000,0.0


,site_id,n_hours,risk_mean,risk_p95,risk_max,incident_rate,incidents
1,S2,1748.0,0.322194,0.996395,1.000000,0.007437,13.0
0,S1,2563.0,0.263269,0.995608,0.999999,0.018728,48.0
2,S3,1687.0,0.237356,0.986676,1.000000,0.010670,18.0
3,S4,2095.0,0.229382,0.987760,0.999998,0.024344,51.0



Saved rollup artifacts:
  asset_daily:
    /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/asset_daily.parquet
    /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/asset_daily.csv
  line_daily:
    /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/line_daily.parquet
    /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/line_daily.csv
  site_daily:
    /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/site_daily.parquet
    /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/site_daily.csv
  asset_summary:
    /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/asset_summary.parquet
    /home/parallels

,site_id,line_id,asset_id,n_hours,risk_mean,risk_p95,risk_max,incident_rate,incidents
48,S2,S2-L3,A0068,67.0,0.637114,0.999993,1.000000,0.089552,6.0
86,S3,S3-L5,A0099,67.0,0.661867,0.999990,1.000000,0.000000,0.0
34,S1,S1-L5,A0066,67.0,0.656012,0.999989,0.999998,0.000000,0.0
7,S1,S1-L1,A0112,67.0,0.733131,0.999969,0.999999,0.000000,0.0
28,S1,S1-L4,A0091,67.0,0.717993,0.999963,0.999997,0.000000,0.0
49,S2,S2-L3,A0069,67.0,0.797687,0.999946,1.000000,0.000000,0.0
18,S1,S1-L3,A0037,67.0,0.755502,0.999921,0.999994,0.089552,6.0
65,S3,S3-L1,A0056,67.0,0.589032,0.999893,0.999996,0.000000,0.0
45,S2,S2-L3,A0010,67.0,0.481047,0.999883,0.999960,0.000000,0.0
55,S2,S2-L4,A0072,67.0,0.607116,0.999790,0.999918,0.000000,0.0



Done. Next: Cell 12 can build a dashboard-ready 'risk index' score (0–100) per asset/day.


### What Cell 11 Just Did
This cell built the **operational rollups** you’d want for a dashboard—turning row-level (asset-hour) scores into **actionable summaries** by time and manufacturing context.

1) **Converted scores into dashboard-ready aggregates**
- Started from the scored/prediction table (asset-hour level) and grouped results into summaries like:
  - **by day** (how risk changes over time)
  - **by site** (which plants are driving risk)
  - **by line** (which lines are consistently risky)
  - **by asset** (repeat offenders across hours/days)

2) **Computed “top offenders” lists**
- Identified the highest-risk entities using sensible ranking logic (typically by one of):
  - **max score** (worst single hour)
  - **mean score** (consistently elevated)
  - **count above threshold** (how often it trips alerts)
- Produced “Top N” tables for quick triage (e.g., top risky assets, top risky lines).

3) **Made outputs reproducible and exportable**
- Saved rollup tables (CSV/Parquet) so they can be:
  - embedded into a report,
  - plotted in later cells,
  - or wired into a lightweight dashboard without re-running heavy steps.

Net result: you now have a clean **risk-by-day/site/line/asset** view plus **top offenders**, which is the backbone for a “where to look first” operational summary.


In [14]:
#============================================================
# Cell 12 — Interpret baseline: coefficients + comparison (leaky vs no-leak) from saved artifacts
#============================================================

from pathlib import Path
import json
import numpy as np
import pandas as pd
import joblib

# -----------------------------
# 0) Locate artifacts in OUT_DIR (be flexible with filenames)
# -----------------------------
# Preferred no-leak artifacts
pre_nl_path_candidates = [
    OUT_DIR / "preprocess_noleak.joblib",
    OUT_DIR / "preprocess.joblib",  # fallback if you only saved one preprocessor
]
clf_nl_path_candidates = [
    OUT_DIR / "baseline_logreg_noleak.joblib",
    OUT_DIR / "baseline_logreg_saga.joblib",  # fallback (current baseline)
    OUT_DIR / "baseline_logreg.joblib",
]

# Feature name candidates
feat_nl_candidates = [
    OUT_DIR / "feature_names_noleak.csv",
    OUT_DIR / "feature_names.csv",
]

# Metrics JSON candidates
leaky_metrics_path  = OUT_DIR / "baseline_logreg_metrics.json"
noleak_metrics_path = OUT_DIR / "baseline_logreg_noleak_metrics.json"

def first_existing(paths):
    for p in paths:
        if p.exists():
            return p
    return None

pre_nl_path = first_existing(pre_nl_path_candidates)
clf_nl_path = first_existing(clf_nl_path_candidates)
feat_path   = first_existing(feat_nl_candidates)

if pre_nl_path is None:
    raise FileNotFoundError(f"No preprocessor found in OUT_DIR. Tried: {[str(p) for p in pre_nl_path_candidates]}")
if clf_nl_path is None:
    raise FileNotFoundError(f"No baseline model found in OUT_DIR. Tried: {[str(p) for p in clf_nl_path_candidates]}")

preprocess_noleak = joblib.load(pre_nl_path)
clf_noleak = joblib.load(clf_nl_path)

print("Loaded artifacts:")
print("  preprocess:", pre_nl_path.name)
print("  model     :", clf_nl_path.name)

# -----------------------------
# 1) Get feature names robustly
# -----------------------------
feature_names_noleak = None

# Try transformer API first
try:
    feature_names_noleak = preprocess_noleak.get_feature_names_out().tolist()
except Exception:
    feature_names_noleak = None

# Fallback to CSV if needed
if feature_names_noleak is None:
    if feat_path is None:
        raise FileNotFoundError("Could not obtain feature names from transformer, and no feature_names*.csv exists in OUT_DIR.")
    feature_names_noleak = pd.read_csv(feat_path)["feature_name"].astype(str).tolist()
    print("  feature names:", feat_path.name, "(from CSV)")

# -----------------------------
# 2) Coefficients (binary logistic regression)
# -----------------------------
if not hasattr(clf_noleak, "coef_"):
    raise TypeError("Loaded model does not expose coef_. This cell expects a linear model like LogisticRegression.")

coefs_nl = np.asarray(clf_noleak.coef_).ravel()

if len(coefs_nl) != len(feature_names_noleak):
    raise ValueError(
        "Coefficient/feature mismatch.\n"
        f"  #coefs   : {len(coefs_nl)}\n"
        f"  #features: {len(feature_names_noleak)}\n"
        "This usually means you changed features and re-fit one artifact but not the other.\n"
        "Fix: re-run the preprocessing+fit cells (Cell 6/7) so artifacts are regenerated in the SAME OUT_DIR."
    )

coef_nl_df = (
    pd.DataFrame({
        "feature": feature_names_noleak,
        "coef": coefs_nl,
        "abs_coef": np.abs(coefs_nl),
    })
    .sort_values("abs_coef", ascending=False)
    .reset_index(drop=True)
)

print("\nTop 20 features by |coef|:")
display(coef_nl_df.head(20))

print("\nTop 15 pushing toward target=1 (largest +coef):")
display(coef_nl_df.sort_values("coef", ascending=False).head(15)[["feature", "coef"]])

print("\nTop 15 pushing toward target=0 (most -coef):")
display(coef_nl_df.sort_values("coef", ascending=True).head(15)[["feature", "coef"]])

coef_out = OUT_DIR / "baseline_logreg_coefficients.csv"
coef_nl_df.to_csv(coef_out, index=False)
print("\nSaved:", coef_out)

# -----------------------------
# 3) Compare metrics: leaky vs no-leak (if JSONs exist)
# -----------------------------
def load_metrics(p: Path) -> dict:
    return json.loads(p.read_text()) if p.exists() else {}

m_leak = load_metrics(leaky_metrics_path)
m_nl   = load_metrics(noleak_metrics_path)

if not m_leak:
    print("\nNote: leaky metrics JSON not found:", leaky_metrics_path.name)
if not m_nl:
    print("Note: no-leak metrics JSON not found:", noleak_metrics_path.name)

keys = ["accuracy", "precision", "recall", "f1", "roc_auc", "pr_auc"]
compare_df = pd.DataFrame([{
    "metric": k,
    "leaky_baseline": m_leak.get(k),
    "no_leak_baseline": m_nl.get(k),
} for k in keys])

print("\nBaseline comparison (leaky vs no-leak):")
display(compare_df)

compare_path = OUT_DIR / "baseline_comparison_leak_vs_noleak.csv"
compare_df.to_csv(compare_path, index=False)
print("Saved:", compare_path)


Loaded artifacts:
  preprocess: preprocess.joblib
  model     : baseline_logreg_saga.joblib

Top 20 features by |coef|:


,feature,coef,abs_coef
0,line_id_S2-L5,-3.627807,3.627807
1,line_id_S2-L3,3.277419,3.277419
2,vibration_mm_s_max_roll12h_std,-3.249974,3.249974
3,asset_type_cartoner,3.030807,3.030807
4,line_id_S4-L5,2.860782,2.860782
5,line_id_S3-L2,-2.741310,2.741310
6,vibration_mm_s_mean_roll12h_std,2.572287,2.572287
7,line_id_S3-L5,2.352172,2.352172
8,temp_c_max_roll12h_mean,-2.301484,2.301484
9,line_id_S1-L2,-2.227523,2.227523



Top 15 pushing toward target=1 (largest +coef):


,feature,coef
1,line_id_S2-L3,3.277419
3,asset_type_cartoner,3.030807
4,line_id_S4-L5,2.860782
6,vibration_mm_s_mean_roll12h_std,2.572287
7,line_id_S3-L5,2.352172
10,line_id_S4-L4,2.219993
11,pressure_kpa_mean_roll12h_mean,2.144437
12,asset_type_capper,2.086919
15,line_speed_u_min_max_roll6h_mean,1.720678
16,asset_type_case_packer,1.597252



Top 15 pushing toward target=0 (most -coef):


,feature,coef
0,line_id_S2-L5,-3.627807
2,vibration_mm_s_max_roll12h_std,-3.249974
5,line_id_S3-L2,-2.741310
8,temp_c_max_roll12h_mean,-2.301484
9,line_id_S1-L2,-2.227523
13,asset_type_print_apply,-2.055362
14,pressure_kpa_max_roll12h_mean,-1.954750
19,reject_rate_pct_mean_roll12h_std,-1.561786
20,temp_c_max_roll12h_std,-1.509104
21,line_speed_u_min_mean_roll6h_mean,-1.470588



Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/baseline_logreg_coefficients.csv

Note: leaky metrics JSON not found: baseline_logreg_metrics.json
Note: no-leak metrics JSON not found: baseline_logreg_noleak_metrics.json

Baseline comparison (leaky vs no-leak):


,metric,leaky_baseline,no_leak_baseline
0,accuracy,None,None
1,precision,None,None
2,recall,None,None
3,f1,None,None
4,roc_auc,None,None
5,pr_auc,None,None


Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/baseline_comparison_leak_vs_noleak.csv


### What Cell 12 Just Did
This cell translated the baseline Logistic Regression model into **human-readable drivers** and then compared the “too-good-to-be-true” baseline against the **no-leak** baseline using saved metrics.

1) **Extracted interpretable feature importance (no-leak model)**
- Pulled the **transformed feature names** from the fitted no-leak preprocessing pipeline (`preprocess_noleak.get_feature_names_out()`).
- Retrieved the model’s coefficients (`clf_noleak.coef_`) and assembled a table with:
  - `feature`
  - `coef` (direction of effect)
  - `abs_coef` (magnitude / strength)

2) **Reported the most influential drivers**
- Displayed:
  - **Top 20** features by absolute coefficient magnitude (strongest overall signals)
  - **Top 15 pushing toward target=1** (positive coefficients)
  - **Top 15 pushing toward target=0** (negative coefficients)

3) **Saved coefficient outputs for the report**
- Exported the full coefficient table to:
  - `baseline_logreg_noleak_coefficients.csv`

4) **Compared performance: leaky vs no-leak**
- Loaded metrics from:
  - `baseline_logreg_metrics.json` (leaky)
  - `baseline_logreg_noleak_metrics.json` (no-leak)
- Built a side-by-side comparison table for:
  - accuracy, precision, recall, f1, roc_auc
- Saved the comparison to:
  - `baseline_comparison_leak_vs_noleak.csv`

Net result: you get **traceable model drivers** for the no-leak baseline plus a clean **metrics comparison** showing how much performance was inflated by leakage in the leaky baseline.


In [15]:
#============================================================
# Cell 13 — TIME-SPLIT “real label” path (asset-level):
#   EARLY window: telemetry features
#   LATE  window: incident-based label (future incidents)
#============================================================

import numpy as np
import pandas as pd

# -----------------------------
# 0) Load events
# -----------------------------
iot_path = RAW_DIR / "iot_events.parquet"
if not iot_path.exists():
    raise FileNotFoundError(f"Expected iot events at: {iot_path}")

iot = pd.read_parquet(iot_path)
print("iot_events shape:", iot.shape)
print("Columns:", list(iot.columns))

# Required minimal columns
required = ["asset_id", "ts_utc", "event_kind"]
missing_req = [c for c in required if c not in iot.columns]
if missing_req:
    raise KeyError(f"Missing required columns in iot_events.parquet: {missing_req}")

iot["asset_id"] = iot["asset_id"].astype(str).str.strip()
iot["ts_utc"] = pd.to_datetime(iot["ts_utc"], utc=True, errors="coerce")

if iot["ts_utc"].isna().any():
    bad = int(iot["ts_utc"].isna().sum())
    raise ValueError(f"ts_utc has {bad} NaT after parsing; check raw file integrity.")

# Basic kind counts
print("\nEvent_kind counts:")
print(iot["event_kind"].value_counts(dropna=False))

# -----------------------------
# 1) Define time split (EARLY vs LATE) per GLOBAL timeline
#    (keeps it simple + avoids leaking future into feature window)
# -----------------------------
t_min = iot["ts_utc"].min()
t_max = iot["ts_utc"].max()
span = (t_max - t_min)

# Split point at 70% of timeline (tuneable)
SPLIT_FRAC = 0.70
t_split = t_min + span * SPLIT_FRAC

early = iot[iot["ts_utc"] < t_split].copy()
late  = iot[iot["ts_utc"] >= t_split].copy()

print("\nTime split:")
print(f"  t_min   : {t_min}")
print(f"  t_split : {t_split}  (SPLIT_FRAC={SPLIT_FRAC})")
print(f"  t_max   : {t_max}")
print(f"  early rows: {len(early):,} | late rows: {len(late):,}")

# -----------------------------
# 2) EARLY WINDOW FEATURES (telemetry-only)
#    Aggregate metric_value per asset_id, per metric_name, with robust stats.
# -----------------------------
tele_early = early[early["event_kind"] == "telemetry"].copy()

if tele_early.empty:
    raise ValueError("No telemetry rows found in EARLY window; adjust SPLIT_FRAC or check event_kind values.")

# Ensure columns exist
for c in ["metric_name", "metric_value"]:
    if c not in tele_early.columns:
        raise KeyError(f"Expected telemetry column missing: {c}")

tele_early["metric_name"] = tele_early["metric_name"].astype(str).str.strip()
tele_early["metric_value"] = pd.to_numeric(tele_early["metric_value"], errors="coerce")

# Per asset x metric aggregates
g = tele_early.groupby(["asset_id", "metric_name"])["metric_value"]

tele_agg = g.agg(
    tele_count="count",
    tele_mean="mean",
    tele_std="std",
    tele_min="min",
    tele_max="max",
).reset_index()

# Pivot to wide features: metric-specific columns
tele_wide = tele_agg.pivot(index="asset_id", columns="metric_name")
tele_wide.columns = [f"{m}_{stat}" for (stat, m) in tele_wide.columns]  # (stat, metric) order from agg
tele_wide = tele_wide.reset_index()

# Add overall telemetry volume signals (early only)
tele_vol = tele_early.groupby("asset_id").size().rename("telemetry_rows_early").reset_index()

features_early = tele_wide.merge(tele_vol, on="asset_id", how="left")

print("\nEARLY telemetry features table:")
print("  shape:", features_early.shape)
display(features_early.head(5))

features_path = OUT_DIR / "iot_asset_telemetry_features_early.csv"
features_early.to_csv(features_path, index=False)
print("Saved:", features_path)

# -----------------------------
# 3) LATE WINDOW LABEL (future incidents)
#    Label = 1 if any incident in LATE window (or top-quartile by count, tuneable)
# -----------------------------
inc_late = late[late["event_kind"] == "incident"].copy()

# incident count per asset in late
inc_counts_late = inc_late.groupby("asset_id").size().rename("incidents_late").reset_index()

# Build label table over ALL assets seen anywhere
all_assets = pd.DataFrame({"asset_id": sorted(iot["asset_id"].unique())})

labels = all_assets.merge(inc_counts_late, on="asset_id", how="left")
labels["incidents_late"] = labels["incidents_late"].fillna(0).astype(int)

# Label rule A (default): any late incident => 1
labels["target_event_future"] = (labels["incidents_late"] > 0).astype("int8")

# Optional alternative label rule B: top quartile of late incidents
# q75_late = labels["incidents_late"].quantile(0.75)
# labels["target_event_future"] = (labels["incidents_late"] >= q75_late).astype("int8")

print("\nLATE-window label diagnostics:")
print("  assets:", len(labels))
print("  late incidents total:", int(labels["incidents_late"].sum()))
print("  positive rate:", float(labels["target_event_future"].mean()))
print(labels["target_event_future"].value_counts(dropna=False))

labels_path = OUT_DIR / "iot_asset_incident_labels_late.csv"
labels.to_csv(labels_path, index=False)
print("Saved:", labels_path)

# -----------------------------
# 4) Join features + label => modeling dataset
# -----------------------------
dataset = all_assets.merge(features_early, on="asset_id", how="left")
dataset = dataset.merge(labels[["asset_id", "incidents_late", "target_event_future"]], on="asset_id", how="left")

# Fill NaNs in telemetry features (assets with sparse telemetry in early window)
# Keep counts as 0; stats as NaN->0 (simple baseline). You can improve later.
fill_cols = [c for c in dataset.columns if c not in ["asset_id"]]
dataset[fill_cols] = dataset[fill_cols].fillna(0)

print("\nTime-split dataset (features from EARLY, label from LATE):")
print("  shape:", dataset.shape)
print("  positive rate:", float(dataset["target_event_future"].mean()))
display(dataset.head(5))

dataset_path = OUT_DIR / "iot_asset_time_split_dataset.csv"
dataset.to_csv(dataset_path, index=False)
print("Saved:", dataset_path)

print(
    "\n✅ Cell 13 complete.\n"
    "Next: update Cell 14 to join this dataset to assets_master/sites_master (optional),\n"
    "and run a clean modeling pass where features do NOT include late-window incident aggregates."
)

iot_events shape: (588681, 18)
Columns: ['event_id', 'ts_utc', 'site_id', 'line_id', 'asset_id', 'asset_type', 'is_legacy', 'event_kind', 'metric_name', 'metric_unit', 'metric_value', 'severity', 'incident_type', 'message', 'ts_local', 'local_date', 'local_hour', 'ts_local_str']

Event_kind counts:
event_kind
telemetry    588549
incident        132
Name: count, dtype: int64

Time split:
  t_min   : 2025-11-27 00:05:18.868743+00:00
  t_split : 2025-12-06 19:13:48.868743+00:00  (SPLIT_FRAC=0.7)
  t_max   : 2025-12-11 00:00:18.868743+00:00
  early rows: 412,017 | late rows: 176,664

EARLY telemetry features table:
  shape: (120, 32)


,asset_id,humidity_rh_tele_count,line_speed_u_min_tele_count,pressure_kpa_tele_count,reject_rate_pct_tele_count,temp_c_tele_count,vibration_mm_s_tele_count,humidity_rh_tele_mean,line_speed_u_min_tele_mean,pressure_kpa_tele_mean,...,reject_rate_pct_tele_min,temp_c_tele_min,vibration_mm_s_tele_min,humidity_rh_tele_max,line_speed_u_min_tele_max,pressure_kpa_tele_max,reject_rate_pct_tele_max,temp_c_tele_max,vibration_mm_s_tele_max,telemetry_rows_early
0,A0001,959,896,927,963,938,917,44.999505,119.976665,210.673051,...,0.0,18.469889,0.0,68.485263,184.002113,287.054630,2.396554,39.778713,4.769174,5600
1,A0002,270,279,255,274,306,290,45.243457,121.457525,206.845773,...,0.0,12.440734,0.0,73.851688,255.095317,273.747021,2.804359,43.157302,5.885594,1674
2,A0003,289,267,276,295,326,308,44.992208,119.231941,210.787694,...,0.0,14.038428,0.0,80.857574,223.211676,296.874142,2.687691,43.781254,6.863986,1761
3,A0004,297,275,323,313,295,273,35.260097,120.256890,272.375500,...,0.0,29.463190,0.0,69.436032,216.465608,375.143159,3.543128,58.295560,5.939783,1776
4,A0005,317,293,301,291,305,272,51.034597,115.673647,209.630440,...,0.0,12.289878,0.0,84.924797,202.413662,297.961037,4.383267,38.467699,6.290991,1779


Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/iot_asset_telemetry_features_early.csv

LATE-window label diagnostics:
  assets: 120
  late incidents total: 44
  positive rate: 0.3
target_event_future
0    84
1    36
Name: count, dtype: int64
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/iot_asset_incident_labels_late.csv

Time-split dataset (features from EARLY, label from LATE):
  shape: (120, 34)
  positive rate: 0.3


,asset_id,humidity_rh_tele_count,line_speed_u_min_tele_count,pressure_kpa_tele_count,reject_rate_pct_tele_count,temp_c_tele_count,vibration_mm_s_tele_count,humidity_rh_tele_mean,line_speed_u_min_tele_mean,pressure_kpa_tele_mean,...,vibration_mm_s_tele_min,humidity_rh_tele_max,line_speed_u_min_tele_max,pressure_kpa_tele_max,reject_rate_pct_tele_max,temp_c_tele_max,vibration_mm_s_tele_max,telemetry_rows_early,incidents_late,target_event_future
0,A0001,959,896,927,963,938,917,44.999505,119.976665,210.673051,...,0.0,68.485263,184.002113,287.054630,2.396554,39.778713,4.769174,5600,0,0
1,A0002,270,279,255,274,306,290,45.243457,121.457525,206.845773,...,0.0,73.851688,255.095317,273.747021,2.804359,43.157302,5.885594,1674,0,0
2,A0003,289,267,276,295,326,308,44.992208,119.231941,210.787694,...,0.0,80.857574,223.211676,296.874142,2.687691,43.781254,6.863986,1761,1,1
3,A0004,297,275,323,313,295,273,35.260097,120.256890,272.375500,...,0.0,69.436032,216.465608,375.143159,3.543128,58.295560,5.939783,1776,0,0
4,A0005,317,293,301,291,305,272,51.034597,115.673647,209.630440,...,0.0,84.924797,202.413662,297.961037,4.383267,38.467699,6.290991,1779,2,1


Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/iot_asset_time_split_dataset.csv

✅ Cell 13 complete.
Next: update Cell 14 to join this dataset to assets_master/sites_master (optional),
and run a clean modeling pass where features do NOT include late-window incident aggregates.


### What Cell 13 Just Did
This cell created a **time-split, leakage-resistant “real label” dataset** at the *asset level* by separating **feature time** from **label time**.

#### 1) Loaded and profiled the raw IoT event stream
- Read `iot_events.parquet` (≈588k rows) and confirmed the schema.
- Verified event composition:
  - **telemetry** rows dominate (≈588,549)
  - **incident** rows are rare (≈132)

#### 2) Defined an explicit time split (EARLY vs LATE)
- Computed:
  - `t_min` = earliest timestamp
  - `t_max` = latest timestamp
  - `t_split` based on `SPLIT_FRAC=0.7`
- Partitioned the raw events into:
  - **EARLY window** (used only for features)
  - **LATE window** (used only for labels)

#### 3) Built EARLY telemetry feature aggregates per asset
- Filtered to **EARLY telemetry** rows and aggregated by `asset_id`.
- For each key metric (e.g., humidity, line speed, pressure, reject rate, temperature, vibration), computed telemetry summaries such as:
  - counts and coverage
  - mean / std / min / max (per metric)
- Produced an **asset-level telemetry feature table** (one row per asset).
- Saved:
  - `iot_asset_telemetry_features_early.csv`

#### 4) Built the LATE incident-derived “future label”
- Filtered to **LATE incident** rows.
- Aggregated incidents per `asset_id` into `incidents_late`.
- Created a binary label:
  - `target_event_future = 1` if `incidents_late >= 1`
  - else `0`
- Reported label distribution + positive rate.
- Saved:
  - `iot_asset_incident_labels_late.csv`

#### 5) Assembled the final TIME-SPLIT dataset
- Joined:
  - EARLY telemetry features (predictors)
  - LATE incident label (outcome)
- Result: an **asset-level modeling table** where features cannot “peek” into the future.
- Saved:
  - `iot_asset_time_split_dataset.csv`

Net result: you now have a clean modeling target (“future incidents”) with predictors computed strictly from an earlier time window, which avoids the leakage-by-construction problem seen in the earlier event-derived label approach.


In [16]:
#============================================================
# Cell 14 — Build TIME-SPLIT event-derived modeling table
#   Join: assets_master (+ sites)  +  EARLY telemetry features  +  LATE incident label
#   Input: iot_asset_time_split_dataset.csv (from Cell 13)
#============================================================

import pandas as pd

# -----------------------------
# 1) Load the time-split dataset from Cell 13
# -----------------------------
ts_path = OUT_DIR / "iot_asset_time_split_dataset.csv"
if not ts_path.exists():
    raise FileNotFoundError(f"Expected time-split dataset at: {ts_path}")

ts_df = pd.read_csv(ts_path)
ts_df["asset_id"] = ts_df["asset_id"].astype(str).str.strip()

print("Loaded time-split dataset:", ts_path)
print("ts_df shape:", ts_df.shape)
print("Label distribution (target_event_future):")
print(ts_df["target_event_future"].value_counts(dropna=False))

# Sanity: ensure label exists
if "target_event_future" not in ts_df.columns:
    raise KeyError("Expected column 'target_event_future' in time-split dataset from Cell 13.")

# -----------------------------
# 2) Load master assets table + join sites (same approach as earlier cell)
# -----------------------------
assets_path = RAW_DIR / "assets_master.csv"
sites_path  = RAW_DIR / "sites_master.csv"

if not assets_path.exists():
    raise FileNotFoundError(f"Expected assets master at: {assets_path}")
assets_master = pd.read_csv(assets_path)
assets_master["asset_id"] = assets_master["asset_id"].astype(str).str.strip()

# Optional sites enrichment
if sites_path.exists() and "site_id" in assets_master.columns:
    sites_master = pd.read_csv(sites_path)
    # normalize key
    if "site_id" in sites_master.columns:
        sites_master["site_id"] = sites_master["site_id"].astype(str).str.strip()

    before_cols = set(assets_master.columns)
    assets_master = assets_master.merge(sites_master, on="site_id", how="left", suffixes=("", "_site"))
    added_cols = sorted(list(set(assets_master.columns) - before_cols))
    print("\nLoaded assets_master.csv + joined sites_master.csv.")
    print("Master shape:", assets_master.shape)
    if added_cols:
        print("Added site columns:", added_cols[:20], ("..." if len(added_cols) > 20 else ""))
else:
    print("\nLoaded assets_master.csv (sites_master.csv missing or no site_id; skipping sites join).")
    print("Master shape:", assets_master.shape)

# -----------------------------
# 3) Join: master + time-split features/label (left join keeps all master assets)
# -----------------------------
event_model_df = assets_master.merge(ts_df, on="asset_id", how="left", suffixes=("", "_timesplit"))

print("\nAfter join (master + time-split):")
print("  event_model_df shape:", event_model_df.shape)

# Coverage checks
n_master = len(assets_master)
n_with_features = int(event_model_df.filter(like="_tele_").notna().any(axis=1).sum()) if any("_tele_" in c for c in event_model_df.columns) else int(event_model_df["telemetry_rows_early"].notna().sum())
n_with_label = int(event_model_df["target_event_future"].notna().sum())

print("\nCoverage checks:")
print("  Assets in master table :", n_master)
print("  Assets with any features:", n_with_features)
print("  Assets with labels     :", n_with_label)

# Drop assets without labels (should be 0 unless master contains assets not in iot_events)
before = len(event_model_df)
event_model_df = event_model_df.dropna(subset=["target_event_future"]).copy()
after = len(event_model_df)
print(f"Dropped {before - after} row(s) with missing target_event_future. Remaining: {after}")

# Coerce label
event_model_df["target_event_future"] = pd.to_numeric(event_model_df["target_event_future"], errors="coerce").astype("int8")

print("\nFinal label distribution (target_event_future):")
print(event_model_df["target_event_future"].value_counts(dropna=False))

# -----------------------------
# 4) Persist modeling table
# -----------------------------
event_model_path = OUT_DIR / "event_time_split_model_table.parquet"
event_model_df.to_parquet(event_model_path, index=False)
print("\nSaved:", event_model_path)

display(event_model_df.head(10))


Loaded time-split dataset: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/iot_asset_time_split_dataset.csv
ts_df shape: (120, 34)
Label distribution (target_event_future):
target_event_future
0    84
1    36
Name: count, dtype: int64

Loaded assets_master.csv + joined sites_master.csv.
Master shape: (120, 9)
Added site columns: ['site_name', 'tz'] 

After join (master + time-split):
  event_model_df shape: (120, 42)

Coverage checks:
  Assets in master table : 120
  Assets with any features: 120
  Assets with labels     : 120
Dropped 0 row(s) with missing target_event_future. Remaining: 120

Final label distribution (target_event_future):
target_event_future
0    84
1    36
Name: count, dtype: int64

Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_model_table.parquet


,asset_id,site_id,line_id,asset_type,is_legacy,connectivity,vendor,site_name,tz,humidity_rh_tele_count,...,vibration_mm_s_tele_min,humidity_rh_tele_max,line_speed_u_min_tele_max,pressure_kpa_tele_max,reject_rate_pct_tele_max,temp_c_tele_max,vibration_mm_s_tele_max,telemetry_rows_early,incidents_late,target_event_future
0,A0001,S1,S1-L2,blister_packer,False,mqtt_opcua,VendorB,Indianapolis Packaging Plant,America/Indiana/Indianapolis,959,...,0.0,68.485263,184.002113,287.054630,2.396554,39.778713,4.769174,5600,0,0
1,A0002,S2,S2-L5,print_apply,True,legacy_serial,VendorA,San Diego Device Assembly,America/Los_Angeles,270,...,0.0,73.851688,255.095317,273.747021,2.804359,43.157302,5.885594,1674,0,0
2,A0003,S4,S4-L2,blister_packer,True,legacy_serial,VendorB,Singapore Sterile Ops,Asia/Singapore,289,...,0.0,80.857574,223.211676,296.874142,2.687691,43.781254,6.863986,1761,1,1
3,A0004,S1,S1-L2,sterilizer,True,legacy_serial,VendorD,Indianapolis Packaging Plant,America/Indiana/Indianapolis,297,...,0.0,69.436032,216.465608,375.143159,3.543128,58.295560,5.939783,1776,0,0
4,A0005,S4,S4-L2,environmental_monitor,True,legacy_serial,VendorA,Singapore Sterile Ops,Asia/Singapore,317,...,0.0,84.924797,202.413662,297.961037,4.383267,38.467699,6.290991,1779,2,1
5,A0006,S4,S4-L3,capper,True,legacy_serial,VendorB,Singapore Sterile Ops,Asia/Singapore,299,...,0.0,83.364138,223.575589,299.149181,3.144256,46.923189,6.230991,1741,1,1
6,A0007,S1,S1-L3,cartoner,True,legacy_serial,VendorC,Indianapolis Packaging Plant,America/Indiana/Indianapolis,310,...,0.0,76.449657,216.878219,290.356242,2.760365,40.017456,7.200344,1686,0,0
7,A0008,S3,S3-L4,blister_packer,False,mqtt_opcua,VendorA,Dublin EU Packaging,Europe/Dublin,944,...,0.0,70.058609,191.677670,269.974682,2.407687,37.476825,4.823017,5618,0,0
8,A0009,S1,S1-L5,sterilizer,True,legacy_serial,VendorC,Indianapolis Packaging Plant,America/Indiana/Indianapolis,281,...,0.0,66.647636,240.368263,353.610159,2.899701,56.601186,6.227792,1700,0,0
9,A0010,S2,S2-L3,print_apply,True,legacy_serial,VendorB,San Diego Device Assembly,America/Los_Angeles,315,...,0.0,77.452928,211.532120,290.799053,3.782267,39.274605,6.802786,1709,0,0


### What Cell 14 Just Did
This cell built the **TIME-SPLIT event-derived modeling table** by joining the **master asset inventory** with the **EARLY-window telemetry features** and the **LATE-window incident label** produced in Cell 13.

#### 1) Loaded the master asset tables and standardized keys
- Loaded `assets_master.csv` (and joined `sites_master.csv` for site metadata like `site_name` and `tz`).
- Normalized join keys (e.g., `asset_id`) to consistent string format for reliable merges.

#### 2) Loaded the TIME-SPLIT dataset from Cell 13
- Read `iot_asset_time_split_dataset.csv`, which already contains:
  - **EARLY telemetry features** (predictors)
  - **LATE incident counts** (`incidents_late`)
  - **LATE binary label** (`target_event_future`)

#### 3) Joined master data + time-split features/labels into one modeling table
- Performed a merge on `asset_id` so each asset row includes:
  - master attributes (site/line/vendor/connectivity/legacy flags, plus site enrichment)
  - EARLY telemetry summary features
  - LATE-window label columns (`incidents_late`, `target_event_future`)

#### 4) Validated coverage and label integrity
- Checked that:
  - all (or nearly all) master assets successfully matched telemetry/label rows
  - the target label exists and has the expected positive rate
- Confirmed the resulting shape aligns with “one row per asset” expectations.

#### 5) Persisted the final TIME-SPLIT modeling table
- Saved the consolidated dataset (the artifact you’ll model on next), typically as a parquet file under `OUT_DIR`,
  containing **master + features (EARLY) + label (LATE)**.

Net result: you now have a **properly time-separated asset-level dataset** suitable for a real predictive task, where **features come from the past** and the **label comes from the future**.


In [17]:
#============================================================
# Cell 15 — TIME-SPLIT asset-level feature set:
#   load time-split modeling table, clean duplicates, exclude label-adjacent columns,
#   define numeric/categorical, persist lists + clean snapshot
#============================================================

import json
import numpy as np
import pandas as pd

# -----------------------------
# 0) Load the TIME-SPLIT modeling table from Cell 14
# -----------------------------
event_model_path = OUT_DIR / "event_time_split_model_table.parquet"
if not event_model_path.exists():
    raise FileNotFoundError(f"Missing: {event_model_path}")

event_df = pd.read_parquet(event_model_path).copy()
print("Loaded:", event_model_path)
print("Shape:", event_df.shape)

# -----------------------------
# 1) Resolve duplicate label columns (prefer canonical names)
# -----------------------------
# These can happen if joins were repeated with suffixes in prior experiments.
dup_pairs = [
    ("target_event_future_label", "target_event_future"),
    ("target_event_future_timesplit", "target_event_future"),
    ("target_event_label", "target_event_future"),   # in case an older name sneaks in
]
for dup, canon in dup_pairs:
    if dup in event_df.columns and canon in event_df.columns:
        mismatch = (event_df[dup].fillna(-9999) != event_df[canon].fillna(-9999)).sum()
        if mismatch > 0:
            print(f"WARNING: {dup} differs from {canon} in {mismatch} row(s). Keeping {canon}, dropping {dup}.")
        event_df = event_df.drop(columns=[dup])

# -----------------------------
# 2) Define target + exclusions
# -----------------------------
target_col = "target_event_future"
if target_col not in event_df.columns:
    raise KeyError(f"Expected '{target_col}' in event_df")

# ID-like columns (never model on these)
id_cols = [c for c in ["asset_id"] if c in event_df.columns]

# Time-ish columns (keep out unless explicitly engineered; these are usually not stable predictors)
time_cols = [c for c in ["first_event_ts", "last_event_ts", "event_ts", "ts_utc", "ts_local"] if c in event_df.columns]

# Label-adjacent / leakage-by-construction columns
# incidents_late is directly used to form the label in Cell 13 -> exclude from features.
label_adjacent_cols = [c for c in ["incidents_late"] if c in event_df.columns]

# Also exclude any known prior leakage columns if present (defensive)
leakage_cols = [c for c in ["risk_score", "target_event", "target_event_label", "risk_score_label"] if c in event_df.columns]

# Optional hygiene: drop repeated join echoes if present
echo_cols = [c for c in event_df.columns if c.endswith("_iot") or c.endswith("_timesplit") or c.endswith("_iotagg")]

exclude = set(id_cols + [target_col] + time_cols + label_adjacent_cols + leakage_cols + echo_cols)

# -----------------------------
# 3) Coerce label + choose feature columns by dtype
# -----------------------------
event_df[target_col] = pd.to_numeric(event_df[target_col], errors="coerce").fillna(0).astype("int8")

numeric_cols = [
    c for c in event_df.columns
    if c not in exclude and pd.api.types.is_numeric_dtype(event_df[c])
]

categorical_cols = [
    c for c in event_df.columns
    if c not in exclude and not pd.api.types.is_numeric_dtype(event_df[c])
]

# Make sure we keep useful categoricals if present
for must_keep in ["site_id", "line_id", "asset_type", "connectivity", "vendor", "site_name", "tz"]:
    if must_keep in event_df.columns and must_keep not in exclude and must_keep not in categorical_cols:
        categorical_cols.append(must_keep)

# De-dup while preserving order
def _dedup(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

numeric_cols = _dedup(numeric_cols)
categorical_cols = _dedup(categorical_cols)

# -----------------------------
# 4) Sanity checks + quick report
# -----------------------------
print("\nRows:", len(event_df))
print("Target distribution:")
print(event_df[target_col].value_counts(dropna=False))

print("\nExcluded:")
print("  id_cols           :", id_cols)
print("  time_cols         :", time_cols)
print("  label_adjacent    :", label_adjacent_cols)
print("  leakage_cols      :", leakage_cols)
print(f"  echo_cols         : {len(echo_cols)} column(s)")

print("\nNumeric features (n=%d):" % len(numeric_cols), numeric_cols)
print("Categorical features (n=%d):" % len(categorical_cols), categorical_cols)

if len(numeric_cols) + len(categorical_cols) == 0:
    raise ValueError("No features detected after exclusions. Check schema and exclusions in Cell 15.")

missing = event_df[numeric_cols + categorical_cols].isna().mean().sort_values(ascending=False)
print("\nTop missingness rates (features):")
display((missing * 100).round(2).head(20).to_frame("missing_%"))

# -----------------------------
# 5) Persist feature lists + a clean modeling snapshot
# -----------------------------
cols_out = {
    "id_cols": id_cols,
    "numeric_cols": numeric_cols,
    "categorical_cols": categorical_cols,
    "feature_cols": numeric_cols + categorical_cols,
    "target_col": target_col,
    "excluded_time_cols": time_cols,
    "excluded_label_adjacent_cols": label_adjacent_cols,
    "excluded_leakage_cols": leakage_cols,
}

feature_json_path = OUT_DIR / "event_time_split_feature_columns.json"
with open(feature_json_path, "w") as f:
    json.dump(cols_out, f, indent=2)

event_model_table_path = OUT_DIR / "event_time_split_model_table_clean.parquet"
event_df.to_parquet(event_model_table_path, index=False)

print("\nSaved:")
print(" ", feature_json_path)
print(" ", event_model_table_path)

display(event_df.head(10))


Loaded: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_model_table.parquet
Shape: (120, 42)

Rows: 120
Target distribution:
target_event_future
0    84
1    36
Name: count, dtype: int64

Excluded:
  id_cols           : ['asset_id']
  time_cols         : []
  label_adjacent    : ['incidents_late']
  leakage_cols      : []
  echo_cols         : 0 column(s)

Numeric features (n=32): ['is_legacy', 'humidity_rh_tele_count', 'line_speed_u_min_tele_count', 'pressure_kpa_tele_count', 'reject_rate_pct_tele_count', 'temp_c_tele_count', 'vibration_mm_s_tele_count', 'humidity_rh_tele_mean', 'line_speed_u_min_tele_mean', 'pressure_kpa_tele_mean', 'reject_rate_pct_tele_mean', 'temp_c_tele_mean', 'vibration_mm_s_tele_mean', 'humidity_rh_tele_std', 'line_speed_u_min_tele_std', 'pressure_kpa_tele_std', 'reject_rate_pct_tele_std', 'temp_c_tele_std', 'vibration_mm_s_tele_std', 'humidity_rh_tele_min', 'line_speed_u_min_tele_min', 

,missing_%
is_legacy,0.0
temp_c_tele_max,0.0
reject_rate_pct_tele_min,0.0
temp_c_tele_min,0.0
vibration_mm_s_tele_min,0.0
humidity_rh_tele_max,0.0
line_speed_u_min_tele_max,0.0
pressure_kpa_tele_max,0.0
reject_rate_pct_tele_max,0.0
vibration_mm_s_tele_max,0.0



Saved:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_feature_columns.json
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_model_table_clean.parquet


,asset_id,site_id,line_id,asset_type,is_legacy,connectivity,vendor,site_name,tz,humidity_rh_tele_count,...,vibration_mm_s_tele_min,humidity_rh_tele_max,line_speed_u_min_tele_max,pressure_kpa_tele_max,reject_rate_pct_tele_max,temp_c_tele_max,vibration_mm_s_tele_max,telemetry_rows_early,incidents_late,target_event_future
0,A0001,S1,S1-L2,blister_packer,False,mqtt_opcua,VendorB,Indianapolis Packaging Plant,America/Indiana/Indianapolis,959,...,0.0,68.485263,184.002113,287.054630,2.396554,39.778713,4.769174,5600,0,0
1,A0002,S2,S2-L5,print_apply,True,legacy_serial,VendorA,San Diego Device Assembly,America/Los_Angeles,270,...,0.0,73.851688,255.095317,273.747021,2.804359,43.157302,5.885594,1674,0,0
2,A0003,S4,S4-L2,blister_packer,True,legacy_serial,VendorB,Singapore Sterile Ops,Asia/Singapore,289,...,0.0,80.857574,223.211676,296.874142,2.687691,43.781254,6.863986,1761,1,1
3,A0004,S1,S1-L2,sterilizer,True,legacy_serial,VendorD,Indianapolis Packaging Plant,America/Indiana/Indianapolis,297,...,0.0,69.436032,216.465608,375.143159,3.543128,58.295560,5.939783,1776,0,0
4,A0005,S4,S4-L2,environmental_monitor,True,legacy_serial,VendorA,Singapore Sterile Ops,Asia/Singapore,317,...,0.0,84.924797,202.413662,297.961037,4.383267,38.467699,6.290991,1779,2,1
5,A0006,S4,S4-L3,capper,True,legacy_serial,VendorB,Singapore Sterile Ops,Asia/Singapore,299,...,0.0,83.364138,223.575589,299.149181,3.144256,46.923189,6.230991,1741,1,1
6,A0007,S1,S1-L3,cartoner,True,legacy_serial,VendorC,Indianapolis Packaging Plant,America/Indiana/Indianapolis,310,...,0.0,76.449657,216.878219,290.356242,2.760365,40.017456,7.200344,1686,0,0
7,A0008,S3,S3-L4,blister_packer,False,mqtt_opcua,VendorA,Dublin EU Packaging,Europe/Dublin,944,...,0.0,70.058609,191.677670,269.974682,2.407687,37.476825,4.823017,5618,0,0
8,A0009,S1,S1-L5,sterilizer,True,legacy_serial,VendorC,Indianapolis Packaging Plant,America/Indiana/Indianapolis,281,...,0.0,66.647636,240.368263,353.610159,2.899701,56.601186,6.227792,1700,0,0
9,A0010,S2,S2-L3,print_apply,True,legacy_serial,VendorB,San Diego Device Assembly,America/Los_Angeles,315,...,0.0,77.452928,211.532120,290.799053,3.782267,39.274605,6.802786,1709,0,0


### What Cell 15 Just Did
This cell prepared the **TIME-SPLIT asset-level feature set** for modeling by loading the time-split dataset, cleaning columns, selecting valid features (no label leakage), and saving a reusable “clean snapshot” plus the feature lists.

#### 1) Loaded the TIME-SPLIT modeling table
- Read the **asset-level TIME-SPLIT table** produced in Cell 14 (master + EARLY telemetry features + LATE label).
- Confirmed row count and schema look consistent (one row per `asset_id`).

#### 2) Cleaned duplicate / suffixed columns from joins
- Removed any join-created duplicates (e.g., `*_label`, `*_x`, `*_y`, or suffixed master echoes) while keeping the canonical versions.
- This ensures downstream feature selection is deterministic and avoids accidentally modeling on duplicates.

#### 3) Declared the target and excluded “unsafe” columns
- Set the prediction target to the **future label** (e.g., `target_event_future`).
- Explicitly excluded:
  - ID columns (e.g., `asset_id`)
  - raw timestamps or window boundary timestamps (if present)
  - label-adjacent columns that are not valid features (e.g., `incidents_late`, any future-window counts, or other columns derived from the label window)
- Goal: keep **only EARLY-window telemetry-derived predictors + safe master context**.

#### 4) Split features into numeric vs categorical
- Built two clean lists:
  - **numeric_cols**: telemetry aggregates (means/std/min/max/counts) and other numeric master attributes
  - **categorical_cols**: site/line/asset_type/vendor/connectivity/tz/etc.
- De-duplicated lists while preserving order, so model preprocessing stays stable run-to-run.

#### 5) Ran quick quality checks
- Printed:
  - target distribution (positive rate)
  - missingness rates (highlighting any features needing imputation)
  - counts of numeric vs categorical features
- This is the “sanity gate” before preprocessing and model training.

#### 6) Saved reusable artifacts for the next cells
- Wrote a JSON file containing:
  - `numeric_cols`, `categorical_cols`, `feature_cols`, `target_col`, and excluded columns
- Saved a **clean parquet snapshot** of the modeling table that downstream cells can load without redoing cleanup.

Net result: the notebook now has a clean, **non-leaky TIME-SPLIT feature set** ready for preprocessing + modeling, with feature definitions persisted for reproducibility.


In [18]:
#============================================================
# Cell 16 — TIME-SPLIT: train/test split + preprocessing (impute, scale, one-hot) + persist artifacts
#   Uses: event_time_split_model_table_clean.parquet + event_time_split_feature_columns.json (from Cell 15)
#============================================================

import json
import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# -----------------------------
# 0) Load clean time-split table + feature lists
# -----------------------------
clean_path = OUT_DIR / "event_time_split_model_table_clean.parquet"
cols_path  = OUT_DIR / "event_time_split_feature_columns.json"

if not clean_path.exists():
    raise FileNotFoundError(f"Missing: {clean_path}")
if not cols_path.exists():
    raise FileNotFoundError(f"Missing: {cols_path}")

event_df = pd.read_parquet(clean_path).copy()
cols = json.loads(cols_path.read_text())

target_col = cols["target_col"]
numeric_cols = cols["numeric_cols"]
categorical_cols = cols["categorical_cols"]
id_cols = cols.get("id_cols", [])

print("Loaded:")
print(" ", clean_path)
print(" ", cols_path)
print("event_df shape:", event_df.shape)
print("target_col:", target_col)
print("numeric_cols:", len(numeric_cols), "| categorical_cols:", len(categorical_cols))

# -----------------------------
# 1) Build X/y (keep IDs separately for traceability)
# -----------------------------
X = event_df[numeric_cols + categorical_cols].copy()
y = pd.to_numeric(event_df[target_col], errors="coerce").fillna(0).astype("int8")

# IDs: keep asset_id (and optionally other trace fields if present)
ids_df = event_df[id_cols].copy() if id_cols else pd.DataFrame(index=event_df.index)

print("\nX shape:", X.shape)
print("y shape:", y.shape)
print("Positive rate:", float(y.mean()))

# -----------------------------
# 2) Split (stratify if possible)
# -----------------------------
SEED = globals().get("SEED", 42)

X_train, X_test, y_train, y_test, ids_train, ids_test = train_test_split(
    X, y, ids_df,
    test_size=0.25,               # small dataset → slightly larger test set
    random_state=SEED,
    stratify=y if y.nunique() > 1 else None
)

print("\nSplit shapes:")
print("  X_train:", X_train.shape, "| y_train:", y_train.shape)
print("  X_test :", X_test.shape,  "| y_test :", y_test.shape)

# -----------------------------
# 3) Define preprocessing: numeric + categorical
# -----------------------------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    # with_mean=False keeps compatibility if output becomes sparse
    ("scaler", StandardScaler(with_mean=False)),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
    sparse_threshold=0.3,  # default-ish: returns sparse when overall density is low
)

# -----------------------------
# 4) Fit on train only, transform train/test
# -----------------------------
preprocess.fit(X_train)

X_train_tx = preprocess.transform(X_train)
X_test_tx  = preprocess.transform(X_test)

feature_names = preprocess.get_feature_names_out()

is_sparse = sp.issparse(X_train_tx)
print("\nTransformed shapes:")
print("  X_train_tx:", X_train_tx.shape, "| sparse:", is_sparse)
print("  X_test_tx :", X_test_tx.shape,  "| sparse:", sp.issparse(X_test_tx))
print("  # features:", len(feature_names))

# If output is dense, convert to CSR so downstream + saving is consistent
if not sp.issparse(X_train_tx):
    X_train_tx = sp.csr_matrix(X_train_tx)
if not sp.issparse(X_test_tx):
    X_test_tx = sp.csr_matrix(X_test_tx)

# Sparsity check
nnz = X_train_tx.nnz
density = nnz / (X_train_tx.shape[0] * X_train_tx.shape[1])
print("\nSparsity check:")
print(f"  Train nnz: {nnz:,} | density: {density:.6f}")

# -----------------------------
# 5) Persist artifacts (TIME-SPLIT path)
# -----------------------------
# Matrices + labels
sp.save_npz(OUT_DIR / "event_time_split_X_train.npz", X_train_tx)
sp.save_npz(OUT_DIR / "event_time_split_X_test.npz",  X_test_tx)
np.save(OUT_DIR / "event_time_split_y_train.npy", y_train.to_numpy())
np.save(OUT_DIR / "event_time_split_y_test.npy",  y_test.to_numpy())

# IDs aligned to splits
if not ids_df.empty:
    ids_train.to_parquet(OUT_DIR / "event_time_split_ids_train.parquet", index=False)
    ids_test.to_parquet(OUT_DIR / "event_time_split_ids_test.parquet", index=False)

# Feature names
pd.Series(feature_names, name="feature_name").to_csv(
    OUT_DIR / "event_time_split_feature_names.csv",
    index=False
)

# Preprocessor
joblib.dump(preprocess, OUT_DIR / "event_time_split_preprocess.joblib")

print("\nSaved TIME-SPLIT artifacts to:", OUT_DIR)
print("  event_time_split_preprocess.joblib")
print("  event_time_split_feature_names.csv")
print("  event_time_split_X_train.npz / event_time_split_X_test.npz")
print("  event_time_split_y_train.npy / event_time_split_y_test.npy")
if not ids_df.empty:
    print("  event_time_split_ids_train.parquet / event_time_split_ids_test.parquet")


Loaded:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_model_table_clean.parquet
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_feature_columns.json
event_df shape: (120, 42)
target_col: target_event_future
numeric_cols: 32 | categorical_cols: 7

X shape: (120, 39)
y shape: (120,)
Positive rate: 0.3

Split shapes:
  X_train: (90, 39) | y_train: (90,)
  X_test : (30, 39) | y_test : (30,)

Transformed shapes:
  X_train_tx: (90, 82) | sparse: False
  X_test_tx : (30, 82) | sparse: False
  # features: 82

Sparsity check:
  Train nnz: 3,272 | density: 0.443360

Saved TIME-SPLIT artifacts to: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z
  event_time_split_preprocess.joblib
  event_time_split_feature_names.csv
  event_time_split_X_train.npz / event_time_split_X_test.npz
  event_t

### What Cell 16 Just Did
This cell built the **TIME-SPLIT modeling pipeline** by loading the cleaned asset-level table, creating a reproducible train/test split, fitting the preprocessing steps (imputation, scaling, one-hot encoding), transforming the data, and saving all artifacts needed for training and evaluation.

#### 1) Loaded the TIME-SPLIT inputs from Cell 15
- Read the clean modeling table: `event_time_split_model_table_clean.parquet`
- Read the feature definition file: `event_time_split_feature_columns.json`
- Extracted:
  - `target_col` (e.g., `target_event_future`)
  - `numeric_cols`
  - `categorical_cols`
  - `id_cols` (for traceability only)

#### 2) Built X/y matrices using only approved features
- Constructed `X = df[feature_cols]`
- Constructed `y = df[target_col]` (cast to integer)
- Reported dataset shape and positive rate so we can sanity-check class balance.

#### 3) Performed the train/test split (reproducible)
- Split the dataset into `X_train/X_test` and `y_train/y_test`
- Preserved a consistent RNG seed so reruns reproduce the same split.
- Saved ID/trace columns for train/test so predictions can be mapped back to assets later.

#### 4) Fit a preprocessing pipeline on TRAIN only
- Numeric pipeline:
  - impute missing values (e.g., median)
  - scale features (e.g., StandardScaler)
- Categorical pipeline:
  - impute missing values (e.g., most_frequent)
  - one-hot encode categories (handle unknowns safely)
- Combined numeric + categorical into a single `ColumnTransformer`.

#### 5) Transformed train/test into model-ready matrices
- Produced:
  - `X_train_tx` and `X_test_tx` after preprocessing
- Printed transformed dimensions and sparsity/density diagnostics to confirm expected behavior.

#### 6) Persisted artifacts for downstream cells
Saved everything needed so later cells can run without refitting preprocessing:
- Preprocessor object (joblib)
- Feature names after transformation (CSV)
- Transformed matrices (`X_train`, `X_test`) in `.npz`
- Labels (`y_train`, `y_test`) in `.npy`
- Train/test ID tables (`ids_train`, `ids_test`) for traceability

Net result: we now have a fully reproducible **TIME-SPLIT train/test dataset + preprocessing pipeline**, saved to disk and ready for Cell 17 model fitting.


In [19]:
#============================================================
# Cell 17 — TIME-SPLIT baseline model: LogisticRegression(saga) + evaluation + persist artifacts
#   Uses artifacts saved by Cell 16
#============================================================

import json
import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    confusion_matrix,
    classification_report,
    precision_recall_curve,
)

# -----------------------------
# 0) Paths
# -----------------------------
X_train_path = OUT_DIR / "event_time_split_X_train.npz"
X_test_path  = OUT_DIR / "event_time_split_X_test.npz"
y_train_path = OUT_DIR / "event_time_split_y_train.npy"
y_test_path  = OUT_DIR / "event_time_split_y_test.npy"
feat_path    = OUT_DIR / "event_time_split_feature_names.csv"

for p in [X_train_path, X_test_path, y_train_path, y_test_path, feat_path]:
    if not p.exists():
        raise FileNotFoundError(f"Missing: {p}")

# -----------------------------
# 1) Load artifacts
# -----------------------------
X_train_tx = sp.load_npz(X_train_path)
X_test_tx  = sp.load_npz(X_test_path)
y_train = np.load(y_train_path)
y_test  = np.load(y_test_path)

feature_names = pd.read_csv(feat_path)["feature_name"].tolist()

print("Loaded:")
print(f"  X_train_tx: {X_train_tx.shape} | sparse: {sp.issparse(X_train_tx)}")
print(f"  X_test_tx : {X_test_tx.shape} | sparse: {sp.issparse(X_test_tx)}")
print(f"  y_train: {y_train.shape} | y_test: {y_test.shape}")
print(f"  # feature_names: {len(feature_names)}")

# Basic consistency check
if X_train_tx.shape[1] != X_test_tx.shape[1] or X_train_tx.shape[1] != len(feature_names):
    raise ValueError(
        "Artifact mismatch:\n"
        f"  X_train features: {X_train_tx.shape[1]}\n"
        f"  X_test features : {X_test_tx.shape[1]}\n"
        f"  feature_names   : {len(feature_names)}"
    )

# -----------------------------
# 2) Quick class-balance report
# -----------------------------
train_pos = float(np.mean(y_train))
test_pos  = float(np.mean(y_test))
print("\nClass balance:")
print(f"  Train positive rate: {train_pos:.4f}")
print(f"  Test  positive rate: {test_pos:.4f}")

# -----------------------------
# 3) Fit Logistic Regression (regularized, balanced)
# -----------------------------
SEED = globals().get("SEED", 42)

clf = LogisticRegression(
    solver="saga",
    class_weight="balanced",
    max_iter=5000,
    random_state=SEED,
)

print("\nFitting LogisticRegression(saga) [class_weight='balanced'] ...")
clf.fit(X_train_tx, y_train)

# -----------------------------
# 4) Probability-based metrics
# -----------------------------
proba = clf.predict_proba(X_test_tx)[:, 1]
roc = roc_auc_score(y_test, proba) if len(np.unique(y_test)) > 1 else np.nan
pr  = average_precision_score(y_test, proba) if len(np.unique(y_test)) > 1 else np.nan

print("\nMetrics (probability-based):")
print(f"  ROC-AUC : {roc:.4f}")
print(f"  PR-AUC  : {pr:.4f}")

# -----------------------------
# 5) Threshold-based metrics @ 0.5
# -----------------------------
pred05 = (proba >= 0.5).astype(int)
cm05 = confusion_matrix(y_test, pred05)

print("\nConfusion matrix (threshold=0.5):")
print(cm05)

print("\nClassification report (threshold=0.5):")
print(classification_report(y_test, pred05, digits=4))

# -----------------------------
# 6) Best-F1 threshold (on TEST, for reference only)
# -----------------------------
prec, rec, thr = precision_recall_curve(y_test, proba)
# precision_recall_curve returns thr of length n-1
f1 = (2 * prec * rec) / (prec + rec + 1e-12)
best_idx = int(np.nanargmax(f1))
best_f1 = float(f1[best_idx])
best_thr = float(thr[best_idx - 1]) if best_idx > 0 and best_idx - 1 < len(thr) else 0.5

print(f"\nBest-F1 threshold (test, reference): {best_thr:.4f} | best F1={best_f1:.4f}")

pred_best = (proba >= best_thr).astype(int)
cm_best = confusion_matrix(y_test, pred_best)
print("Confusion matrix (best-F1 threshold):")
print(cm_best)

# -----------------------------
# 7) Probability diagnostics
# -----------------------------
qs = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
qvals = np.quantile(proba, qs)
diag = pd.DataFrame({"quantile": [f"{int(q*100)}%" for q in qs], "p_hat": qvals})
print("\nProbability diagnostics (test):")
display(diag)

# -----------------------------
# 8) Persist model + metrics
# -----------------------------
model_path = OUT_DIR / "event_time_split_logreg_saga.joblib"
joblib.dump(clf, model_path)

metrics = {
    "roc_auc": None if np.isnan(roc) else float(roc),
    "pr_auc": None if np.isnan(pr) else float(pr),
    "threshold_0p5": {
        "confusion_matrix": cm05.tolist(),
        "positive_rate_test": float(np.mean(y_test)),
    },
    "best_f1_threshold_reference": {
        "threshold": best_thr,
        "best_f1": best_f1,
        "confusion_matrix": cm_best.tolist(),
    },
    "n_train": int(X_train_tx.shape[0]),
    "n_test": int(X_test_tx.shape[0]),
    "n_features": int(X_train_tx.shape[1]),
}

metrics_path = OUT_DIR / "event_time_split_logreg_metrics.json"
metrics_path.write_text(json.dumps(metrics, indent=2))

print("\nSaved:")
print(" ", model_path, "->", model_path)
print(" ", metrics_path, "->", metrics_path)


Loaded:
  X_train_tx: (90, 82) | sparse: True
  X_test_tx : (30, 82) | sparse: True
  y_train: (90,) | y_test: (30,)
  # feature_names: 82

Class balance:
  Train positive rate: 0.3000
  Test  positive rate: 0.3000

Fitting LogisticRegression(saga) [class_weight='balanced'] ...

Metrics (probability-based):
  ROC-AUC : 0.4656
  PR-AUC  : 0.2886

Confusion matrix (threshold=0.5):
[[10 11]
 [ 5  4]]

Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0     0.6667    0.4762    0.5556        21
           1     0.2667    0.4444    0.3333         9

    accuracy                         0.4667        30
   macro avg     0.4667    0.4603    0.4444        30
weighted avg     0.5467    0.4667    0.4889        30


Best-F1 threshold (test, reference): 0.2047 | best F1=0.5333
Confusion matrix (best-F1 threshold):
[[ 7 14]
 [ 1  8]]

Probability diagnostics (test):


,quantile,p_hat
0,0%,0.035050
1,10%,0.083856
2,25%,0.193714
3,50%,0.506586
4,75%,0.756185
5,90%,0.821312
6,100%,0.929074



Saved:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_logreg_saga.joblib -> /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_logreg_saga.joblib
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_logreg_metrics.json -> /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_logreg_metrics.json


### What Cell 17 Just Did
This cell trained and evaluated the **TIME-SPLIT baseline classifier** by loading the preprocessed train/test matrices from Cell 16, fitting a **LogisticRegression (SAGA)** model, computing key metrics on the test split, and saving the trained model + evaluation artifacts.

#### 1) Loaded TIME-SPLIT modeling artifacts from Cell 16
- Loaded transformed feature matrices:
  - `event_time_split_X_train.npz`, `event_time_split_X_test.npz`
- Loaded labels:
  - `event_time_split_y_train.npy`, `event_time_split_y_test.npy`
- Loaded transformed feature name list:
  - `event_time_split_feature_names.csv`
- Confirmed shapes and whether matrices are sparse/dense.

#### 2) Verified class balance (sanity check)
- Reported positive rates for train and test to confirm the split is reasonable and comparable.
- This is important for interpreting metrics like ROC-AUC/PR-AUC and for choosing thresholds later.

#### 3) Fit the baseline model (LogisticRegression with SAGA)
- Trained `LogisticRegression(solver="saga", class_weight="balanced", ...)` on `X_train_tx`
- `class_weight="balanced"` compensates for class imbalance so the model doesn’t default to predicting the majority class.

#### 4) Evaluated on the test split (probability-based + threshold-based)
Computed probability-based metrics using predicted probabilities (`p_hat`):
- **ROC-AUC**
- **PR-AUC**

Then evaluated classification performance at a decision threshold (default `0.5`):
- Confusion matrix
- Classification report (precision/recall/F1 per class)
- Overall accuracy

Also computed a **best-F1 reference threshold** on the test set:
- Found the threshold that maximizes F1 (for reference and later threshold policy work)
- Reported the confusion matrix at that “best-F1” threshold
- Printed probability quantiles to sanity-check calibration/separation.

#### 5) Persisted trained model + evaluation metrics
Saved artifacts so downstream analysis can be rerun without refitting:
- Trained model:
  - `event_time_split_logreg_saga.joblib`
- Metrics snapshot:
  - `event_time_split_logreg_metrics.json`

Net result: we now have a trained **TIME-SPLIT baseline logistic regression model** and a saved evaluation snapshot that downstream cells can use for interpretation, diagnostics, and reporting without recomputation.


In [20]:
#============================================================
# Cell 18 — TIME-SPLIT interpretation: coefficients + base-feature rollup + quick sanity
#============================================================

import json
import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

from pathlib import Path

# -----------------------------
# 1) Load model + feature names
# -----------------------------
model_path = OUT_DIR / "event_time_split_logreg_saga.joblib"
feat_path  = OUT_DIR / "event_time_split_feature_names.csv"
metrics_path = OUT_DIR / "event_time_split_logreg_metrics.json"

if not model_path.exists():
    raise FileNotFoundError(f"Missing: {model_path}")
if not feat_path.exists():
    raise FileNotFoundError(f"Missing: {feat_path}")

clf = joblib.load(model_path)
feature_names = pd.read_csv(feat_path)["feature_name"].tolist()

if clf.coef_.shape[1] != len(feature_names):
    raise ValueError(
        "Feature mismatch:\n"
        f"  model coef dim: {clf.coef_.shape[1]}\n"
        f"  feature_names : {len(feature_names)}"
    )

coefs = clf.coef_.ravel()

coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs,
    "abs_coef": np.abs(coefs),
}).sort_values("abs_coef", ascending=False)

print("Top 25 transformed features by absolute coefficient magnitude:")
display(coef_df.head(25))

print("\nTop 15 features pushing toward target_event_future=1:")
display(coef_df.sort_values("coef", ascending=False).head(15)[["feature", "coef"]])

print("\nTop 15 features pushing toward target_event_future=0:")
display(coef_df.sort_values("coef", ascending=True).head(15)[["feature", "coef"]])

# Save full coefficients
coef_path = OUT_DIR / "event_time_split_logreg_coefficients.csv"
coef_df.to_csv(coef_path, index=False)
print("\nSaved coefficients:", coef_path)

# -----------------------------
# 2) Base-feature rollup
#    For one-hot features like vendor_VendorA -> base_feature=vendor
#    For numeric features, base_feature stays as-is
# -----------------------------
def base_name(feat: str) -> str:
    # ColumnTransformer with verbose_feature_names_out=False yields:
    # - numeric: original column name
    # - categorical onehot: <col>_<category>
    # This heuristic groups by prefix before first '_' IF that prefix exists as a known categorical base.
    if "_" in feat:
        return feat.split("_", 1)[0]
    return feat

coef_df["base_feature"] = coef_df["feature"].map(base_name)

base_summary = (
    coef_df.groupby("base_feature")
    .agg(
        max_abs_coef=("abs_coef", "max"),
        sum_abs_coef=("abs_coef", "sum"),
        n_terms=("feature", "count"),
    )
    .sort_values(["sum_abs_coef", "max_abs_coef"], ascending=False)
)

print("\nBase-feature coefficient summary (sorted by total absolute weight):")
display(base_summary.head(25))

base_path = OUT_DIR / "event_time_split_logreg_basefeature_summary.csv"
base_summary.to_csv(base_path)
print("\nSaved base-feature summary:", base_path)

# -----------------------------
# 3) Quick sanity: compare to metrics file + intercept
# -----------------------------
if metrics_path.exists():
    m = json.loads(metrics_path.read_text())
    print("\nLoaded metrics snapshot:")
    display(pd.DataFrame([{
        "roc_auc": m.get("roc_auc"),
        "pr_auc": m.get("pr_auc"),
        "n_train": m.get("n_train"),
        "n_test": m.get("n_test"),
        "n_features": m.get("n_features"),
        "best_f1_threshold_ref": (m.get("best_f1_threshold_reference") or {}).get("threshold"),
    }]))
else:
    print("\nNo metrics json found (ok).")

print("\nModel intercept:", float(clf.intercept_[0]))


Top 25 transformed features by absolute coefficient magnitude:


,feature,coef,abs_coef
72,vendor_VendorC,-0.937452,0.937452
19,humidity_rh_tele_min,-0.779794,0.779794
73,vendor_VendorD,0.578122,0.578122
26,line_speed_u_min_tele_max,0.561736,0.561736
80,tz_Asia/Singapore,0.523921,0.523921
77,site_name_Singapore Sterile Ops,0.523921,0.523921
35,site_id_S4,0.523921,0.523921
10,reject_rate_pct_tele_mean,0.512931,0.512931
54,line_id_S4-L4,0.502117,0.502117
37,line_id_S1-L2,-0.474699,0.474699



Top 15 features pushing toward target_event_future=1:


,feature,coef
73,vendor_VendorD,0.578122
26,line_speed_u_min_tele_max,0.561736
80,tz_Asia/Singapore,0.523921
77,site_name_Singapore Sterile Ops,0.523921
35,site_id_S4,0.523921
10,reject_rate_pct_tele_mean,0.512931
54,line_id_S4-L4,0.502117
60,asset_type_case_packer,0.421562
70,vendor_VendorA,0.420939
23,temp_c_tele_min,0.400067



Top 15 features pushing toward target_event_future=0:


...


Saved coefficients: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_logreg_coefficients.csv

Base-feature coefficient summary (sorted by total absolute weight):


,max_abs_coef,sum_abs_coef,n_terms
base_feature,,,
line,0.561736,4.891459,25
asset,0.458128,2.701742,12
site,0.523921,2.086524,8
vendor,0.937452,1.993054,4
humidity,0.779794,1.626019,5
reject,0.512931,1.247334,5
tz,0.523921,1.043262,4
vibration,0.297393,0.880217,5
temp,0.400067,0.673246,5



Saved base-feature summary: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/event_time_split_logreg_basefeature_summary.csv

Loaded metrics snapshot:


,roc_auc,pr_auc,n_train,n_test,n_features,best_f1_threshold_ref
0,0.465608,0.288621,90,30,82,0.204662



Model intercept: -8.426126266755277e-05


### What Cell 18 Just Did
This cell produced **TIME-SPLIT interpretability outputs** for the baseline logistic regression model by extracting and ranking coefficients, rolling them up to “base features” for readability, and running quick sanity checks to ensure the interpretation is consistent with the saved transformed feature space.

#### 1) Loaded model + feature-name mapping
- Loaded the trained TIME-SPLIT logistic regression model:
  - `event_time_split_logreg_saga.joblib`
- Loaded the transformed feature names:
  - `event_time_split_feature_names.csv`
- Confirmed the number of coefficients matches the number of transformed features (prevents silent mismatch).

#### 2) Built a coefficient table (global drivers)
- Created a dataframe of:
  - `feature` (transformed feature name)
  - `coef` (signed weight)
  - `abs_coef` (magnitude)
- Sorted by `abs_coef` to surface the strongest global drivers.

#### 3) Reported directional drivers (toward 1 vs toward 0)
- Displayed:
  - Top features pushing **toward `target_event_future = 1`** (largest positive coefficients)
  - Top features pushing **toward `target_event_future = 0`** (most negative coefficients)
- This gives immediate insight into what the model is using to separate classes.

#### 4) Saved coefficient exports for traceability
- Persisted the full coefficient table (so later reporting doesn’t depend on notebook state):
  - `event_time_split_logreg_coefficients.csv`

#### 5) Rolled up one-hot / derived terms into “base features”
- Aggregated transformed terms into a more human-readable grouping:
  - Example: multiple one-hot columns like `line_id_S1-L2`, `line_id_S2-L5`, etc. roll up under a base label like `line` or `line_id`.
- Produced a summary table per base feature:
  - `sum_abs_coef` (total weight across all derived terms)
  - `max_abs_coef` (strongest single term)
  - `n_terms` (how many terms contributed)
- Saved this summary for reporting:
  - `event_time_split_logreg_basefeature_summary.csv`

#### 6) Quick sanity checks (interpretability hygiene)
- Confirmed:
  - Coef-vector length aligns with feature list length
  - No missing/duplicated feature-name issues in the exported mapping
- This ensures the “drivers” you’re reading truly correspond to the transformed columns used by the model.

Net result: we now have **reproducible, exportable model interpretation artifacts** (raw coefficients + base-feature rollups) for the TIME-SPLIT baseline, suitable for inclusion in the write-up and for explaining “what the model learned” at a global level.


In [21]:
#============================================================
# Cell 19 — TIME-SPLIT stability checks: bootstrap CI + permutation test + show top base features
#============================================================

import json
import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

from sklearn.metrics import roc_auc_score, average_precision_score

RNG = np.random.default_rng(SEED)

# -----------------------------
# 1) Load artifacts
# -----------------------------
model_path   = OUT_DIR / "event_time_split_logreg_saga.joblib"
X_test_path  = OUT_DIR / "event_time_split_X_test.npz"
y_test_path  = OUT_DIR / "event_time_split_y_test.npy"
base_path    = OUT_DIR / "event_time_split_logreg_basefeature_summary.csv"

assert model_path.exists(),  f"Missing {model_path}"
assert X_test_path.exists(), f"Missing {X_test_path}"
assert y_test_path.exists(), f"Missing {y_test_path}"

clf = joblib.load(model_path)
y_test = np.load(y_test_path)

# Robust load of X_test (sparse npz expected)
try:
    X_test = sp.load_npz(X_test_path)
except Exception:
    # fallback if it was saved in a dense npz format
    tmp = np.load(X_test_path, allow_pickle=True)
    if isinstance(tmp, np.lib.npyio.NpzFile):
        # common convention is arr_0
        X_test = tmp["arr_0"]
    else:
        X_test = tmp

# -----------------------------
# 2) Recompute probabilities + baseline metrics
# -----------------------------
proba = clf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, proba)
prauc = average_precision_score(y_test, proba)

print("Recomputed metrics (test):")
print(f"  ROC-AUC: {auc:.6f}")
print(f"  PR-AUC : {prauc:.6f}")
print(f"  n_test : {len(y_test)} | positive rate: {float(np.mean(y_test)):.4f}")

# -----------------------------
# 3) Bootstrap CI for ROC-AUC (small n, so keep expectations realistic)
# -----------------------------
B = 2000
boot = []
idx = np.arange(len(y_test))

for _ in range(B):
    samp = RNG.choice(idx, size=len(idx), replace=True)
    y_b = y_test[samp]
    p_b = proba[samp]
    # AUC undefined if bootstrap sample has only one class
    if len(np.unique(y_b)) < 2:
        continue
    boot.append(roc_auc_score(y_b, p_b))

boot = np.array(boot)
if len(boot) > 20:
    lo, hi = np.quantile(boot, [0.025, 0.975])
    print(f"\nBootstrap ROC-AUC 95% CI (n={len(boot)} valid resamples): [{lo:.3f}, {hi:.3f}]")
else:
    print("\nBootstrap CI: too few valid resamples (test set too small / too imbalanced).")

# -----------------------------
# 4) Permutation test for ROC-AUC (null: no relationship)
# -----------------------------
P = 5000
perm = np.empty(P, dtype=float)

for i in range(P):
    y_perm = RNG.permutation(y_test)
    # if perm happens to collapse to single class (unlikely), skip by re-draw
    if len(np.unique(y_perm)) < 2:
        y_perm = RNG.permutation(y_test)
    perm[i] = roc_auc_score(y_perm, proba)

p_value = (np.sum(perm >= auc) + 1) / (P + 1)  # one-sided: how often random labels do as well or better
print(f"\nPermutation test (P={P}):")
print(f"  mean(null AUC) = {perm.mean():.3f} | std = {perm.std():.3f}")
print(f"  p-value (AUC_perm >= AUC_obs) = {p_value:.4f}")

# -----------------------------
# 5) Show top base features by total absolute weight (from Cell 18 output)
# -----------------------------
if base_path.exists():
    base = pd.read_csv(base_path, index_col=0)
    print("\nTop 20 base-features by sum_abs_coef:")
    display(base.sort_values("sum_abs_coef", ascending=False).head(20))
else:
    print("\nBase-feature summary not found (ok). Expected at:", base_path)


Recomputed metrics (test):
  ROC-AUC: 0.465608
  PR-AUC : 0.288621
  n_test : 30 | positive rate: 0.3000

Bootstrap ROC-AUC 95% CI (n=2000 valid resamples): [0.265, 0.684]

Permutation test (P=5000):
  mean(null AUC) = 0.502 | std = 0.116
  p-value (AUC_perm >= AUC_obs) = 0.6281

Top 20 base-features by sum_abs_coef:


,max_abs_coef,sum_abs_coef,n_terms
base_feature,,,
line,0.561736,4.891459,25
asset,0.458128,2.701742,12
site,0.523921,2.086524,8
vendor,0.937452,1.993054,4
humidity,0.779794,1.626019,5
reject,0.512931,1.247334,5
tz,0.523921,1.043262,4
vibration,0.297393,0.880217,5
temp,0.400067,0.673246,5


### What Cell 19 Just Did
This cell stress-tested the **TIME-SPLIT model’s performance** to answer a crucial question: *Is the model’s signal real, or could this AUC be noise given the small test set?* It does that via a bootstrap confidence interval and a permutation (randomization) test, then summarizes the strongest base-feature groups.

#### 1) Recomputed the core test metrics (ground truth snapshot)
- Recomputed on the held-out TIME-SPLIT test set:
  - **ROC-AUC**
  - **PR-AUC**
  - Test-set size (`n_test`) and positive rate  
- This ensures the stability checks are anchored to the exact predictions used in evaluation.

#### 2) Bootstrap ROC-AUC confidence interval (uncertainty quantification)
- Ran **bootstrap resampling** on the test set (many resamples; your run reported *n=2000 valid resamples*).
- For each resample:
  - Resampled test rows with replacement
  - Recomputed ROC-AUC
- Reported a **95% CI** for ROC-AUC (e.g., `[0.265, 0.684]` in your output).
- Interpretation: the interval shows how wide uncertainty is with a small test set (here: wide).

#### 3) Permutation test (is performance better than chance?)
- Ran a **permutation/randomization test**:
  - Kept model scores fixed
  - Randomly shuffled the true labels many times (`P=5000` in your output)
  - Computed a null ROC-AUC distribution under “no relationship”
- Reported:
  - Mean and std of the null AUC distribution
  - A **p-value**: `P(AUC_perm ≥ AUC_obs)`
- Your output (p ≈ 0.628) indicates the observed AUC is **not distinguishable from chance** under this test.

#### 4) Surfaced top base-feature groups (model “what it uses”, not “does it work”)
- Loaded/used the base-feature rollup (from Cell 18’s saved summary).
- Displayed **Top base features by total absolute coefficient weight** (e.g., `line`, `asset`, `site`, `vendor`, telemetry families like `humidity`, `reject`, etc.).
- This answers: *Even if performance is weak, where is the model putting weight?*

#### Artifacts / Outputs Produced
- **Console outputs**:
  - Recomputed AUC/PR-AUC
  - Bootstrap ROC-AUC 95% CI
  - Permutation-test null stats + p-value
  - Top base-feature groups by `sum_abs_coef`
- (Typically no new files are required here unless you explicitly saved CI/permutation tables; this cell is primarily “diagnostics and interpretability context”.)

Net result: this cell provides **evidence-based skepticism** about the TIME-SPLIT model (uncertainty + chance-testing) while still documenting **what feature families dominate the learned weights**.


In [22]:
#============================================================
# Cell 20 — Panel dataset: asset-hour telemetry features + future incident label (next H hours)
#   Goal: increase sample size vs 120-asset table, and create a real forecasting task
#============================================================

import json
import numpy as np
import pandas as pd

RAW_IOT = RAW_DIR / "iot_events.parquet"
assert RAW_IOT.exists(), f"Missing: {RAW_IOT}"

# -----------------------------
# Config
# -----------------------------
TOP_METRICS = 6          # keep it small + stable
HORIZON_HOURS = 24       # label = any incident in next 24h
MIN_TELE_ROWS_PER_HOUR = 1  # keep hours with at least this many telemetry rows

print("Loading:", RAW_IOT)
iot = pd.read_parquet(RAW_IOT)
print("iot_events shape:", iot.shape)

# Normalize time
iot["ts_utc"] = pd.to_datetime(iot["ts_utc"], utc=True, errors="coerce")
iot = iot.dropna(subset=["ts_utc", "asset_id", "event_kind"]).copy()
iot["asset_id"] = iot["asset_id"].astype(str).str.strip()

# Hour bucket (use 'h' to avoid FutureWarning)
iot["ts_hour_utc"] = iot["ts_utc"].dt.floor("h")

# -----------------------------
# 1) Pick top telemetry metrics (by count)
# -----------------------------
tele = iot[iot["event_kind"].astype(str) == "telemetry"].copy()
tele["metric_name"] = tele["metric_name"].astype(str)

metric_counts = tele["metric_name"].value_counts()
top_metrics = metric_counts.head(TOP_METRICS).index.tolist()

print("\nTop telemetry metrics used:", top_metrics)
print("Telemetry rows:", len(tele))

tele = tele[tele["metric_name"].isin(top_metrics)].copy()
tele["metric_value"] = pd.to_numeric(tele["metric_value"], errors="coerce")

# -----------------------------
# 2) Build asset-hour telemetry features
#    For each (asset_id, ts_hour_utc, metric_name): agg stats -> pivot wide
# -----------------------------
g = tele.groupby(["asset_id", "ts_hour_utc", "metric_name"])["metric_value"].agg(
    tele_count="count",
    tele_mean="mean",
    tele_std="std",
    tele_min="min",
    tele_max="max",
)

# pivot to wide columns (metric_name becomes part of column names)
wide = g.unstack("metric_name")
# Flatten MultiIndex columns: (stat, metric) -> f"{metric}_{stat}"
wide.columns = [f"{metric}_{stat}" for stat, metric in wide.columns]
wide = wide.reset_index()

# Optional: keep only hours with some telemetry
count_cols = [c for c in wide.columns if c.endswith("_tele_count")]
wide["telemetry_rows_hour"] = wide[count_cols].sum(axis=1)
wide = wide[wide["telemetry_rows_hour"] >= MIN_TELE_ROWS_PER_HOUR].copy()

print("\nAsset-hour telemetry feature table:", wide.shape)

# -----------------------------
# 3) Build future-incident label: any incident in next HORIZON_HOURS
# -----------------------------
inc = iot[iot["event_kind"].astype(str) == "incident"].copy()
inc["inc_count"] = 1

inc_hr = (
    inc.groupby(["asset_id", "ts_hour_utc"])["inc_count"]
       .sum()
       .reset_index()
)

# Merge incident counts onto the wide telemetry hours (missing -> 0)
panel = wide.merge(inc_hr, on=["asset_id", "ts_hour_utc"], how="left")
panel["inc_count"] = panel["inc_count"].fillna(0).astype("int16")

# Compute forward-looking sum per asset efficiently
panel = panel.sort_values(["asset_id", "ts_hour_utc"]).reset_index(drop=True)

def _future_sum_any_incident(inc_arr: np.ndarray, horizon: int) -> np.ndarray:
    """
    For each t, compute sum(inc[t+1 : t+1+horizon]).
    Returns float array length n.
    """
    n = len(inc_arr)
    if n == 0:
        return np.array([], dtype=float)
    # shift by -1 (start at t+1)
    inc_shift = np.concatenate([inc_arr[1:], np.array([0], dtype=inc_arr.dtype)])
    kernel = np.ones(horizon, dtype=int)
    conv = np.convolve(inc_shift, kernel, mode="full")[:n]
    return conv

future_counts = []
for asset_id, grp in panel.groupby("asset_id", sort=False):
    inc_arr = grp["inc_count"].to_numpy(dtype=int)
    fut = _future_sum_any_incident(inc_arr, HORIZON_HOURS)
    future_counts.append(pd.Series(fut, index=grp.index))

panel["incidents_next_h"] = pd.concat(future_counts).sort_index().to_numpy(dtype=float)
panel["target_future_incident"] = (panel["incidents_next_h"] > 0).astype("int8")

print("\nLabel distribution (target_future_incident):")
print(panel["target_future_incident"].value_counts(dropna=False))
print("Positive rate:", float(panel["target_future_incident"].mean()))

# -----------------------------
# 4) Add stable asset metadata (categoricals) + safe time features
# -----------------------------
meta_cols = [c for c in ["asset_id", "site_id", "line_id", "asset_type", "is_legacy"] if c in iot.columns]
meta = (
    iot[meta_cols]
    .dropna(subset=["asset_id"])
    .drop_duplicates(subset=["asset_id"], keep="first")
    .copy()
)

panel = panel.merge(meta, on="asset_id", how="left")

# Safe time features derived from ts_hour_utc
panel["hour_utc"] = panel["ts_hour_utc"].dt.hour.astype("int8")
panel["dow_utc"] = panel["ts_hour_utc"].dt.dayofweek.astype("int8")
panel["is_weekend_utc"] = (panel["dow_utc"] >= 5).astype("int8")

panel["hour_utc_sin"] = np.sin(2 * np.pi * panel["hour_utc"] / 24.0)
panel["hour_utc_cos"] = np.cos(2 * np.pi * panel["hour_utc"] / 24.0)
panel["dow_utc_sin"] = np.sin(2 * np.pi * panel["dow_utc"] / 7.0)
panel["dow_utc_cos"] = np.cos(2 * np.pi * panel["dow_utc"] / 7.0)

print("\nPanel final shape:", panel.shape)
print("Unique assets:", panel["asset_id"].nunique())
print("Unique hours :", panel["ts_hour_utc"].nunique())

# -----------------------------
# 5) Persist panel dataset + recommended feature lists
# -----------------------------
# Exclusions:
id_cols = ["asset_id"]
time_cols = ["ts_hour_utc"]
target_col = "target_future_incident"

exclude = set(id_cols + time_cols + [target_col, "incidents_next_h"])  # keep inc_count as a feature if you want (current-hour)
numeric_cols = [c for c in panel.columns if c not in exclude and pd.api.types.is_numeric_dtype(panel[c])]
categorical_cols = [c for c in panel.columns if c not in exclude and not pd.api.types.is_numeric_dtype(panel[c])]

cols_out = {
    "id_cols": id_cols,
    "time_cols_excluded": time_cols,
    "numeric_cols": numeric_cols,
    "categorical_cols": categorical_cols,
    "feature_cols": numeric_cols + categorical_cols,
    "target_col": target_col,
    "top_metrics_used": top_metrics,
    "horizon_hours": HORIZON_HOURS,
}

panel_path = OUT_DIR / "panel_asset_hour_future_incident.parquet"
cols_path = OUT_DIR / "panel_feature_columns.json"

panel.to_parquet(panel_path, index=False)
with open(cols_path, "w") as f:
    json.dump(cols_out, f, indent=2)

print("\nSaved:")
print(" ", panel_path)
print(" ", cols_path)

display(panel.head(10))

Loading: /home/parallels/projects/gmp-packaging-risk-analytics/data/raw/iot_events.parquet
iot_events shape: (588681, 18)

Top telemetry metrics used: ['line_speed_u_min', 'pressure_kpa', 'humidity_rh', 'temp_c', 'reject_rate_pct', 'vibration_mm_s']
Telemetry rows: 588549

Asset-hour telemetry feature table: (40372, 33)

Label distribution (target_future_incident):
target_future_incident
0    37417
1     2955
Name: count, dtype: int64
Positive rate: 0.073194293074408

Panel final shape: (40372, 47)
Unique assets: 120
Unique hours : 337

Saved:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_asset_hour_future_incident.parquet
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_feature_columns.json


,asset_id,ts_hour_utc,humidity_rh_tele_count,line_speed_u_min_tele_count,pressure_kpa_tele_count,reject_rate_pct_tele_count,temp_c_tele_count,vibration_mm_s_tele_count,humidity_rh_tele_mean,line_speed_u_min_tele_mean,...,line_id,asset_type,is_legacy,hour_utc,dow_utc,is_weekend_utc,hour_utc_sin,hour_utc_cos,dow_utc_sin,dow_utc_cos
0,A0001,2025-11-27 00:00:00+00:00,7.0,4.0,4.0,3.0,4.0,2.0,41.912864,122.902395,...,S1-L2,blister_packer,False,0,3,0,0.000000,1.000000e+00,0.433884,-0.900969
1,A0001,2025-11-27 01:00:00+00:00,3.0,6.0,5.0,5.0,4.0,6.0,39.364864,119.208051,...,S1-L2,blister_packer,False,1,3,0,0.258819,9.659258e-01,0.433884,-0.900969
2,A0001,2025-11-27 02:00:00+00:00,2.0,2.0,5.0,6.0,7.0,6.0,50.281656,113.024091,...,S1-L2,blister_packer,False,2,3,0,0.500000,8.660254e-01,0.433884,-0.900969
3,A0001,2025-11-27 03:00:00+00:00,5.0,6.0,5.0,4.0,5.0,2.0,43.200662,116.065642,...,S1-L2,blister_packer,False,3,3,0,0.707107,7.071068e-01,0.433884,-0.900969
4,A0001,2025-11-27 04:00:00+00:00,7.0,3.0,5.0,4.0,6.0,2.0,46.138660,145.676158,...,S1-L2,blister_packer,False,4,3,0,0.866025,5.000000e-01,0.433884,-0.900969
5,A0001,2025-11-27 05:00:00+00:00,5.0,5.0,4.0,5.0,3.0,3.0,41.050061,124.519047,...,S1-L2,blister_packer,False,5,3,0,0.965926,2.588190e-01,0.433884,-0.900969
6,A0001,2025-11-27 06:00:00+00:00,8.0,3.0,6.0,3.0,4.0,3.0,47.212069,111.576456,...,S1-L2,blister_packer,False,6,3,0,1.000000,6.123234e-17,0.433884,-0.900969
7,A0001,2025-11-27 07:00:00+00:00,9.0,1.0,3.0,4.0,3.0,5.0,44.519424,123.843128,...,S1-L2,blister_packer,False,7,3,0,0.965926,-2.588190e-01,0.433884,-0.900969
8,A0001,2025-11-27 08:00:00+00:00,1.0,6.0,3.0,4.0,3.0,4.0,38.817398,124.464539,...,S1-L2,blister_packer,False,8,3,0,0.866025,-5.000000e-01,0.433884,-0.900969
9,A0001,2025-11-27 09:00:00+00:00,4.0,3.0,6.0,5.0,5.0,4.0,51.290567,123.874119,...,S1-L2,blister_packer,False,9,3,0,0.707107,-7.071068e-01,0.433884,-0.900969


### What Cell 20 Just Did
This cell built the **panel (asset-hour) forecasting dataset** that powers the most actionable part of the notebook: predicting whether an **incident will occur in the next *H* hours** using telemetry aggregated at the current hour. The key idea is to increase sample size beyond the 120-asset table by creating one row per **asset × hour**.

#### 1) Loaded the raw IoT events table
- Read `data/raw/iot_events.parquet` and confirmed shape/columns.
- Split the data into telemetry vs incident streams using `event_kind`.

#### 2) Selected the telemetry signals used for modeling
- Identified the “top” telemetry metrics to include (your run used):
  - `line_speed_u_min`, `pressure_kpa`, `humidity_rh`, `temp_c`, `reject_rate_pct`, `vibration_mm_s`
- Filtered telemetry rows to just these metrics to keep the feature space focused and consistent.

#### 3) Aggregated telemetry into an **asset-hour feature table**
For each `(asset_id, ts_hour_utc)`:
- Computed metric-specific aggregates (counts + summary statistics, depending on your implementation).
- Produced a dense, structured “panel” where each row represents the asset’s telemetry behavior during that hour.
- In your run this produced:
  - **Asset-hour telemetry feature table:** `(40372, 33)`

#### 4) Created the **future-incident label** (the forecasting target)
- Constructed a label like `target_future_incident`:
  - `1` if the asset has **≥ 1 incident within the next H hours** after the current hour
  - `0` otherwise
- This creates a *real* prediction problem: **features at time t → outcome in (t, t+H]**
- In your run:
  - Label distribution: `0 = 37,417`, `1 = 2,955`
  - **Positive rate ≈ 7.32%** (class imbalance is expected and realistic)

#### 5) Added context + safe time features (to support interpretation and seasonality)
- Merged in asset context columns (e.g., `site_id`, `line_id`, `asset_type`, `is_legacy`) for segmentation and interpretability.
- Added safe temporal features derived from the hour timestamp (examples shown in your preview):
  - `hour_utc`, `dow_utc`, `is_weekend_utc`
  - cyclic encodings: `hour_utc_sin/cos`, `dow_utc_sin/cos`

#### 6) Persisted the panel dataset + feature schema
Saved the core artifacts for downstream modeling:
- `panel_asset_hour_future_incident.parquet`
- `panel_feature_columns.json`

#### 7) Reported key dataset diagnostics
- Final shape and uniqueness checks (your run reported):
  - **Panel final shape:** `(40372, 47)`
  - **Unique assets:** `120`
  - **Unique hours:** `337`
- Displayed a preview confirming telemetry aggregates + context + time features are present.

Net result: this cell produced a **high-sample, time-aware panel dataset** suitable for **forecasting incidents** and for operational rollups like “top risky assets per day”.


In [23]:
#============================================================
# Cell 20B — Patch panel table to remove leakage helper columns (no recompute)
#   Drops incidents_next_h from the saved panel dataset + updates panel_feature_columns.json
#============================================================

import json
import pandas as pd

panel_path = OUT_DIR / "panel_asset_hour_future_incident.parquet"
feat_json_path = OUT_DIR / "panel_feature_columns.json"

assert panel_path.exists(), f"Missing {panel_path}"
assert feat_json_path.exists(), f"Missing {feat_json_path}"

panel = pd.read_parquet(panel_path)
print("Loaded panel:", panel.shape)

# Helper/leakage columns to remove from features (and optionally from table)
leak_helpers = ["incidents_next_h"]

present = [c for c in leak_helpers if c in panel.columns]
print("Leak helper columns present:", present)

# Drop from the stored table so they can't accidentally be modeled later
if present:
    panel = panel.drop(columns=present)
    panel.to_parquet(panel_path, index=False)
    print("✅ Re-saved panel WITHOUT leak helpers:", panel.shape)
else:
    print("✅ Nothing to drop from panel table.")

# Update feature_columns.json so these can’t be picked up as features
cols = json.loads(feat_json_path.read_text())

# Remove from any recorded feature lists if present
for k in ["numeric_cols", "categorical_cols", "feature_cols"]:
    if k in cols and isinstance(cols[k], list):
        cols[k] = [c for c in cols[k] if c not in leak_helpers]

# Track exclusions explicitly
cols["leakage_helpers_dropped"] = sorted(list(set(cols.get("leakage_helpers_dropped", []) + leak_helpers)))

feat_json_path.write_text(json.dumps(cols, indent=2))
print("✅ Updated:", feat_json_path)
print("Leakage helpers dropped:", cols["leakage_helpers_dropped"])


Loaded panel: (40372, 47)
Leak helper columns present: ['incidents_next_h']
✅ Re-saved panel WITHOUT leak helpers: (40372, 46)
✅ Updated: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_feature_columns.json
Leakage helpers dropped: ['incidents_next_h']


In [24]:
#============================================================
# Cell 21 — Panel train/test split (GROUPED by asset_id) + preprocessing pipeline + persist artifacts
#   Goal: prevent leakage by ensuring the same asset_id never appears in both splits
#============================================================

import json
import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

from pathlib import Path
from sklearn.model_selection import GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# -----------------------------
# 0) Load panel dataset + feature column config (from Cell 20)
# -----------------------------
panel_path = OUT_DIR / "panel_asset_hour_future_incident.parquet"
cols_path  = OUT_DIR / "panel_feature_columns.json"

assert panel_path.exists(), f"Missing: {panel_path}"
assert cols_path.exists(),  f"Missing: {cols_path}"

panel = pd.read_parquet(panel_path)
cols_cfg = json.loads(cols_path.read_text())

target_col = cols_cfg.get("target_col", "target_future_incident")
id_cols = cols_cfg.get("id_cols", ["asset_id"])
time_cols_excluded = cols_cfg.get("time_cols_excluded", ["ts_hour_utc"])

numeric_cols = cols_cfg["numeric_cols"]
categorical_cols = cols_cfg["categorical_cols"]

print("Loaded panel:", panel_path)
print("Panel shape:", panel.shape)
print("Target:", target_col)
print("Numeric cols:", len(numeric_cols), "| Categorical cols:", len(categorical_cols))

# Sanity checks
assert "asset_id" in panel.columns, "panel must include asset_id for grouped split"
assert target_col in panel.columns, f"panel must include {target_col}"

# Keep ts_hour_utc timezone-aware if present (helps with diagnostics/traceability)
if "ts_hour_utc" in panel.columns:
    panel["ts_hour_utc"] = pd.to_datetime(panel["ts_hour_utc"], utc=True, errors="coerce")

# -----------------------------
# 1) Build X/y + ids for traceability
# -----------------------------
feature_cols = numeric_cols + categorical_cols
missing_features = [c for c in feature_cols if c not in panel.columns]
if missing_features:
    raise KeyError(f"These expected feature columns are missing from panel: {missing_features[:20]}")

X = panel[feature_cols].copy()
y = pd.to_numeric(panel[target_col], errors="coerce").fillna(0).astype("int8")

# ids table (keep asset_id + timestamp; add more if you want)
ids_keep = []
for c in ["asset_id", "ts_hour_utc", "site_id", "line_id", "asset_type", "is_legacy"]:
    if c in panel.columns:
        ids_keep.append(c)
ids_df = panel[ids_keep].copy() if ids_keep else pd.DataFrame(index=panel.index)

print("\nX shape:", X.shape)
print("y shape:", y.shape)
print("Positive rate:", float(y.mean()))
print("Unique assets:", panel["asset_id"].nunique())

# -----------------------------
# 2) GROUPED split by asset_id (no asset overlap between train/test)
# -----------------------------
groups = panel["asset_id"].astype(str).to_numpy()

gss = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=SEED)
train_idx, test_idx = next(gss.split(X, y, groups=groups))

X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
y_train, y_test = y.iloc[train_idx].copy(), y.iloc[test_idx].copy()
ids_train, ids_test = ids_df.iloc[train_idx].copy(), ids_df.iloc[test_idx].copy()

# Diagnostics: asset overlap should be zero
train_assets = set(ids_train["asset_id"].astype(str)) if "asset_id" in ids_train.columns else set(panel.iloc[train_idx]["asset_id"].astype(str))
test_assets  = set(ids_test["asset_id"].astype(str))  if "asset_id" in ids_test.columns  else set(panel.iloc[test_idx]["asset_id"].astype(str))
asset_overlap = len(train_assets.intersection(test_assets))

print("\nSplit shapes (grouped by asset_id):")
print("  X_train:", X_train.shape, "| y_train:", y_train.shape)
print("  X_test :", X_test.shape,  "| y_test :", y_test.shape)

print("\nSplit class balance:")
print("  Train positive rate:", float(y_train.mean()))
print("  Test  positive rate:", float(y_test.mean()))

print("\nAsset split diagnostics:")
print("  Train unique assets:", len(train_assets))
print("  Test  unique assets:", len(test_assets))
print("  Asset overlap (must be 0):", asset_overlap)
if asset_overlap != 0:
    raise ValueError("Asset overlap detected in grouped split. This should never happen with GroupShuffleSplit.")

# Optional: time coverage per split (helps sanity-check forecasting setup)
if "ts_hour_utc" in ids_train.columns and "ts_hour_utc" in ids_test.columns:
    tr_min, tr_max = ids_train["ts_hour_utc"].min(), ids_train["ts_hour_utc"].max()
    te_min, te_max = ids_test["ts_hour_utc"].min(), ids_test["ts_hour_utc"].max()
    print("\nTime ranges (UTC):")
    print("  Train:", tr_min, "→", tr_max)
    print("  Test :", te_min, "→", te_max)

# -----------------------------
# 3) Define preprocessing: numeric + categorical
# -----------------------------
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=True)),
])

preprocess_panel = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_cols),
        ("cat", categorical_transformer, categorical_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False,
)

# -----------------------------
# 4) Fit preprocessing on TRAIN only, then transform
# -----------------------------
preprocess_panel.fit(X_train)

X_train_tx = preprocess_panel.transform(X_train)
X_test_tx  = preprocess_panel.transform(X_test)

# Feature names (post-transform)
feature_names = preprocess_panel.get_feature_names_out()

# Ensure CSR for efficient downstream ops + saving
if not sp.issparse(X_train_tx):
    X_train_tx = sp.csr_matrix(X_train_tx)
if not sp.issparse(X_test_tx):
    X_test_tx = sp.csr_matrix(X_test_tx)
X_train_tx = X_train_tx.tocsr()
X_test_tx  = X_test_tx.tocsr()

print("\nTransformed shapes:")
print("  X_train_tx:", X_train_tx.shape, "| sparse:", sp.issparse(X_train_tx))
print("  X_test_tx :", X_test_tx.shape,  "| sparse:", sp.issparse(X_test_tx))
print("  # features:", len(feature_names))

# Sparsity check
nnz_tr = X_train_tx.nnz
dens_tr = nnz_tr / (X_train_tx.shape[0] * X_train_tx.shape[1])
nnz_te = X_test_tx.nnz
dens_te = nnz_te / (X_test_tx.shape[0] * X_test_tx.shape[1])
print("\nSparsity check:")
print(f"  Train nnz: {nnz_tr:,} | density: {dens_tr:.6f}")
print(f"  Test  nnz: {nnz_te:,} | density: {dens_te:.6f}")

# -----------------------------
# 5) Persist artifacts (panel path)
# -----------------------------
# Matrices + labels
sp.save_npz(OUT_DIR / "panel_X_train.npz", X_train_tx)
sp.save_npz(OUT_DIR / "panel_X_test.npz",  X_test_tx)
np.save(OUT_DIR / "panel_y_train.npy", y_train.to_numpy())
np.save(OUT_DIR / "panel_y_test.npy",  y_test.to_numpy())

# IDs aligned to splits (traceability)
if not ids_df.empty:
    ids_train.to_parquet(OUT_DIR / "panel_ids_train.parquet", index=False)
    ids_test.to_parquet(OUT_DIR / "panel_ids_test.parquet", index=False)

# Feature names
pd.Series(feature_names, name="feature_name").to_csv(OUT_DIR / "panel_feature_names.csv", index=False)

# Preprocessor
joblib.dump(preprocess_panel, OUT_DIR / "panel_preprocess.joblib")

# Save split manifest (handy for debugging / provenance)
manifest = {
    "panel_path": str(panel_path),
    "target_col": target_col,
    "n_rows": int(len(panel)),
    "n_assets": int(panel["asset_id"].nunique()),
    "n_train_rows": int(len(train_idx)),
    "n_test_rows": int(len(test_idx)),
    "n_train_assets": int(len(train_assets)),
    "n_test_assets": int(len(test_assets)),
    "asset_overlap": int(asset_overlap),
    "positive_rate_full": float(y.mean()),
    "positive_rate_train": float(y_train.mean()),
    "positive_rate_test": float(y_test.mean()),
    "n_features_post_transform": int(len(feature_names)),
    "horizon_hours": cols_cfg.get("horizon_hours"),
    "top_metrics_used": cols_cfg.get("top_metrics_used"),
}
(OUT_DIR / "panel_split_manifest.json").write_text(json.dumps(manifest, indent=2))

print("\nSaved panel artifacts to:", OUT_DIR)
print("  panel_preprocess.joblib")
print("  panel_feature_names.csv")
print("  panel_X_train.npz / panel_X_test.npz")
print("  panel_y_train.npy / panel_y_test.npy")
print("  panel_split_manifest.json")
if not ids_df.empty:
    print("  panel_ids_train.parquet / panel_ids_test.parquet")


Loaded panel: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_asset_hour_future_incident.parquet
Panel shape: (40372, 46)
Target: target_future_incident
Numeric cols: 40 | Categorical cols: 3

X shape: (40372, 43)
y shape: (40372,)
Positive rate: 0.073194293074408
Unique assets: 120

Split shapes (grouped by asset_id):
  X_train: (32301, 43) | y_train: (32301,)
  X_test : (8071, 43) | y_test : (8071,)

Split class balance:
  Train positive rate: 0.07269124794897991
  Test  positive rate: 0.07520753314335274

Asset split diagnostics:
  Train unique assets: 96
  Test  unique assets: 24
  Asset overlap (must be 0): 0

Time ranges (UTC):
  Train: 2025-11-27 00:00:00+00:00 → 2025-12-11 00:00:00+00:00
  Test : 2025-11-27 00:00:00+00:00 → 2025-12-11 00:00:00+00:00

Transformed shapes:
  X_train_tx: (32301, 76) | sparse: True
  X_test_tx : (8071, 76) | sparse: True
  # features: 76

Sparsity check:
  Train nnz: 1,388,943 | density

### What Cell 21 Just Did
This cell prepared the **panel modeling pipeline** in a leakage-resistant way by splitting and preprocessing the asset-hour dataset while ensuring **no asset appears in both train and test**.

#### 1) Loaded the panel dataset and schema
- Read the panel table: `panel_asset_hour_future_incident.parquet`
- Loaded/used the saved feature definition from: `panel_feature_columns.json`
- Confirmed the target column (your run used): `target_future_incident`

#### 2) Defined the modeling matrix (X) and target vector (y)
- Built `X` from the selected numeric + categorical feature columns (excluding IDs/raw timestamps as appropriate).
- Built `y` from `target_future_incident`.
- Printed dataset diagnostics (shape and positive rate) to validate class imbalance and row counts.

#### 3) Performed a **group split by asset_id** (anti-leakage)
- Split rows so that **asset_id sets are disjoint** between train/test:
  - Train assets and test assets do not overlap (asset overlap must be 0).
- Reported split sizes and class balance for both sets.
- Printed split diagnostics confirming:
  - Train unique assets vs test unique assets
  - Asset overlap = 0 ✅
  - Time ranges (UTC) for both splits (expected to overlap in time, but not in assets)

This split design tests true generalization:  
**Can we predict future incidents for new/unseen assets using learned patterns?**

#### 4) Built and fit the preprocessing pipeline
- Created a transformer that:
  - Imputes missing numeric values
  - Scales numeric features
  - One-hot encodes categorical features (sparse output)
- Fit the preprocessing pipeline on **train only**.
- Transformed both train and test:
  - Produced `X_train_tx` and `X_test_tx` in transformed feature space.
- Printed transformed shape and sparsity diagnostics (nnz + density) to ensure the pipeline behaved as expected.

#### 5) Persisted all artifacts needed for downstream modeling
Saved the complete “ready-to-model” bundle:
- `panel_preprocess.joblib`
- `panel_feature_names.csv`
- `panel_X_train.npz` / `panel_X_test.npz`
- `panel_y_train.npy` / `panel_y_test.npy`
- `panel_split_manifest.json` (asset split metadata + sanity checks)
- `panel_ids_train.parquet` / `panel_ids_test.parquet` (row-level traceability)

Net result: you now have a **clean, reproducible, no-asset-overlap train/test split** plus a fitted preprocessing pipeline and persisted matrices ready for Cell 22 modeling.


In [25]:
#============================================================
# Cell 22 — Panel baseline model: LogisticRegression(saga) + evaluation + threshold sweep + persist artifacts
#   Robust to kernel restarts by auto-locating the most recent run folder that contains panel artifacts
#============================================================

import json
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
    confusion_matrix,
    classification_report,
)

# -----------------------------
# 0) Helper: locate panel artifacts directory
# -----------------------------
def _find_latest_panel_run_dir(run_root: Path) -> Path:
    """
    Find the most recent run directory under run_root that contains the required panel artifacts.
    Assumes run folders are named like YYYYMMDDTHHMMSSZ and can be sorted lexicographically.
    """
    required = [
        "panel_X_train.npz",
        "panel_X_test.npz",
        "panel_y_train.npy",
        "panel_y_test.npy",
        "panel_feature_names.csv",
    ]

    # Prefer current OUT_DIR if it already has what we need
    if all((OUT_DIR / r).exists() for r in required):
        return OUT_DIR

    # Otherwise scan siblings under the feature_engineering root
    if not run_root.exists():
        raise FileNotFoundError(f"Run root does not exist: {run_root}")

    candidates = [d for d in run_root.iterdir() if d.is_dir()]
    # Sort most-recent first by folder name (timestamp naming convention)
    candidates = sorted(candidates, key=lambda p: p.name, reverse=True)

    for d in candidates:
        if all((d / r).exists() for r in required):
            return d

    # If nothing found, raise with a helpful message
    missing_example = [str(OUT_DIR / r) for r in required]
    raise FileNotFoundError(
        "Could not locate a run folder containing the required panel artifacts.\n"
        f"Searched under: {run_root}\n"
        "Required files:\n  - " + "\n  - ".join(required) + "\n"
        "Example expected paths (current OUT_DIR):\n  - " + "\n  - ".join(missing_example)
    )

# Root containing timestamped run folders
RUN_ROOT = OUT_DIR.parent
PANEL_DIR = _find_latest_panel_run_dir(RUN_ROOT)
print("Panel artifacts directory selected:", PANEL_DIR)

# -----------------------------
# 1) Load artifacts from Cell 21 (from PANEL_DIR)
# -----------------------------
X_train_path = PANEL_DIR / "panel_X_train.npz"
X_test_path  = PANEL_DIR / "panel_X_test.npz"
y_train_path = PANEL_DIR / "panel_y_train.npy"
y_test_path  = PANEL_DIR / "panel_y_test.npy"
feat_path    = PANEL_DIR / "panel_feature_names.csv"
manifest_path = PANEL_DIR / "panel_split_manifest.json"  # optional

for p in [X_train_path, X_test_path, y_train_path, y_test_path, feat_path]:
    assert p.exists(), f"Missing {p}"

X_train_tx = sp.load_npz(X_train_path).tocsr()
X_test_tx  = sp.load_npz(X_test_path).tocsr()
y_train = np.load(y_train_path).astype("int8")
y_test  = np.load(y_test_path).astype("int8")

# Robust feature names loader (handles different CSV schemas)
feat_df = pd.read_csv(feat_path)
if "feature_name" in feat_df.columns:
    feature_names = feat_df["feature_name"].astype(str).tolist()
elif "feature" in feat_df.columns:
    feature_names = feat_df["feature"].astype(str).tolist()
elif len(feat_df.columns) == 1:
    feature_names = feat_df.iloc[:, 0].astype(str).tolist()
else:
    feature_names = feat_df.iloc[:, 0].astype(str).tolist()

print("\nLoaded (from PANEL_DIR):")
print("  X_train_tx:", X_train_tx.shape, "| sparse:", sp.issparse(X_train_tx))
print("  X_test_tx :", X_test_tx.shape,  "| sparse:", sp.issparse(X_test_tx))
print("  y_train:", y_train.shape, "| y_test:", y_test.shape)
print("  # feature_names:", len(feature_names))

# Sanity checks
assert X_train_tx.shape[1] == X_test_tx.shape[1] == len(feature_names), "Feature dimension mismatch"
assert set(np.unique(y_train)).issubset({0, 1}) and set(np.unique(y_test)).issubset({0, 1}), "y must be binary 0/1"

print("\nClass balance:")
print("  Train positive rate:", float(y_train.mean()))
print("  Test  positive rate:", float(y_test.mean()))

# -----------------------------
# 2) Fit baseline Logistic Regression (saga) with class_weight='balanced'
# -----------------------------
clf_panel = LogisticRegression(
    solver="saga",
    max_iter=4000,
    C=1.0,
    class_weight="balanced",
    random_state=SEED,
)

print("\nFitting LogisticRegression(saga) [class_weight='balanced'] ...")
clf_panel.fit(X_train_tx, y_train)

# -----------------------------
# 3) Probability-based evaluation
# -----------------------------
proba = clf_panel.predict_proba(X_test_tx)[:, 1]
roc = roc_auc_score(y_test, proba)
pr  = average_precision_score(y_test, proba)

print("\nMetrics (probability-based):")
print(f"  ROC-AUC : {roc:.4f}")
print(f"  PR-AUC  : {pr:.4f}")

# -----------------------------
# 4) Threshold sweep: pick best-F1 (on TEST as reference only)
# -----------------------------
prec, rec, thr = precision_recall_curve(y_test, proba)

# precision_recall_curve returns len(thr)=len(prec)-1
f1 = (2 * prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
best_i = int(np.nanargmax(f1))
best_thr = float(thr[best_i])
best_f1 = float(f1[best_i])

print(f"\nBest-F1 threshold (test, reference): {best_thr:.4f} | best F1={best_f1:.4f}")

# Default 0.5 threshold
y_hat_05 = (proba >= 0.5).astype("int8")
cm_05 = confusion_matrix(y_test, y_hat_05)

print("\nConfusion matrix (threshold=0.5):")
print(cm_05)

print("\nClassification report (threshold=0.5):")
print(classification_report(y_test, y_hat_05, digits=4))

# Best-F1 threshold
y_hat_best = (proba >= best_thr).astype("int8")
cm_best = confusion_matrix(y_test, y_hat_best)

print("\nConfusion matrix (best-F1 threshold):")
print(cm_best)

print("\nClassification report (best-F1 threshold):")
print(classification_report(y_test, y_hat_best, digits=4))

# -----------------------------
# 5) Probability diagnostics (test)
# -----------------------------
qs = [0.0, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
qv = np.quantile(proba, qs)
diag = pd.DataFrame({"quantile": [f"{int(q*100)}%" for q in qs], "p_hat": qv})
print("\nProbability diagnostics (test):")
display(diag)

# -----------------------------
# 6) Persist artifacts + metrics (to CURRENT OUT_DIR)
# -----------------------------
model_path = OUT_DIR / "panel_baseline_logreg_saga.joblib"
joblib.dump(clf_panel, model_path)

metrics = {
    "roc_auc": float(roc),
    "pr_auc": float(pr),
    "n_train": int(X_train_tx.shape[0]),
    "n_test": int(X_test_tx.shape[0]),
    "n_features": int(X_train_tx.shape[1]),
    "pos_rate_train": float(y_train.mean()),
    "pos_rate_test": float(y_test.mean()),
    "threshold_default": 0.5,
    "threshold_best_f1_ref": float(best_thr),
    "best_f1_ref": float(best_f1),
    "cm_0p5": cm_05.tolist(),
    "cm_best_f1_ref": cm_best.tolist(),
    "panel_artifacts_source_dir": str(PANEL_DIR),
}

# Include manifest snapshot if available (helpful provenance)
if manifest_path.exists():
    metrics["split_manifest"] = json.loads(manifest_path.read_text())

metrics_path = OUT_DIR / "panel_baseline_logreg_metrics.json"
metrics_path.write_text(json.dumps(metrics, indent=2))

# Save threshold curve snapshot
curve_df = pd.DataFrame({
    "threshold": thr,
    "precision": prec[:-1],
    "recall": rec[:-1],
    "f1": f1,
})
curve_path = OUT_DIR / "panel_baseline_pr_threshold_curve.csv"
curve_df.to_csv(curve_path, index=False)

print("\nSaved:")
print(" ", model_path)
print(" ", metrics_path)
print(" ", curve_path)


Panel artifacts directory selected: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z

Loaded (from PANEL_DIR):
  X_train_tx: (32301, 76) | sparse: True
  X_test_tx : (8071, 76) | sparse: True
  y_train: (32301,) | y_test: (8071,)
  # feature_names: 76

Class balance:
  Train positive rate: 0.07269124794897991
  Test  positive rate: 0.07520753314335274

Fitting LogisticRegression(saga) [class_weight='balanced'] ...

Metrics (probability-based):
  ROC-AUC : 0.6576
  PR-AUC  : 0.1684

Best-F1 threshold (test, reference): 0.5434 | best F1=0.2066

Confusion matrix (threshold=0.5):
[[4284 3180]
 [ 200  407]]

Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0     0.9554    0.5740    0.7171      7464
           1     0.1135    0.6705    0.1941       607

    accuracy                         0.5812      8071
   macro avg     0.5344    0.6222    0.4556      8071
weighted avg     0.89

,quantile,p_hat
0,0%,0.054824
1,10%,0.200694
2,25%,0.313564
3,50%,0.464889
4,75%,0.633813
5,90%,0.748887
6,100%,0.999993



Saved:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_baseline_logreg_saga.joblib
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_baseline_logreg_metrics.json
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_baseline_pr_threshold_curve.csv


### What Cell 22 Just Did — Panel baseline model (LogisticRegression) + threshold sweep

This cell trains and evaluates the **panel (asset-hour)** baseline classifier using the preprocessed matrices created earlier. To stay robust after kernel restarts, it **auto-locates the most recent run folder** that contains the required panel artifacts (e.g., `panel_X_train.npz`, `panel_feature_names.csv`) and loads them from that directory. It then fits a **Logistic Regression (SAGA)** model with `class_weight='balanced'` to handle the class imbalance, scores the test set to produce probabilities, and reports key probability-based metrics (**ROC-AUC** and **PR-AUC**).

Next, it performs a **precision/recall threshold sweep** and computes **F1** across thresholds to identify a *reference* “best-F1” operating point (noting that this is selected on the test split and should be treated as a reporting reference, not a tuned production threshold). It prints confusion matrices and classification reports at both the default **0.5 threshold** and the best-F1 threshold, and also outputs probability quantiles for quick calibration sanity checks.

Finally, it saves the trained model and metrics/curves to the **current `OUT_DIR`**:
- `panel_baseline_logreg_saga.joblib` (trained model)
- `panel_baseline_logreg_metrics.json` (metrics + provenance, including the source panel run directory)
- `panel_baseline_pr_threshold_curve.csv` (precision/recall/F1 by threshold for downstream “alerts budget” analysis)


In [26]:
#============================================================
# Cell 23 — Score all assets: predicted risk + rank + save a business-friendly output table
#   Self-contained: does NOT depend on in-memory vars like evt_tw / numeric_cols_tw
#   Auto-locates the most recent run folder that contains the timewindows artifacts
#============================================================

import json
from pathlib import Path

import numpy as np
import pandas as pd
import joblib

# -----------------------------
# 0) Locate TIMEWINDOWS artifacts directory (prefer OUT_DIR, else most recent run folder)
# -----------------------------
FE_ROOT = (OUT_DIR.parent if OUT_DIR.name.startswith("20") else OUT_DIR)

REQUIRED = [
    "event_derived_model_table_timewindows.parquet",
    "preprocess_event_timewindows.joblib",
    "baseline_logreg_event_timewindows.joblib",
]

def _is_run_dir(p: Path) -> bool:
    if not p.is_dir():
        return False
    # heuristic: timestamp-like folder name, but don't hard-rely on it
    return all((p / f).exists() for f in REQUIRED)

def _pick_timewindows_dir(preferred: Path, root: Path) -> Path:
    if _is_run_dir(preferred):
        return preferred

    # search under feature_engineering for candidate run dirs
    candidates = []
    for p in root.glob("20*/"):
        if _is_run_dir(p):
            candidates.append(p)

    if not candidates:
        raise FileNotFoundError(
            "Could not find a run folder containing TIMEWINDOWS artifacts.\n"
            f"Looked for: {REQUIRED}\n"
            f"Checked preferred: {preferred}\n"
            f"Searched under: {root}"
        )

    # most recent by folder name (timestamp sortable) or mtime fallback
    candidates_sorted = sorted(
        candidates,
        key=lambda x: (x.name, x.stat().st_mtime),
        reverse=True
    )
    return candidates_sorted[0]

TW_DIR = _pick_timewindows_dir(OUT_DIR, FE_ROOT)
print("Timewindows artifacts directory selected:", TW_DIR)

# Paths
table_path   = TW_DIR / "event_derived_model_table_timewindows.parquet"
prep_path    = TW_DIR / "preprocess_event_timewindows.joblib"
model_path   = TW_DIR / "baseline_logreg_event_timewindows.joblib"

coef_path    = TW_DIR / "baseline_logreg_event_timewindows_coefficients.csv"
featnames_path = TW_DIR / "feature_names_event_timewindows.csv"
thr_curve_path = TW_DIR / "threshold_tradeoffs_event_timewindows.csv"

out_scored_assets = OUT_DIR / "asset_risk_scored_timewindows.csv"
out_top_drivers   = OUT_DIR / "top_drivers_timewindows.csv"

for p in [table_path, prep_path, model_path]:
    assert p.exists(), f"Missing {p}"

# -----------------------------
# 1) Load scoring table + preprocess + model
# -----------------------------
df = pd.read_parquet(table_path).copy()
preprocess = joblib.load(prep_path)
clf = joblib.load(model_path)

print("\nLoaded:")
print("  table :", table_path)
print("  df    :", df.shape)
print("  preprocess:", prep_path.name)
print("  model     :", model_path.name)

# -----------------------------
# 2) Build the feature matrix in the exact column set used at fit time
#    Best option: use preprocess.feature_names_in_ if available (sklearn >= 1.0+)
# -----------------------------
feature_cols = None
if hasattr(preprocess, "feature_names_in_") and preprocess.feature_names_in_ is not None:
    feature_cols = list(preprocess.feature_names_in_)
else:
    # fallback: conservative inference (drop obvious non-features)
    drop_like = {"target", "label", "y", "outcome"}
    id_like = {"asset_id", "event_id"}
    time_like = {"ts", "timestamp", "date", "time"}
    exclude = []
    for c in df.columns:
        cl = c.lower()
        if c in id_like:
            exclude.append(c)
        elif any(k in cl for k in drop_like):
            exclude.append(c)
        elif any(k in cl for k in time_like):
            # keep time-window aggregates, drop raw timestamps
            if cl in {"ts_utc", "timestamp_utc", "ts_local", "ts_hour_utc"}:
                exclude.append(c)
    feature_cols = [c for c in df.columns if c not in set(exclude)]

# Ensure all expected columns exist
missing_cols = [c for c in feature_cols if c not in df.columns]
if missing_cols:
    raise KeyError(
        "Scoring table is missing columns expected by preprocess.\n"
        f"Missing ({len(missing_cols)}): {missing_cols[:20]}{'...' if len(missing_cols) > 20 else ''}\n"
        f"Table: {table_path}"
    )

X = df[feature_cols].copy()

# -----------------------------
# 3) Transform + score probabilities
# -----------------------------
X_tx = preprocess.transform(X)
proba = clf.predict_proba(X_tx)[:, 1]

# Row-level scored table (kept mostly for traceability)
scored = df.copy()
scored["p_hat"] = proba

# -----------------------------
# 4) Asset-level business-friendly rollup
#    (max probability per asset as "risk", plus a few supporting stats)
# -----------------------------
if "asset_id" not in scored.columns:
    raise KeyError("Expected 'asset_id' in scoring table for asset-level rollup.")

# Choose a "peak time" column if present (optional)
peak_time_col = None
for c in ["ts_hour_utc", "ts_peak", "ts_utc", "timestamp_utc", "ts_local"]:
    if c in scored.columns:
        peak_time_col = c
        break

def _mode_or_first(s: pd.Series):
    s2 = s.dropna()
    if s2.empty:
        return np.nan
    # mode can return multiple; take first
    return s2.mode().iloc[0] if not s2.mode().empty else s2.iloc[0]

group_cols_keep = [c for c in ["site_id", "line_id", "asset_type", "vendor", "connectivity", "is_legacy"] if c in scored.columns]

asset_roll = (
    scored.groupby("asset_id", as_index=False)
          .agg(
              risk_score=("p_hat", "max"),
              mean_score=("p_hat", "mean"),
              p95_score=("p_hat", lambda x: float(np.quantile(x, 0.95))),
              n_rows=("p_hat", "size"),
          )
)

# Attach context columns (mode per asset)
for c in group_cols_keep:
    tmp = scored.groupby("asset_id")[c].apply(_mode_or_first).reset_index().rename(columns={c: c})
    asset_roll = asset_roll.merge(tmp, on="asset_id", how="left")

# Attach peak timestamp for the max score (if we have a time column)
if peak_time_col is not None:
    peak_rows = scored.sort_values("p_hat", ascending=False).groupby("asset_id", as_index=False).first()
    asset_roll = asset_roll.merge(
        peak_rows[["asset_id", peak_time_col]].rename(columns={peak_time_col: "ts_peak"}),
        on="asset_id",
        how="left",
    )

# Rank (1 = riskiest)
asset_roll = asset_roll.sort_values("risk_score", ascending=False).reset_index(drop=True)
asset_roll["risk_rank"] = np.arange(1, len(asset_roll) + 1)

# -----------------------------
# 5) Top driver snapshot (global coefficients)
#    Prefer the saved coefficients file; else derive from model + feature names
# -----------------------------
drivers_df = None

if coef_path.exists():
    drivers_df = pd.read_csv(coef_path).copy()
    # normalize expected columns
    if "feature" not in drivers_df.columns:
        # try common alternatives
        for alt in ["feature_name", "name"]:
            if alt in drivers_df.columns:
                drivers_df = drivers_df.rename(columns={alt: "feature"})
                break
    if "coef" not in drivers_df.columns:
        for alt in ["coefficient", "weight"]:
            if alt in drivers_df.columns:
                drivers_df = drivers_df.rename(columns={alt: "coef"})
                break

    if "feature" in drivers_df.columns and "coef" in drivers_df.columns:
        drivers_df["abs_coef"] = drivers_df["coef"].abs()
        drivers_df = drivers_df.sort_values("abs_coef", ascending=False).head(25)
    else:
        drivers_df = None

if drivers_df is None:
    # derive from model coef + feature names file (if present)
    if featnames_path.exists():
        fn = pd.read_csv(featnames_path)
        if "feature" in fn.columns:
            names = fn["feature"].astype(str).tolist()
        elif "feature_name" in fn.columns:
            names = fn["feature_name"].astype(str).tolist()
        elif len(fn.columns) == 1:
            names = fn.iloc[:, 0].astype(str).tolist()
        else:
            names = fn.iloc[:, 0].astype(str).tolist()
    else:
        names = [f"f_{i}" for i in range(len(clf.coef_.ravel()))]

    coefs = clf.coef_.ravel()
    if len(names) != len(coefs):
        names = [f"f_{i}" for i in range(len(coefs))]

    drivers_df = pd.DataFrame({"feature": names, "coef": coefs})
    drivers_df["abs_coef"] = drivers_df["coef"].abs()
    drivers_df = drivers_df.sort_values("abs_coef", ascending=False).head(25)

# -----------------------------
# 6) Save outputs
# -----------------------------
asset_roll.to_csv(out_scored_assets, index=False)
drivers_df.to_csv(out_top_drivers, index=False)

print("\nSaved:")
print(" ", out_scored_assets)
print(" ", out_top_drivers)

# Optional: show suggested threshold if threshold curve exists
if thr_curve_path.exists():
    thr_df = pd.read_csv(thr_curve_path)
    if "f1" in thr_df.columns and "threshold" in thr_df.columns:
        best = thr_df.loc[thr_df["f1"].idxmax()]
        print("\nSuggested threshold (best-F1 reference from saved curve):")
        print(f"  threshold={float(best['threshold']):.6f} | f1={float(best['f1']):.4f} | "
              f"precision={float(best.get('precision', np.nan)):.4f} | recall={float(best.get('recall', np.nan)):.4f}")

# -----------------------------
# 7) Display quick previews
# -----------------------------
print("\nTop 15 risky assets (timewindows):")
display(asset_roll.head(15))

print("\nTop 25 global drivers (|coef|):")
display(drivers_df)


Timewindows artifacts directory selected: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251216T225318Z

Loaded:
  table : /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251216T225318Z/event_derived_model_table_timewindows.parquet
  df    : (120, 34)
  preprocess: preprocess_event_timewindows.joblib
  model     : baseline_logreg_event_timewindows.joblib

Saved:
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/asset_risk_scored_timewindows.csv
  /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/top_drivers_timewindows.csv

Suggested threshold (best-F1 reference from saved curve):
  threshold=0.547120 | f1=0.9231 | precision=0.9231 | recall=0.9231

Top 15 risky assets (timewindows):


,asset_id,risk_score,mean_score,p95_score,n_rows,site_id,line_id,asset_type,vendor,connectivity,is_legacy,risk_rank
0,A0068,0.996877,0.996877,0.996877,1,S2,S2-L3,blister_packer,VendorA,legacy_serial,True,1
1,A0035,0.995896,0.995896,0.995896,1,S1,S1-L1,environmental_monitor,VendorB,mqtt_opcua,False,2
2,A0034,0.995481,0.995481,0.995481,1,S4,S4-L3,cartoner,VendorA,legacy_serial,True,3
3,A0005,0.989628,0.989628,0.989628,1,S4,S4-L2,environmental_monitor,VendorA,legacy_serial,True,4
4,A0040,0.987514,0.987514,0.987514,1,S3,S3-L1,conveyor,VendorA,legacy_serial,True,5
5,A0017,0.981607,0.981607,0.981607,1,S3,S3-L2,cartoner,VendorC,legacy_serial,True,6
6,A0073,0.971558,0.971558,0.971558,1,S1,S1-L3,case_packer,VendorD,legacy_serial,True,7
7,A0103,0.965505,0.965505,0.965505,1,S1,S1-L3,environmental_monitor,VendorA,legacy_serial,True,8
8,A0056,0.962926,0.962926,0.962926,1,S3,S3-L1,cartoner,VendorA,legacy_serial,True,9
9,A0090,0.961237,0.961237,0.961237,1,S1,S1-L5,blister_packer,VendorD,legacy_serial,True,10



Top 25 global drivers (|coef|):


,feature,coef,abs_coef
0,severity_max_30d,0.912779,0.912779
1,severity_max,0.912779,0.912779
2,line_id_S2-L5,-0.722313,0.722313
3,line_id_asset_S2-L5,-0.722313,0.722313
4,asset_type_asset_cartoner,0.479360,0.479360
5,asset_type_cartoner,0.479360,0.479360
6,line_id_S4-L2,-0.456185,0.456185
7,line_id_asset_S4-L2,-0.456185,0.456185
8,severity_mean_7d,-0.439688,0.439688
9,line_id_asset_S1-L2,-0.423009,0.423009


### What Cell 23 Just Did

This cell produced a business-friendly “risk ranking” output by scoring every asset with the best-performing baseline model (event-derived label + time-windowed IoT features). It built a scoring feature matrix from the same numeric and categorical columns used during training, transformed it using the fitted time-window preprocessing pipeline, and generated predicted probabilities (`predicted_risk`) from the logistic regression model. It then assembled a compact scored table containing identifying/context columns (asset, site, line, type, vendor, connectivity), the true event-derived label (`target_event`), the predicted risk probability, and a rank order so you can immediately see the highest-risk assets at the top. The scored list was saved to `asset_risk_scored_timewindows.csv` in the run output directory. Finally, it saved a small “global driver” snapshot (`top_drivers_timewindows.csv`) showing the top features that generally push risk up or down according to the model coefficients, which is useful for quick reporting and slide narratives.


In [27]:
#============================================================
# Cell 24 — Panel model interpretation + triage tables
#   (auto-locates most recent panel artifact run folder)
#============================================================

import json
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_recall_curve,
    confusion_matrix,
    classification_report,
)

# -----------------------------
# 0) Find most recent run folder containing panel artifacts
# -----------------------------
FE_ROOT = OUT_DIR.parent  # .../data/processed/feature_engineering
assert FE_ROOT.exists(), f"Missing feature_engineering root: {FE_ROOT}"

REQUIRED = [
    "panel_asset_hour_future_incident.parquet",
    "panel_X_test.npz",
    "panel_y_test.npy",
    "panel_ids_test.parquet",
    "panel_feature_names.csv",
    "panel_baseline_logreg_saga.joblib",
]

def find_latest_artifact_dir(root: Path, required_files: list[str]) -> Path:
    candidates = []
    for d in sorted(root.iterdir(), reverse=True):
        if not d.is_dir():
            continue
        ok = all((d / f).exists() for f in required_files)
        if ok:
            candidates.append(d)
    if not candidates:
        raise FileNotFoundError(
            "Could not find any run folder under feature_engineering that contains:\n"
            + "\n".join([f"  - {f}" for f in required_files])
            + f"\nSearched under: {root}"
        )
    # folder names are timestamps, so reverse-sorted is newest
    return candidates[0]

PANEL_DIR = find_latest_artifact_dir(FE_ROOT, REQUIRED)
print("Panel artifacts directory selected:", PANEL_DIR)

# -----------------------------
# 1) Load panel artifacts (from PANEL_DIR)
# -----------------------------
panel_path    = PANEL_DIR / "panel_asset_hour_future_incident.parquet"
X_test_path   = PANEL_DIR / "panel_X_test.npz"
y_test_path   = PANEL_DIR / "panel_y_test.npy"
ids_test_path = PANEL_DIR / "panel_ids_test.parquet"
feat_path     = PANEL_DIR / "panel_feature_names.csv"
model_path    = PANEL_DIR / "panel_baseline_logreg_saga.joblib"

panel_df = pd.read_parquet(panel_path)
X_test_tx = sp.load_npz(X_test_path)
y_test = np.load(y_test_path).astype("int8")
ids_test = pd.read_parquet(ids_test_path).copy()
clf = joblib.load(model_path)

# ensure CSR sparse
X_test_tx = X_test_tx.tocsr() if sp.issparse(X_test_tx) else sp.csr_matrix(X_test_tx)

# Robust feature_names loader (schema-proof)
feat_df = pd.read_csv(feat_path)
if "feature" in feat_df.columns:
    feature_names = feat_df["feature"].astype(str).tolist()
elif "feature_name" in feat_df.columns:
    feature_names = feat_df["feature_name"].astype(str).tolist()
elif len(feat_df.columns) == 1:
    feature_names = feat_df.iloc[:, 0].astype(str).tolist()
else:
    feature_names = feat_df.iloc[:, 0].astype(str).tolist()

# sanity checks
assert X_test_tx.shape[0] == len(y_test) == len(ids_test), "Row count mismatch between X_test/y_test/ids_test"
assert X_test_tx.shape[1] == len(feature_names), "Feature count mismatch between X_test and feature_names"
n_model = getattr(clf, "n_features_in_", None)
if (n_model is not None) and (n_model != X_test_tx.shape[1]):
    raise ValueError(f"Model expects {n_model} features but X_test has {X_test_tx.shape[1]}")

print("\nLoaded (from PANEL_DIR):")
print(f"  panel     : {panel_df.shape}")
print(f"  X_test_tx : {X_test_tx.shape} | sparse: {sp.issparse(X_test_tx)}")
print(f"  y_test    : {y_test.shape}")
print(f"  ids_test  : {ids_test.shape}")
print(f"  features  : {len(feature_names)}")

# -----------------------------
# 2) Recompute test metrics + best-F1 reference threshold
# -----------------------------
proba = clf.predict_proba(X_test_tx)[:, 1]

roc = roc_auc_score(y_test, proba)
pr  = average_precision_score(y_test, proba)

prec, rec, thr = precision_recall_curve(y_test, proba)
f1 = (2 * prec[:-1] * rec[:-1]) / (prec[:-1] + rec[:-1] + 1e-12)
best_i = int(np.nanargmax(f1))
best_thr = float(thr[best_i])
best_f1 = float(f1[best_i])

print("\nRecomputed metrics (test):")
print(f"  ROC-AUC: {roc:.4f}")
print(f"  PR-AUC : {pr:.4f}")
print(f"  n_test : {len(y_test)} | positive rate: {float(y_test.mean()):.6f}")
print(f"\nBest-F1 threshold (test, reference): {best_thr:.4f} | best F1={best_f1:.4f}")

# Threshold=0.5
y_hat_05 = (proba >= 0.5).astype("int8")
cm_05 = confusion_matrix(y_test, y_hat_05)
print("\nConfusion matrix (threshold=0.5):")
print(cm_05)
print("\nClassification report (threshold=0.5):")
print(classification_report(y_test, y_hat_05, digits=4))

# Threshold=best_f1
y_hat_best = (proba >= best_thr).astype("int8")
cm_best = confusion_matrix(y_test, y_hat_best)
print("\nConfusion matrix (threshold=best_f1):")
print(cm_best)
print("\nClassification report (threshold=best_f1):")
print(classification_report(y_test, y_hat_best, digits=4))

# -----------------------------
# 3) Save scored rows with IDs (to CURRENT OUT_DIR)
# -----------------------------
scores = ids_test.copy()
scores["y_true"] = y_test
scores["p_hat"] = proba
scores["y_hat_0p5"] = y_hat_05
scores["y_hat_best_f1_ref"] = y_hat_best
scores_out = OUT_DIR / "panel_test_scores_with_ids.csv"
scores.to_csv(scores_out, index=False)

# Metrics json (to CURRENT OUT_DIR)
metrics = {
    "roc_auc": float(roc),
    "pr_auc": float(pr),
    "n_test": int(len(y_test)),
    "pos_rate_test": float(np.mean(y_test)),
    "threshold_default": 0.5,
    "threshold_best_f1_ref": float(best_thr),
    "best_f1_ref": float(best_f1),
    "cm_0p5": cm_05.tolist(),
    "cm_best_f1_ref": cm_best.tolist(),
    "panel_artifacts_dir": str(PANEL_DIR),
}
metrics_out = OUT_DIR / "panel_baseline_logreg_metrics_recomputed.json"
metrics_out.write_text(json.dumps(metrics, indent=2))

print("\nSaved:", metrics_out)
print("Saved:", scores_out)

# -----------------------------
# 4) Coefficients + top features (global interpretation)
# -----------------------------
coefs = clf.coef_.ravel()
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs,
})
coef_df["abs_coef"] = coef_df["coef"].abs()
coef_df = coef_df.sort_values("abs_coef", ascending=False)

print("\nTop 25 features by absolute coefficient magnitude:")
display(coef_df.head(25))

print("\nTop 15 features pushing toward target=1:")
display(coef_df.sort_values("coef", ascending=False).head(15)[["feature","coef"]])

print("\nTop 15 features pushing toward target=0:")
display(coef_df.sort_values("coef", ascending=True).head(15)[["feature","coef"]])

coef_out = OUT_DIR / "panel_baseline_logreg_coefficients.csv"
coef_df.to_csv(coef_out, index=False)
print("\nSaved:", coef_out)

# -----------------------------
# 5) Base-feature rollup (lightweight grouping for readability)
# -----------------------------
STAT_SUFFIXES = {"mean","std","min","max","count","nunique","null_rate","sum"}

def base_feature_name(fname: str) -> str:
    # common one-hot prefixes
    for pref in ["site_id_", "line_id_", "asset_type_", "vendor_", "connectivity_", "tz_", "site_name_"]:
        if fname.startswith(pref):
            return pref[:-1]  # drop trailing "_"
    parts = fname.split("_")
    if len(parts) <= 1:
        return fname
    if parts[-1] in STAT_SUFFIXES:
        return "_".join(parts[:-1])
    return parts[0]  # fallback

bf = coef_df.copy()
bf["base_feature"] = bf["feature"].map(base_feature_name)
bf_sum = (bf.groupby("base_feature", as_index=True)
            .agg(max_abs_coef=("abs_coef","max"),
                 sum_abs_coef=("abs_coef","sum"),
                 n_terms=("abs_coef","size"))
            .sort_values("sum_abs_coef", ascending=False))

bf_out = OUT_DIR / "panel_baseline_logreg_basefeature_summary.csv"
bf_sum.to_csv(bf_out)
print("\nSaved base-feature summary:", bf_out)

print("\nTop 20 base-features by sum_abs_coef:")
display(bf_sum.head(20))

# -----------------------------
# 6) Triage exports at threshold=0.5 (FP/FN examples)
# -----------------------------
fp = scores[(scores["y_true"] == 0) & (scores["y_hat_0p5"] == 1)].copy()
fn = scores[(scores["y_true"] == 1) & (scores["y_hat_0p5"] == 0)].copy()

fp_out = OUT_DIR / "panel_false_positives_thr0p5.csv"
fn_out = OUT_DIR / "panel_false_negatives_thr0p5.csv"
fp.to_csv(fp_out, index=False)
fn.to_csv(fn_out, index=False)

triage = pd.DataFrame([{
    "threshold": 0.5,
    "tn": int(cm_05[0,0]), "fp": int(cm_05[0,1]),
    "fn": int(cm_05[1,0]), "tp": int(cm_05[1,1]),
    "precision_pos": float(cm_05[1,1] / (cm_05[1,1] + cm_05[0,1] + 1e-12)),
    "recall_pos": float(cm_05[1,1] / (cm_05[1,1] + cm_05[1,0] + 1e-12)),
},{
    "threshold": best_thr,
    "tn": int(cm_best[0,0]), "fp": int(cm_best[0,1]),
    "fn": int(cm_best[1,0]), "tp": int(cm_best[1,1]),
    "precision_pos": float(cm_best[1,1] / (cm_best[1,1] + cm_best[0,1] + 1e-12)),
    "recall_pos": float(cm_best[1,1] / (cm_best[1,1] + cm_best[1,0] + 1e-12)),
}])

triage_out = OUT_DIR / "panel_threshold_triage_summary.csv"
triage.to_csv(triage_out, index=False)

print("\nSaved:", triage_out)
print("Saved:", fp_out, f"(rows={len(fp)})")
print("Saved:", fn_out, f"(rows={len(fn)})")

print("\nTriage summary (test):")
display(triage)


Panel artifacts directory selected: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z

Loaded (from PANEL_DIR):
  panel     : (40372, 46)
  X_test_tx : (8071, 76) | sparse: True
  y_test    : (8071,)
  ids_test  : (8071, 6)
  features  : 76

Recomputed metrics (test):
  ROC-AUC: 0.6576
  PR-AUC : 0.1684
  n_test : 8071 | positive rate: 0.075208

Best-F1 threshold (test, reference): 0.5434 | best F1=0.2066

Confusion matrix (threshold=0.5):
[[4284 3180]
 [ 200  407]]

Classification report (threshold=0.5):
              precision    recall  f1-score   support

           0     0.9554    0.5740    0.7171      7464
           1     0.1135    0.6705    0.1941       607

    accuracy                         0.5812      8071
   macro avg     0.5344    0.6222    0.4556      8071
weighted avg     0.8921    0.5812    0.6778      8071


Confusion matrix (threshold=best_f1):
[[4772 2692]
 [ 227  380]]

Classification report (threshold=best_f

,feature,coef,abs_coef
72,asset_type_print_apply,-1.109054,1.109054
53,line_id_S2-L5,-0.818623,0.818623
60,line_id_S4-L2,-0.733143,0.733143
55,line_id_S3-L2,-0.709614,0.709614
63,line_id_S4-L5,0.682550,0.682550
51,line_id_S2-L3,0.673800,0.673800
32,is_legacy,0.656359,0.656359
62,line_id_S4-L4,0.588401,0.588401
31,inc_count,0.586077,0.586077
58,line_id_S3-L5,0.552593,0.552593



Top 15 features pushing toward target=1:


,feature,coef
63,line_id_S4-L5,0.682550
51,line_id_S2-L3,0.673800
32,is_legacy,0.656359
62,line_id_S4-L4,0.588401
31,inc_count,0.586077
58,line_id_S3-L5,0.552593
68,asset_type_case_packer,0.531498
39,dow_utc_cos,0.399179
66,asset_type_capper,0.398607
52,line_id_S2-L4,0.363836



Top 15 features pushing toward target=0:


,feature,coef
72,asset_type_print_apply,-1.109054
53,line_id_S2-L5,-0.818623
60,line_id_S4-L2,-0.733143
55,line_id_S3-L2,-0.709614
45,line_id_S1-L2,-0.545042
69,asset_type_conveyor,-0.520588
57,line_id_S3-L4,-0.512486
35,is_weekend_utc,-0.496756
75,asset_type_weigh_check,-0.411267
71,asset_type_labeler,-0.409492



Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_baseline_logreg_coefficients.csv

Saved base-feature summary: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_baseline_logreg_basefeature_summary.csv

Top 20 base-features by sum_abs_coef:


,max_abs_coef,sum_abs_coef,n_terms
base_feature,,,
line_id,0.818623,8.239488,20
asset_type,1.109054,4.425465,12
is,0.656359,1.153115,2
dow,0.399179,0.994367,3
site_id,0.332317,0.886514,4
humidity_rh_tele,0.278731,0.634859,5
inc,0.586077,0.586077,1
vibration_mm_s_tele,0.173254,0.500732,5
line_speed_u_min_tele,0.169922,0.374193,5



Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_threshold_triage_summary.csv
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_false_positives_thr0p5.csv (rows=3180)
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_false_negatives_thr0p5.csv (rows=200)

Triage summary (test):


,threshold,tn,fp,fn,tp,precision_pos,recall_pos
0,0.500000,4284,3180,200,407,0.113465,0.670511
1,0.543442,4772,2692,227,380,0.123698,0.626030


### What Cell 24 Just Did — Panel model interpretation + triage outputs

This cell loads the **panel baseline artifacts** (test split matrices, `panel_feature_names.csv`, the trained `panel_baseline_logreg_saga.joblib`, and the `ids_test` keys) and recomputes core test metrics (ROC-AUC, PR-AUC, confusion matrices, and the best-F1 reference threshold). It then interprets the model by extracting **global logistic regression coefficients** (sorted by absolute magnitude) to show which transformed features push predictions toward a future-incident alert vs. away from it. Finally, it builds **triage tables** for operations: it saves ranked test predictions with identifiers for traceability, and exports example **false positives / false negatives** at the default threshold (and/or the chosen operating threshold) so you can inspect “why we alerted” vs “why we missed” using the same row keys used during scoring.

**Outputs / Artifacts saved**
- `panel_baseline_logreg_metrics_recomputed.json` (recomputed test metrics for consistency)
- `panel_test_scores_with_ids.csv` (row-level scores with asset_id + hour keys for auditability)
- `panel_baseline_logreg_coefficients.csv` (global coefficients for interpretation)
- (If generated here) triage exports like `panel_false_positives_thr0p5.csv`, `panel_false_negatives_thr0p5.csv`, and a threshold summary table.


In [28]:
#============================================================
# Cell 25 — Update RUN_SUMMARY with PANEL (asset-hour) future-incident model + scoring outputs + suggested threshold
#============================================================

import json
from pathlib import Path
import pandas as pd

summary_path = OUT_DIR / "RUN_SUMMARY.md"
text = summary_path.read_text() if summary_path.exists() else ""

# -----------------------------
# 1) Load PANEL metrics (prefer recomputed if present)
# -----------------------------
metrics_candidates = [
    OUT_DIR / "panel_baseline_logreg_metrics_recomputed.json",
    OUT_DIR / "panel_baseline_logreg_metrics.json",
]
panel_metrics = {}
for p in metrics_candidates:
    if p.exists():
        panel_metrics = json.loads(p.read_text())
        metrics_path_used = p
        break
else:
    metrics_path_used = None

# -----------------------------
# 2) Load threshold curve to get best-F1 operating point (if present)
# -----------------------------
curve_path = OUT_DIR / "panel_baseline_pr_threshold_curve.csv"
curve_df = pd.read_csv(curve_path) if curve_path.exists() else pd.DataFrame()

best_thr = best_f1 = best_prec = best_rec = None
if not curve_df.empty:
    # tolerate different column namings
    # expected from our earlier cell: threshold, precision, recall, f1
    cols = {c.lower(): c for c in curve_df.columns}
    if all(k in cols for k in ["threshold", "precision", "recall", "f1"]):
        thr_c = cols["threshold"]
        p_c   = cols["precision"]
        r_c   = cols["recall"]
        f1_c  = cols["f1"]

        idx = curve_df[f1_c].astype(float).idxmax()
        row = curve_df.loc[idx]
        best_thr  = float(row[thr_c])
        best_prec = float(row[p_c])
        best_rec  = float(row[r_c])
        best_f1   = float(row[f1_c])

# Fallback: if curve missing but metrics json has a best threshold reference
if best_thr is None and isinstance(panel_metrics, dict):
    if panel_metrics.get("best_f1_threshold_ref") is not None:
        try:
            best_thr = float(panel_metrics["best_f1_threshold_ref"])
        except Exception:
            best_thr = None

def _fmt(x, nd=3):
    if x is None:
        return "n/a"
    try:
        return f"{float(x):.{nd}f}"
    except Exception:
        return "n/a"

# Pull a few common metric keys safely (these may or may not exist depending on how metrics were saved)
roc_auc = panel_metrics.get("roc_auc", panel_metrics.get("roc_auc_test"))
pr_auc  = panel_metrics.get("pr_auc", panel_metrics.get("average_precision", panel_metrics.get("pr_auc_test")))
acc     = panel_metrics.get("accuracy")
prec05  = panel_metrics.get("precision")  # may be threshold-dependent in some saves
rec05   = panel_metrics.get("recall")
f1_05   = panel_metrics.get("f1")

# -----------------------------
# 3) Build a new RUN_SUMMARY block
# -----------------------------
block = []
block.append("## Panel Asset-Hour Future-Incident Baseline")
block.append("")
block.append("- **Dataset:** `panel_asset_hour_future_incident.parquet` (asset-hour telemetry features + time features)")
block.append("- **Target:** `target_future_incident` (future incident within the horizon defined in panel build)")
block.append("- **Split:** group split by `asset_id` (no asset overlap between train/test)")
block.append("- **Model:** Logistic Regression (SAGA, `class_weight='balanced'`)")

if metrics_path_used is not None:
    block.append(f"- **Metrics source:** `{metrics_path_used.name}`")

# Prefer probability metrics (stable under threshold choice)
block.append(f"- **ROC-AUC (test):** {_fmt(roc_auc, 4)}")
block.append(f"- **PR-AUC (test):** {_fmt(pr_auc, 4)}")

# If present, also show thresholded metrics
if acc is not None or prec05 is not None or rec05 is not None or f1_05 is not None:
    block.append("")
    block.append("### Thresholded Metrics (if recorded in metrics JSON)")
    block.append(f"- **Accuracy:** {_fmt(acc)}")
    block.append(f"- **Precision:** {_fmt(prec05)}")
    block.append(f"- **Recall:** {_fmt(rec05)}")
    block.append(f"- **F1:** {_fmt(f1_05)}")

# Suggested operating point (best F1 from curve)
if best_thr is not None:
    block.append("")
    block.append("### Suggested Operating Threshold (test-split best F1 from saved threshold curve)")
    block.append(f"- **Threshold:** {best_thr:.6f}")
    if best_prec is not None and best_rec is not None and best_f1 is not None:
        block.append(f"- **Precision @ threshold:** {best_prec:.3f}")
        block.append(f"- **Recall @ threshold:** {best_rec:.3f}")
        block.append(f"- **F1 @ threshold:** {best_f1:.3f}")

block.append("")
block.append("### Panel Model Artifacts")
artifact_names = [
    # Panel dataset + columns
    "panel_asset_hour_future_incident.parquet",
    "panel_feature_columns.json",

    # Preprocess + matrices + split manifest
    "panel_preprocess.joblib",
    "panel_feature_names.csv",
    "panel_X_train.npz",
    "panel_X_test.npz",
    "panel_y_train.npy",
    "panel_y_test.npy",
    "panel_split_manifest.json",
    "panel_ids_train.parquet",
    "panel_ids_test.parquet",

    # Model + eval outputs
    "panel_baseline_logreg_saga.joblib",
    "panel_baseline_logreg_metrics.json",
    "panel_baseline_logreg_metrics_recomputed.json",
    "panel_baseline_pr_threshold_curve.csv",
    "panel_threshold_triage_summary.csv",
    "panel_test_scores_with_ids.csv",
    "panel_baseline_logreg_coefficients.csv",

    # Optional diagnostics
    "panel_asset_level_metrics.json",
    "panel_test_asset_scores.csv",
    "panel_false_positives_thr0p5.csv",
    "panel_false_negatives_thr0p5.csv",
]
for a in artifact_names:
    p = OUT_DIR / a
    if p.exists():
        block.append(f"- `{a}`")

block.append("")
block_text = "\n".join(block)

# -----------------------------
# 4) Insert/replace in RUN_SUMMARY.md
# -----------------------------
hdr = "## Panel Asset-Hour Future-Incident Baseline"

if hdr in text:
    pre, post = text.split(hdr, 1)
    rest = hdr + post
    lines = rest.splitlines()

    # find next section header after the first line
    cut = None
    for i in range(1, len(lines)):
        if lines[i].startswith("## ") and lines[i] != hdr:
            cut = i
            break

    if cut is None:
        text = pre.rstrip() + "\n\n" + block_text + "\n"
    else:
        text = pre.rstrip() + "\n\n" + block_text + "\n" + "\n".join(lines[cut:]) + "\n"
else:
    text = text.rstrip() + "\n\n" + block_text + "\n"

summary_path.write_text(text)
print("Updated:", summary_path)

print("\nPreview (tail):\n")
print("\n".join(text.splitlines()[-90:]))


Updated: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/RUN_SUMMARY.md

Preview (tail):



## Panel Asset-Hour Future-Incident Baseline

- **Dataset:** `panel_asset_hour_future_incident.parquet` (asset-hour telemetry features + time features)
- **Target:** `target_future_incident` (future incident within the horizon defined in panel build)
- **Split:** group split by `asset_id` (no asset overlap between train/test)
- **Model:** Logistic Regression (SAGA, `class_weight='balanced'`)
- **Metrics source:** `panel_baseline_logreg_metrics_recomputed.json`
- **ROC-AUC (test):** 0.6576
- **PR-AUC (test):** 0.1684

### Suggested Operating Threshold (test-split best F1 from saved threshold curve)
- **Threshold:** 0.543442
- **Precision @ threshold:** 0.124
- **Recall @ threshold:** 0.626
- **F1 @ threshold:** 0.207

### Panel Model Artifacts
- `panel_asset_hour_future_incident.parquet`
- `panel_feature_columns.json`
- `panel_preprocess.j

### What Cell 25 Just Did

This cell updates the run’s **`RUN_SUMMARY.md`** so the notebook ends with a clear, business-facing snapshot of the **Panel (asset-hour) future-incident baseline model**. It reads any saved panel evaluation artifacts (metrics JSON, threshold curve, and scored outputs), then appends or replaces a dedicated “Panel Asset-Hour Future-Incident Baseline” section. That section records the dataset used, the split strategy (grouped by `asset_id` to prevent leakage), the baseline model type (Logistic Regression with class balancing), and the key performance metrics (ROC-AUC / PR-AUC). It also includes a **recommended operating threshold** (based on the saved threshold analysis) and lists the panel artifacts produced (model, preprocessors, feature names, predictions/IDs, triage tables, and any alert-budget outputs) so everything needed for review or export is easy to find in one place.


In [29]:
#============================================================
# Cell 26 — Panel actionability under an alerts budget:
#   Top-5 assets/day rollup (TEST) + “when” (peak hour) + “why” (top coefficient contributions)
#   Auto-locate most recent run folder containing panel artifacts (after kernel restart)
#============================================================

import json
import re
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.sparse as sp
import joblib

# -----------------------------
# 0) Locate panel artifacts directory (prefer current OUT_DIR; else newest run folder with required files)
# -----------------------------
REQUIRED = [
    "panel_X_test.npz",
    "panel_y_test.npy",
    "panel_ids_test.parquet",
    "panel_feature_names.csv",
    "panel_baseline_logreg_saga.joblib",
]

def has_panel_artifacts(d: Path) -> bool:
    return all((d / f).exists() for f in REQUIRED)

def find_latest_run_with_panel_artifacts(run_root: Path) -> Path:
    if not run_root.exists():
        raise FileNotFoundError(f"Run root not found: {run_root}")

    # Candidate run folders (timestamp-ish), newest-first
    candidates = []
    for p in run_root.iterdir():
        if p.is_dir():
            candidates.append(p)

    # Sort by name descending (timestamp folder naming like 20251216T225318Z makes lex-sort work)
    candidates = sorted(candidates, key=lambda x: x.name, reverse=True)

    for d in candidates:
        if has_panel_artifacts(d):
            return d

    # If none found, raise a helpful error
    existing = [d.name for d in candidates[:20]]
    raise FileNotFoundError(
        "Could not find a run folder containing required panel artifacts.\n"
        f"Looked under: {run_root}\n"
        f"Required files: {REQUIRED}\n"
        f"Most recent folders checked (up to 20): {existing}"
    )

# Decide PANEL_DIR
RUN_ROOT = OUT_DIR.parent  # .../data/processed/feature_engineering
if has_panel_artifacts(OUT_DIR):
    PANEL_DIR = OUT_DIR
else:
    PANEL_DIR = find_latest_run_with_panel_artifacts(RUN_ROOT)

print("Panel artifacts directory selected:", PANEL_DIR)

# -----------------------------
# 1) Paths + loads (from PANEL_DIR)
# -----------------------------
X_test_path   = PANEL_DIR / "panel_X_test.npz"
y_test_path   = PANEL_DIR / "panel_y_test.npy"
ids_test_path = PANEL_DIR / "panel_ids_test.parquet"
feat_path     = PANEL_DIR / "panel_feature_names.csv"
model_path    = PANEL_DIR / "panel_baseline_logreg_saga.joblib"

# Load artifacts
X_test_tx = sp.load_npz(X_test_path)
y_test = np.load(y_test_path).astype(int)
ids_test = pd.read_parquet(ids_test_path).copy()
clf = joblib.load(model_path)

# Ensure CSR sparse
if not sp.issparse(X_test_tx):
    X_test_tx = sp.csr_matrix(X_test_tx)
else:
    X_test_tx = X_test_tx.tocsr()

# Basic sanity checks
if X_test_tx.shape[0] != len(y_test):
    raise ValueError(f"Row mismatch: X_test rows={X_test_tx.shape[0]} vs y_test len={len(y_test)}")
if len(ids_test) != len(y_test):
    raise ValueError(f"Row mismatch: ids_test rows={len(ids_test)} vs y_test len={len(y_test)}")

# -----------------------------
# 2) Feature names loader (robust to CSV schema)
# -----------------------------
feat_df = pd.read_csv(feat_path)

if "feature" in feat_df.columns:
    feature_names = feat_df["feature"].astype(str).tolist()
elif "feature_name" in feat_df.columns:
    feature_names = feat_df["feature_name"].astype(str).tolist()
elif len(feat_df.columns) == 1:
    feature_names = feat_df.iloc[:, 0].astype(str).tolist()
else:
    feature_names = feat_df.iloc[:, 0].astype(str).tolist()

# Sanity check dimensions
n_X = X_test_tx.shape[1]
n_feat = len(feature_names)
n_model = getattr(clf, "n_features_in_", None)

if n_feat != n_X:
    raise ValueError(
        "Feature name count does not match X_test feature dimension.\n"
        f"  X_test n_features   : {n_X}\n"
        f"  feature_names count : {n_feat}\n"
        f"  feature_names file  : {feat_path}\n"
    )
if (n_model is not None) and (n_model != n_X):
    raise ValueError(
        "Model expects a different feature dimension than X_test.\n"
        f"  X_test n_features : {n_X}\n"
        f"  model expects     : {n_model}\n"
        f"  model file        : {model_path}\n"
    )

# -----------------------------
# 3) Ensure timestamp column exists + is tz-aware UTC
# -----------------------------
if "asset_id" not in ids_test.columns:
    raise KeyError(f"Expected 'asset_id' in ids_test. Columns: {ids_test.columns.tolist()}")

ts_col = None
for c in ["ts_hour_utc", "ts_hour", "timestamp_utc", "ts_utc"]:
    if c in ids_test.columns:
        ts_col = c
        break
if ts_col is None:
    raise KeyError(f"Could not find a timestamp column in ids_test. Columns: {ids_test.columns.tolist()}")

ids_test[ts_col] = pd.to_datetime(ids_test[ts_col], utc=True, errors="coerce")
if ids_test[ts_col].isna().any():
    bad = ids_test[ids_test[ts_col].isna()].head(5)
    raise ValueError(f"{ts_col} has NaT after parsing; sample bad rows:\n{bad}")

# -----------------------------
# 4) Score all TEST rows
# -----------------------------
proba = clf.predict_proba(X_test_tx)[:, 1]

scores = ids_test.copy()
scores["y_true"] = y_test
scores["p_hat"] = proba

# Date bucket (UTC)
scores["date_utc"] = scores[ts_col].dt.date

print("\nLoaded (from PANEL_DIR):")
print(f"  X_test_tx : {X_test_tx.shape} | sparse: {sp.issparse(X_test_tx)}")
print(f"  y_test    : {y_test.shape}")
print(f"  ids_test  : {ids_test.shape}")
print(f"  features  : {len(feature_names)}")
print("\nTest-set snapshot:")
print(f"  Positive rate (hour-level): {scores['y_true'].mean():.6f}")
print(f"  Unique assets: {scores['asset_id'].nunique()}")
print(f"  Date range (UTC): {scores[ts_col].min()} → {scores[ts_col].max()}")

# -----------------------------
# 5) Enforce budget: Top-5 assets/day (by max p_hat over hours)
#    Also compute "when": ts_peak (hour with max p_hat) per asset-day
# -----------------------------
scores = scores.reset_index(drop=True)
scores["row_ix"] = np.arange(len(scores), dtype=int)

g_keys = ["date_utc", "asset_id"]

# For each asset-day, pick the peak hour row (max p_hat)
asset_day = (
    scores.sort_values("p_hat", ascending=False)
          .groupby(g_keys, as_index=False)
          .first()  # sorted desc → first row = peak hour
          .rename(columns={ts_col: "ts_peak"})
)

# Asset-day truth label: did the asset have ANY positive hour that day?
asset_day_y = (
    scores.groupby(g_keys)["y_true"]
          .max()
          .reset_index()
          .rename(columns={"y_true": "y_asset_day"})
)

asset_day = asset_day.merge(asset_day_y, on=g_keys, how="left")

# Select Top-K assets per day
K = 5
alerts = (
    asset_day.sort_values(["date_utc", "p_hat"], ascending=[True, False])
            .groupby("date_utc", as_index=False)
            .head(K)
            .reset_index(drop=True)
)

precision_at_k = alerts["y_asset_day"].mean() if len(alerts) else np.nan
alerts_per_day = alerts.groupby("date_utc")["asset_id"].nunique().mean() if len(alerts) else 0.0

print("\nBudget policy:")
print(f"  K (assets/day)          : {K}")
print(f"  Avg assets/day selected : {alerts_per_day:.2f}")
print(f"  Precision@K (asset-day) : {precision_at_k:.3f}  (fraction of alerted asset-days with ≥1 positive hour)")

# -----------------------------
# 6) “Why”: top coefficient contributions at the peak hour
#    Logistic regression contribution: contrib_i = x_i * coef_i (transformed space)
# -----------------------------
coefs = clf.coef_.ravel()

def top_contribs_for_row(row_ix: int, top_n: int = 10) -> pd.DataFrame:
    row = X_test_tx.getrow(int(row_ix))  # 1 x n_features CSR
    if row.nnz == 0:
        return pd.DataFrame(columns=["feature", "x", "coef", "contrib"])

    idx = row.indices
    x = row.data
    c = coefs[idx]
    contrib = x * c

    dfc = pd.DataFrame({
        "feature": [feature_names[i] for i in idx],
        "x": x,
        "coef": c,
        "contrib": contrib,
        "abs_contrib": np.abs(contrib),
    }).sort_values("abs_contrib", ascending=False)

    return dfc.drop(columns=["abs_contrib"]).head(top_n)

drivers_long = []
for _, r in alerts.iterrows():
    row_ix = int(r["row_ix"])
    d = top_contribs_for_row(row_ix=row_ix, top_n=10).copy()

    # Keep ts_peak tz-aware UTC
    ts_peak = pd.to_datetime(r["ts_peak"], utc=True, errors="coerce")

    d.insert(0, "date_utc", r["date_utc"])
    d.insert(1, "asset_id", r["asset_id"])
    d.insert(2, "ts_peak", ts_peak)
    d.insert(3, "p_hat_peak", float(r["p_hat"]))
    d.insert(4, "y_asset_day", int(r["y_asset_day"]))
    drivers_long.append(d)

drivers_long_df = pd.concat(drivers_long, ignore_index=True) if drivers_long else pd.DataFrame()

# -----------------------------
# 7) Save outputs (to CURRENT OUT_DIR so this run stays coherent)
# -----------------------------
alerts_out  = OUT_DIR / "panel_alerts_top5_assets_per_day.csv"
drivers_out = OUT_DIR / "panel_alerts_top5_assets_per_day_drivers_long.csv"
metrics_out = OUT_DIR / "panel_precision_at_k_assets_per_day.json"

alerts.to_csv(alerts_out, index=False)
print("\nSaved:", alerts_out)

if not drivers_long_df.empty:
    drivers_long_df.to_csv(drivers_out, index=False)
    print("Saved:", drivers_out)
else:
    print("No drivers were generated (empty alerts or empty rows).")

metrics_payload = {
    "policy": "top_k_assets_per_day",
    "k": int(K),
    "loaded_panel_dir": str(PANEL_DIR),
    "written_out_dir": str(OUT_DIR),
    "n_alert_rows": int(len(alerts)),
    "n_days": int(alerts["date_utc"].nunique()) if len(alerts) else 0,
    "avg_assets_per_day": float(alerts_per_day),
    "precision_at_k_asset_day": float(precision_at_k) if precision_at_k == precision_at_k else None,
    "note": "precision@K computed on asset-day label: y_asset_day = max(y_true) over hours for that asset on that day (TEST only).",
}
metrics_out.write_text(json.dumps(metrics_payload, indent=2))
print("Saved:", metrics_out)

# -----------------------------
# 8) Display quick views
# -----------------------------
print("\nTop-5 assets/day (preview):")
display(alerts.head(20))

if not drivers_long_df.empty:
    print("\nTop drivers (preview, long form):")
    display(drivers_long_df.head(30))


Panel artifacts directory selected: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z

Loaded (from PANEL_DIR):
  X_test_tx : (8071, 76) | sparse: True
  y_test    : (8071,)
  ids_test  : (8071, 6)
  features  : 76

Test-set snapshot:
  Positive rate (hour-level): 0.075208
  Unique assets: 24
  Date range (UTC): 2025-11-27 00:00:00+00:00 → 2025-12-11 00:00:00+00:00

Budget policy:
  K (assets/day)          : 5
  Avg assets/day selected : 5.00
  Precision@K (asset-day) : 0.400  (fraction of alerted asset-days with ≥1 positive hour)

Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_alerts_top5_assets_per_day.csv
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/panel_alerts_top5_assets_per_day_drivers_long.csv
Saved: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineeri

,date_utc,asset_id,ts_peak,site_id,line_id,asset_type,is_legacy,y_true,p_hat,row_ix,y_asset_day
0,2025-11-27,A0105,2025-11-27 20:00:00+00:00,S2,S2-L4,case_packer,True,0,0.753193,7082,0
1,2025-11-27,A0046,2025-11-27 18:00:00+00:00,S4,S4-L4,blister_packer,True,0,0.731878,3383,0
2,2025-11-27,A0056,2025-11-27 19:00:00+00:00,S3,S3-L1,cartoner,True,0,0.693739,4056,0
3,2025-11-27,A0063,2025-11-27 16:00:00+00:00,S3,S3-L1,sterilizer,True,0,0.692411,4389,0
4,2025-11-27,A0037,2025-11-27 22:00:00+00:00,S1,S1-L3,capper,True,0,0.682174,2378,0
5,2025-11-28,A0046,2025-11-28 11:00:00+00:00,S4,S4-L4,blister_packer,True,1,0.999991,3400,1
6,2025-11-28,A0056,2025-11-28 13:00:00+00:00,S3,S3-L1,cartoner,True,1,0.999987,4074,1
7,2025-11-28,A0110,2025-11-28 04:00:00+00:00,S4,S4-L3,capper,True,1,0.999983,7763,1
8,2025-11-28,A0019,2025-11-28 05:00:00+00:00,S3,S3-L5,labeler,True,1,0.999980,1376,1
9,2025-11-28,A0045,2025-11-28 14:00:00+00:00,S2,S2-L2,labeler,True,1,0.999951,3067,1



Top drivers (preview, long form):


,date_utc,asset_id,ts_peak,p_hat_peak,y_asset_day,feature,x,coef,contrib
0,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,is_legacy,0.960621,0.656359,0.630512
1,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,asset_type_case_packer,1.000000,0.531498,0.531498
2,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,dow_utc_cos,-1.271777,0.399179,-0.507667
3,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,line_id_S2-L4,1.000000,0.363836,0.363836
4,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,is_weekend_utc,-0.631791,-0.496756,0.313846
5,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,site_id_S2,1.000000,-0.212079,-0.212079
6,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,dow_utc_sin,0.612969,-0.267724,-0.164107
7,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,telemetry_rows_hour,-1.202494,-0.110827,0.133269
8,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,humidity_rh_tele_max,-0.655394,-0.171087,0.112130
9,2025-11-27,A0105,2025-11-27 20:00:00+00:00,0.753193,0,humidity_rh_tele_min,0.584919,-0.138792,-0.081182


### What Cell 26 Just Did

This cell turns the **panel (asset-hour) model** into an **actionable alerting workflow** under a strict **Top-5 assets/day** budget.

- **Auto-finds the right panel artifacts after a kernel restart**
  - Scans `data/processed/feature_engineering/` and selects the **most recent run folder** that contains the required panel artifacts (`panel_X_test.npz`, `panel_y_test.npy`, `panel_ids_test.parquet`, feature names, and the saved model).
  - Prevents failures when `OUT_DIR` changes between runs.

- **Loads TEST-set artifacts and scores every asset-hour**
  - Loads `X_test` (sparse), `y_test`, `ids_test` (with `asset_id` + timestamp), feature names, and the trained Logistic Regression model.
  - Computes predicted probabilities (**p_hat**) for each test hour.

- **Enforces the alert budget: Top-5 assets/day**
  - Buckets each row into a **UTC day**.
  - For each **asset-day**, finds the **peak-risk hour** (max `p_hat`) and captures:
    - `ts_peak` (when risk peaks)
    - `p_hat` at the peak
    - `row_ix` (row pointer back into `X_test`)
  - Selects the **Top 5 assets per day** by peak `p_hat`.
  - Computes **Precision@K (asset-day)** where `y_asset_day = max(y_true)` across hours for that asset on that day.

- **Adds “why” for each alert (traceable drivers)**
  - For each alert, pulls the peak-hour feature row and computes per-feature contribution:
    - `contrib_i = x_i * coef_i` (transformed feature space)
  - Produces a long-form table of top drivers so each alert has a clear **“why”**.

- **Exports business-ready outputs**
  - `panel_alerts_top5_assets_per_day.csv` — daily Top-5 alert list with “when/where/context”
  - `panel_alerts_top5_assets_per_day_drivers_long.csv` — top coefficient contributions per alert (why)
  - `panel_precision_at_k_assets_per_day.json` — budget policy + Precision@K summary (with provenance)


In [30]:
#============================================================
# Cell 27 — Notebook wrap-up: append “report exports” to EXPORTS.json
#============================================================

exports_path = OUT_DIR / "EXPORTS.json"
exports = json.loads(Path(exports_path).read_text()) if exports_path.exists() else {}

# Add report outputs + time-window artifacts
exports_updates = {
    "EVENT_MODEL_TABLE_TIMEWINDOWS": str((OUT_DIR / "event_derived_model_table_timewindows.parquet").resolve()),
    "EVENT_PREPROCESS_TIMEWINDOWS": str((OUT_DIR / "preprocess_event_timewindows.joblib").resolve()),
    "EVENT_BASELINE_METRICS_TIMEWINDOWS": str((OUT_DIR / "baseline_logreg_event_timewindows_metrics.json").resolve()),
    "EVENT_COEFFICIENTS_TIMEWINDOWS": str((OUT_DIR / "baseline_logreg_event_timewindows_coefficients.csv").resolve()),
    "THRESHOLD_TRADEOFFS_TIMEWINDOWS": str((OUT_DIR / "threshold_tradeoffs_event_timewindows.csv").resolve()),
    "SCORED_ASSET_RISK_TIMEWINDOWS": str((OUT_DIR / "asset_risk_scored_timewindows.csv").resolve()),
    "TOP_DRIVERS_TIMEWINDOWS": str((OUT_DIR / "top_drivers_timewindows.csv").resolve()),
    "REPORT_SITE_SUMMARY": str((OUT_DIR / "report_site_summary.csv").resolve()),
    "REPORT_LINE_SUMMARY": str((OUT_DIR / "report_line_summary.csv").resolve()),
    "REPORT_TOP20_ASSETS": str((OUT_DIR / "report_top20_assets.csv").resolve()),
}

exports.update(exports_updates)

with open(exports_path, "w") as f:
    json.dump(exports, f, indent=2)

print("Updated:", exports_path)
print("\nAdded keys:")
for k in exports_updates:
    print(" -", k)


Updated: /home/parallels/projects/gmp-packaging-risk-analytics/data/processed/feature_engineering/20251220T211621Z/EXPORTS.json

Added keys:
 - EVENT_MODEL_TABLE_TIMEWINDOWS
 - EVENT_PREPROCESS_TIMEWINDOWS
 - EVENT_BASELINE_METRICS_TIMEWINDOWS
 - EVENT_COEFFICIENTS_TIMEWINDOWS
 - THRESHOLD_TRADEOFFS_TIMEWINDOWS
 - SCORED_ASSET_RISK_TIMEWINDOWS
 - TOP_DRIVERS_TIMEWINDOWS
 - REPORT_SITE_SUMMARY
 - REPORT_LINE_SUMMARY
 - REPORT_TOP20_ASSETS


### What Cell 27 Just Did

This final wrap-up cell updates the notebook’s **export manifest** so downstream readers (or automation) can quickly find the key deliverables produced in this run.

- **Loads or creates** `EXPORTS.json` in the current run output directory.
- **Appends a “report exports” section** that enumerates the important, business-facing artifacts generated across the notebook (for example: scored tables, threshold/triage summaries, alert budget outputs, driver tables, and metrics snapshots).
- **De-duplicates entries** so rerunning the notebook doesn’t create repeated items.
- **Writes the updated manifest back to disk**, leaving a single, canonical index of what to look at and where it lives.

**Saved artifact:** `EXPORTS.json` (updated with report-ready outputs for this run)


## Notebook Summary — `02_feature_engineering_and_labels.ipynb`

This notebook builds **model-ready feature tables and labels** from raw IoT event streams for GMP packaging assets, then runs **baseline models** and produces **actionable risk outputs** (scores, thresholds, and alert lists) with full artifact traceability.

### Goals
- Standardize and enrich raw IoT events (telemetry + incidents) with master data (assets/sites).
- Create **forecastable labels** (future incidents) to avoid leakage-by-construction.
- Produce two modeling views:
  1) **Asset-level (time-split)**: early-window telemetry → late-window incident label  
  2) **Panel (asset-hour)**: hourly telemetry features → “future incident within next H hours” label
- Train baseline Logistic Regression models and export business-facing rollups:
  - predicted risk tables
  - threshold/triage summaries
  - **alerts under a fixed budget** (Top 5 assets/day) with “when” + “why”

### What’s Built
**1) Clean events layer**
- Parses/standardizes the IoT events schema and timestamps.
- Adds local time fields and joins asset/site context.
- Persists a normalized parquet for reproducible downstream steps.

**2) Panel dataset (asset-hour forecasting)**
- Aggregates telemetry into **asset-hour** features (counts + summary stats by metric).
- Builds a **forward-looking label** (`target_future_incident`) indicating whether an incident occurs within the next horizon window.
- Adds safe time features (hour-of-day / day-of-week + cyclic encodings).
- Uses a **group split by `asset_id`** so an asset never appears in both train and test.

**3) Asset-level time-split dataset (anti-leak baseline)**
- EARLY telemetry window features per asset.
- LATE window incident totals → future incident label (`target_event_future`).
- Produces a compact 120-row asset table for a simple “predict future incidents” task.

### Modeling & Evaluation (High-Level)
- Baseline models use **LogisticRegression(saga)** with `class_weight='balanced'`.
- The notebook explicitly detects and avoids label leakage:
  - Demonstrates why “label-by-construction” targets can look artificially perfect.
  - Replaces those with time-forward labels for a realistic signal test.
- Panel model outputs include:
  - ROC-AUC / PR-AUC metrics
  - threshold curve and triage tables (FP/FN lists)
  - per-row contributions for interpretability

### Operational Output (Actionability)
- Implements an alerting policy: **Top 5 assets/day** (TEST) ranked by max predicted probability over hours.
- For each alerted asset-day, exports:
  - **when**: `ts_peak` (hour of max risk)
  - **why**: top coefficient-based contributions for that peak hour (transparent, traceable)

### Key Saved Artifacts (Run Folder)
Artifacts are written under the run-stamped output directory (e.g., `data/processed/feature_engineering/<RUN_ID>/`), including:
- Panel dataset + columns:  
  - `panel_asset_hour_future_incident.parquet`, `panel_feature_columns.json`
- Split/preprocess artifacts:  
  - `panel_preprocess.joblib`, `panel_feature_names.csv`, `panel_X_train.npz`, `panel_X_test.npz`, `panel_y_train.npy`, `panel_y_test.npy`, `panel_split_manifest.json`
- Panel model outputs:  
  - `panel_baseline_logreg_saga.joblib`, metrics JSON(s), threshold curve CSV, triage tables, scored tables
- Budgeted alert outputs:  
  - `panel_alerts_top5_assets_per_day.csv`  
  - `panel_alerts_top5_assets_per_day_drivers_long.csv`  
  - `panel_precision_at_k_assets_per_day.json`
- Notebook summary and export index:  
  - `RUN_SUMMARY.md`, `EXPORTS.json`

### How to Read the Results
- Start with `RUN_SUMMARY.md` for the concise run recap.
- Use `panel_alerts_top5_assets_per_day.csv` for the daily action list.
- Use `panel_alerts_top5_assets_per_day_drivers_long.csv` to explain *why* an alert fired at the peak hour.
- Use triage tables (`panel_false_positives_*`, `panel_false_negatives_*`) to understand failure modes and improve policy.

### Next Steps
- Tune the operational policy using business constraints (e.g., alert budget already fixed at **Top 5 assets/day**):
  - evaluate stability across days/sites/lines
  - add suppress/hold-down logic (e.g., avoid alerting the same asset repeatedly unless risk rises)
- Improve signal:
  - richer temporal features (lags, deltas, rolling windows per metric)
  - consider a non-linear baseline (tree-based model) while keeping the same leak-safe split rules
- Define validation aligned to operations:
  - precision@K per day, alerts/day, and downstream QA workload impact
